In [1]:
""" 
Packages
"""
import collections
import itertools
import logging
import os
import random
import re
import sys
import time
import matplotlib
import shutil
import glob
import traceback
import statsmodels

from gurobipy import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.patches import Patch
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.stats import trim_mean
from collections import Counter
import functools as ft
import seaborn as sns
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde

%matplotlib widget

In [2]:
class Cargo_Item():
    def __init__(self, index, serialnumber, number_of_pieces, weight, CRT, COL, dangerous, commodity):
        self.index = index 
        self.serialnumber = serialnumber
        self.prefix_serialnumber = str(serialnumber).split('-')[0] if '-' in str(serialnumber) else str(serialnumber)
        self.number_of_pieces = number_of_pieces
        self.weight = weight
        self.volume = None
        self.CRT = CRT
        self.COL = COL
        self.dangerous = dangerous
        self.height = None
        self.length = None
        self.rotate = None
        self.stack = None
        self.width = None
        self.position = None
        self.density = None
        self.commodity = commodity

In [3]:
class ULD():
    def __init__(self, index, type, serialnumber):
        self.index = index
        self.type = type
        self.serialnumber = serialnumber
        self.height = None
        self.length = None
        self.width = None
        self.cost = None
        self.cut_a = None
        self.cut_b = None
        self.a = None
        self.weight = None
        self.max_weight = None
        self.weight = 0
        self.isNeitherBAXnorBUPnorT = 'BAX' not in serialnumber and 'BUP' not in serialnumber and 'T' not in serialnumber
        self.isBAXorBUPorT = 'BAX' in serialnumber or 'BUP' in serialnumber or 'T' in serialnumber
        self.isBAX = 'BAX' in serialnumber
        self.CRT = 0
        self.COL = 0
        self.actual_position_bax = None

In [4]:
class LoadLocations():
    def __init__(self, index, location):
        self.index = index
        self.location = location

In [5]:
class Cargo():
    def __init__(self):
        self.piece_filename = None
        self.cargo = []
        self.items = None
        self.uld = []
        self.uld_cut = []
        self.uld_nocut = []
        self.uld_bpp = []
        self.BUP_data = None
        self.total_number_of_build_ULDs = None

    def read_cargo_pieces(self, filename_cargo_pieces, filename_buildup_information, arrival_airport):
        """
        Reads and processes the cargo pieces from a csv file, removing the duplicates and creating a list of Cargo_Item objects.
        Making invidual items for serialnumber with multiple pieces
        Assigning the dimensions and characteristics of the cargo items to the Cargo_Item objects.

        Args:
            filename (str): The path to the CSV file containing cargo data.
        """
        data_analysis = Data_Analysis()
        #Reading the file and removing duplicates
        self.piece_filename = filename_cargo_pieces
        data = pd.read_csv(filename_cargo_pieces)
        data_deindividualize = data.copy()
        data.drop_duplicates(inplace = True)
        self.BUP_data = data[data['IsBUP'] == 1]
        data = data[data['IsBUP'] != 1]

        '''Removing items if airwaybill both not in buildup and pieceinfo [TEST --> WOUTER]'''
        data_buildup_information = pd.read_csv(filename_buildup_information) 
        build_up_serialnumbers = {re.sub(r'^74[0]*|-.*$', '', str(row['AirWaybillNumber']))
                          for _, row in data_buildup_information.iterrows()}
        str_airwaybillserialnumber = data['BookingAirWaybillSerialNumber'].astype(str)
        data = data[str_airwaybillserialnumber.isin(build_up_serialnumbers)]

        #Removing the items that are part of a T-ULD
        df_T_uld = data_buildup_information.groupby('ULD').filter(lambda x: not x['IsBuildUpInVG3'].any())
        short_serials = [re.sub(r'^74[0]*|-.*$', '', str(row['AirWaybillNumber'])) for _, row in df_T_uld.iterrows()]
        str_airwaybillserialnumber = data['BookingAirWaybillSerialNumber'].astype(str)
        data = data[~str_airwaybillserialnumber.isin(short_serials)]


        #Grouping the data by the serial number and aggregating the characteristics of the cargo items
        grouped_data = data.groupby('BookingAirWaybillSerialNumber').agg({
            'BookingSegmentPieceCount': 'first',
            'BookingSegmentWeight': 'first',
            'IsCRT': 'max',
            'IsCOL': 'max',
            'IsDangerousGoods': 'max', 
            'BookingCommodityCode': 'first'
        }).reset_index()

        #Creating a list of Cargo_Item objects
        for key, row in grouped_data.iterrows():
                self.cargo.append(Cargo_Item(key, int(row.BookingAirWaybillSerialNumber), int(row.BookingSegmentPieceCount), float(row.BookingSegmentWeight), 
                                        row.IsCRT, row.IsCOL, row.IsDangerousGoods, row.BookingCommodityCode))

        #Creating individual items for serialnumber with multiple pieces and assigning them with a new serialnumber
        individual_items = []
        key = 0
        for i in self.cargo: 
            if i.number_of_pieces > 1:
                individual_weight = float(i.weight / i.number_of_pieces)
                for n in range(i.number_of_pieces):
                    new_serial = f"{i.serialnumber}-{n+1}"
                    individual_item = Cargo_Item(i.index, new_serial, 1, individual_weight, i.CRT, i.COL, i.dangerous, i.commodity)
                    individual_items.append(individual_item)
            else:
                individual_items.append(i)

        individual = []
        for key, i in enumerate(individual_items):
            indivi = Cargo_Item(key, i.serialnumber, i.number_of_pieces, i.weight, i.CRT, i.COL, i.dangerous, i.commodity)
            individual.append(indivi)
        self.items = individual

        data['BookingLinePieceHeight'] = pd.to_numeric(data['BookingLinePieceHeight'], errors='coerce')
        data['BookingLinePieceWidth'] = pd.to_numeric(data['BookingLinePieceWidth'], errors='coerce')
        data['BookingLinePieceLength'] = pd.to_numeric(data['BookingLinePieceLength'], errors='coerce')

        #Assigning the dimensions and characteristics of the cargo items to the Cargo_Item objects
        for i in self.items:
            serial_str = str(i.serialnumber)
            if '-' in serial_str:
                index_piece = int(serial_str.split('-')[1]) - 1
            else:
                index_piece = 0
            short_serial = re.sub(r'^74[0]*|-.*$', '', serial_str)

            relevant_rows = data[data['BookingAirWaybillSerialNumber'] == int(short_serial)]

            filtered_rows = relevant_rows[relevant_rows['BookingLinePieceIsInformational'] == True]

            if not filtered_rows.empty:
                heights = []
                widths = []
                lengths = []
                rotatables = []
                stackables = []

                for _, row in filtered_rows.iterrows():
                    n = int(row['BookingSegmentPiecesCount'])
                    heights += n * [row['BookingLinePieceHeight']]
                    widths += n * [row['BookingLinePieceWidth']]
                    lengths += n * [row['BookingLinePieceLength']]
                    rotatables += n * [row['BookingSegmentPiecesTurnable']]
                    stackables += n * [row['BookingSegmentPiecesStackable']]

                if index_piece < len(heights):
                    i.height = int(heights[index_piece])
                    i.width = int(widths[index_piece])
                    i.length = int(lengths[index_piece])
                    i.volume = i.height * i.length * i.width
                    i.density = i.weight / i.volume
                    if rotatables[index_piece] == True:
                        i.rotate = 1
                    else:
                        i.rotate = 0
                    if stackables[index_piece] == True:
                        i.stack = 0
                    else:
                        i.stack = 1

            else:
                if not relevant_rows.empty:
                    row = relevant_rows.iloc[0]
                    i.volume = int((row['BookingSegmentVolume'] * 1000000) / row['BookingSegmentPieceCount'])
                    i.height = int((i.volume ** (1/3)) * data_analysis.dimensions_proportions_per_commodity(arrival_airport, i.commodity, 'HeightProportion'))
                    i.width = int((i.volume ** (1/3)) * data_analysis.dimensions_proportions_per_commodity(arrival_airport, i.commodity, 'WidthProportion'))
                    i.length = int((i.volume ** (1/3)) * data_analysis.dimensions_proportions_per_commodity(arrival_airport, i.commodity, 'LengthProportion'))
                    i.density = i.weight / i.volume
                    i.rotate = 1
                    i.stack = 0

        corrected_data = []
        for i in self.items:
            if i.height is not None:
                corrected_data.append(i)
        self.items = corrected_data

        return
    


    def define_parameters_ULD(self):
        """
        Define the parameters of the ULD types
        """
        for i in self.uld:
            if i.type == 'PMC':
                i.max_weight = int(5102)
                i.volume = int(12.5 * 1000000)
                i.height = int(1.62 * 100)
                i.length = int(3.18 * 100)
                i.width = int(2.44 * 100)
                i.a = 0
                i.b = 0
                i.cut_a = 0

            if i.type == 'AKE':
                i.max_weight = int(1587)
                i.volume = int(2.8 * 1000000)
                i.height = int(1.62 * 100)
                i.length = int(1.56 * 100)
                i.width = int(1.53 * 100)
                i.a = 42
                i.b = 53
                i.cut_a = i.b / i.a

            if i.type == 'PAG':
                i.max_weight = int(4676)
                i.volume = int(11.4 * 1000000)
                i.height = int(1.62 * 100)
                i.length = int(3.18 * 100)
                i.width = int(2.24 * 100)
                i.a = 0
                i.b = 0
                i.cut_a = 0


    def define_uld_cut_sets(self):
        """
        Define the set of ULDs that have a cut and the set of ULDs that do not have a cut
        """
        for i in self.uld:
            if i.type == 'PMC' or i.type == 'PAG':
                self.uld_nocut.append(i)
            if i.type == 'AKE':
                self.uld_cut.append(i)


    def define_ulds_used_flight(self, filename_load_locations, filename_buildup_information):
        """
        Define the ULDs used in the flight

        Args:
            filename (str): The path to the Excel file containing the ULD data.
        """
        number_of_AKE = 0
        number_of_PMC = 0
        number_of_PAG = 0
        filtered_set_of_ulds = []
        weight_bax = []
        positions_bax = []
        serialnumber_list = []

        #Reading the file and creating a list of ULD objects
        data_loadlocations = pd.read_csv(filename_load_locations)
        special_handling_filter = data_loadlocations['SpecialHandlingCode'].isin(['COL', 'CRT'])
        serialnumber_with_special_handling = data_loadlocations[special_handling_filter][['SerialNumber', 'SpecialHandlingCode']]
        special_handling_dict = serialnumber_with_special_handling.set_index('SerialNumber')['SpecialHandlingCode'].to_dict()

        data_buildup_information = pd.read_csv(filename_buildup_information)
        unique_buildup_information = data_buildup_information.drop_duplicates(subset=['ULD', 'AirWaybillNumber'])
        df_T_uld = unique_buildup_information.groupby('ULD').filter(lambda x: not x['IsBuildUpInVG3'].any())
        str_airwaybillserialnumber = df_T_uld['AirWaybillNumber'].astype(str)
        df_T_uld = df_T_uld[~ str_airwaybillserialnumber.isin(self.BUP_data['BookingAirWaybillNumber'].astype(str))]
        df_T_uld_information = pd.DataFrame()
        df_T_uld_information['SerialNumber'] = df_T_uld['ULD'].astype(str)
        df_T_uld_information = pd.merge(df_T_uld_information, data_loadlocations[['SerialNumber', 'Weight']], on='SerialNumber', how='left')
        df_T_uld_information = df_T_uld_information.dropna(subset=['Weight'])

        data_loadlocations = data_loadlocations[~data_loadlocations['SerialNumber'].isin(df_T_uld_information['SerialNumber'])]
        self.data_loadlocations = data_loadlocations

        for row_number, row in enumerate(data_loadlocations.itertuples(index=False), start=0):
            serialnumber = row.SerialNumber
            if pd.notnull(serialnumber):
                if row.DeadloadType == 'C' and 'AKE' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_AKE += 1
                    uld_type = 'AKE'
                    serial = f'{uld_type}-{row_number}'
                    filtered_set_of_ulds.append(ULD(row_number, uld_type, serial))
                    serialnumber_list.append(serialnumber)

                elif row.DeadloadType == 'C' and 'AKY' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_AKE += 1
                    uld_type = 'AKE'
                    serial = f'{uld_type}-{row_number}'
                    filtered_set_of_ulds.append(ULD(row_number, uld_type, serial))
                    serialnumber_list.append(serialnumber)

                elif row.DeadloadType == 'C' and 'RKN' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_AKE += 1
                    uld_type = 'AKE'
                    serial = f'{uld_type}-{row_number}'
                    filtered_set_of_ulds.append(ULD(row_number, uld_type, serial))
                    serialnumber_list.append(serialnumber)
                
                elif row.DeadloadType == 'C' and 'AKN' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_AKE += 1
                    uld_type = 'AKE'
                    serial = f'{uld_type}-{row_number}'
                    filtered_set_of_ulds.append(ULD(row_number, uld_type, serial))
                    serialnumber_list.append(serialnumber)

                elif row.DeadloadType == 'C' and 'PMC' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_PMC += 1
                    uld_type = 'PMC'
                    serial = f'{uld_type}-{row_number}'
                    filtered_set_of_ulds.append(ULD(row_number, uld_type, serial))
                    serialnumber_list.append(serialnumber)

                elif row.DeadloadType == 'C' and 'AAP' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_PMC += 1
                    uld_type = 'PMC'
                    serial = f'{uld_type}-{row_number}'
                    filtered_set_of_ulds.append(ULD(row_number, uld_type, serial))
                    serialnumber_list.append(serialnumber)

                elif row.DeadloadType == 'C' and 'PAG' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_PAG += 1
                    uld_type = 'PAG'
                    serial = f'{uld_type}-{row_number}'
                    filtered_set_of_ulds.append(ULD(row_number, uld_type, serial))
                    serialnumber_list.append(serialnumber)

                elif row.DeadloadType == 'C' and 'PLB' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_PAG += 1
                    uld_type = 'PAG'
                    serial = f'{uld_type}-{row_number}'
                    filtered_set_of_ulds.append(ULD(row_number, uld_type, serial))
                    serialnumber_list.append(serialnumber)

                elif row.DeadloadType == 'C' and 'PLA' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_PAG += 1
                    uld_type = 'PAG'
                    serial = f'{uld_type}-{row_number}'
                    filtered_set_of_ulds.append(ULD(row_number, uld_type, serial))
                    serialnumber_list.append(serialnumber)

                elif row.DeadloadType == 'B' and 'AKE' in serialnumber and serialnumber not in serialnumber_list:
                    uld_type = 'AKE'
                    name_bax = 'BAX'
                    serial = f'{name_bax}-{row_number}'
                    filtered_set_of_ulds.append(ULD(row_number, uld_type, serial))
                    weight_bax.append(row.UldGrossWeight)
                    positions_bax.append(row.LoadLocation)
                    serialnumber_list.append(serialnumber)

        self.uld = filtered_set_of_ulds

        #Assigning the weight of the BAX ULDs
        bax_index = 0
        for j in self.uld:
            if 'BAX' in j.serialnumber:
                j.weight = weight_bax[bax_index]
                j.actual_position_bax = positions_bax[bax_index]
                bax_index += 1

        #Assigning the weight and number of the BUP ULDs
        if len(self.BUP_data) > 0:
            for key, row_bup in self.BUP_data.iterrows():
                row_number = max(uld.index for uld in self.uld) + 1
                uld_type = 'PMC'
                name_bax = 'BUP'
                serial = f'{name_bax}-{row_number}'
                self.uld.append(ULD(row_number, uld_type, serial))

        for j in self.uld:
            if 'BUP' in j.serialnumber:
                j.weight = self.BUP_data['BookingTotalWeight'].max()

        #Assigning T-ULDs
        serialnumber_list = []
        if len(df_T_uld_information) > 0:
            for index, row in df_T_uld_information.iterrows():
                uld = row['SerialNumber']
                weight = row['Weight']
                COL = False
                CRT = False

                if uld not in serialnumber_list and ((str(uld[0]) != 'B') and (str(uld[0]) != 'K')):
                    row_number = max(uld.index for uld in self.uld) + 1
                    uld_type = str(uld[0:3])

                    if uld in special_handling_dict:
                        if special_handling_dict[uld] == 'COL':
                            COL = True
                        if special_handling_dict[uld] == 'CRT':
                            CRT = True

                    if uld_type not in ['PMC', 'PAG', 'AKE']:
                        if uld_type == 'PLB':
                            uld_type = 'PAG'
                        elif uld_type == 'PLA':
                            uld_type = 'PAG'
                        elif uld_type == 'AAP':
                            uld_type = 'PMC'
                        elif uld_type == 'RKN':
                            uld_type = 'AKE'
                        elif uld_type == 'AKY':
                            uld_type = 'AKE'
                        elif uld_type == 'AKN':
                            uld_type = 'AKE'
                        else:
                            uld_type = 'AKE'
                        
                    name_T = 'T'
                    serial = f'{name_T}-{row_number}'
                    T_uld = ULD(row_number, uld_type, serial)
                    if COL:
                        T_uld.COL = 1
                    if CRT:
                        T_uld.CRT = 1
                    T_uld.weight = weight
                    self.uld.append(T_uld)
                    serialnumber_list.append(uld)

    def filter_items_test(self, amount):
        """
        Filter the items to a specific number of items

        Args:
            amount (int): The number of items to filter
        """
        filter_items = random.sample(self.items, amount)
        self.items = filter_items

        # filter_items = self.items[:amount]
        # self.items = filter_items


    def get_prefix_groups(self):
        """
        Get the serialnumber prefix groups of the items
        """
        prefix_groups = {}
        for i in self.items:
            if i.prefix_serialnumber not in prefix_groups:
                prefix_groups[i.prefix_serialnumber] = []
            prefix_groups[i.prefix_serialnumber].append(i)

        return prefix_groups

In [6]:
class CargoActual():
    def __init__(self):
        self.piece_filename = None
        self.cargo = []
        self.items = None
        self.uld = []
        self.uld_cut = []
        self.uld_nocut = []
        self.uld_bpp = []
        self.BUP_data = None
        self.total_number_of_build_ULDs = None

    def read_cargo_pieces(self, filename_cargo_pieces, filename_buildup_information, arrival_airport):
        """
        Reads and processes the cargo pieces from a csv file, removing the duplicates and creating a list of Cargo_Item objects.
        Making invidual items for serialnumber with multiple pieces
        Assigning the dimensions and characteristics of the cargo items to the Cargo_Item objects.

        Args:
            filename (str): The path to the CSV file containing cargo data.
        """
        data_analysis = Data_Analysis()
        #Reading the file and removing duplicates
        self.piece_filename = filename_cargo_pieces
        data = pd.read_csv(filename_cargo_pieces)
        data_deindividualize = data.copy()
        data.drop_duplicates(inplace = True)
        self.BUP_data = data[data['IsBUP'] == 1]
        data = data[data['IsBUP'] != 1]

        '''Removing items if airwaybill both not in buildup and pieceinfo [TEST --> WOUTER]'''
        data_buildup_information = pd.read_csv(filename_buildup_information) 
        build_up_serialnumbers = {re.sub(r'^74[0]*|-.*$', '', str(row['AirWaybillNumber']))
                          for _, row in data_buildup_information.iterrows()}
        str_airwaybillserialnumber = data['BookingAirWaybillSerialNumber'].astype(str)
        data = data[str_airwaybillserialnumber.isin(build_up_serialnumbers)]

        #Removing the items that are part of a T-ULD
        df_T_uld = data_buildup_information.groupby('ULD').filter(lambda x: not x['IsBuildUpInVG3'].any())
        short_serials = [re.sub(r'^74[0]*|-.*$', '', str(row['AirWaybillNumber'])) for _, row in df_T_uld.iterrows()]
        str_airwaybillserialnumber = data['BookingAirWaybillSerialNumber'].astype(str)
        data = data[~str_airwaybillserialnumber.isin(short_serials)]


        #Grouping the data by the serial number and aggregating the characteristics of the cargo items
        grouped_data = data.groupby('BookingAirWaybillSerialNumber').agg({
            'BookingSegmentPieceCount': 'first',
            'BookingSegmentWeight': 'first',
            'IsCRT': 'max',
            'IsCOL': 'max',
            'IsDangerousGoods': 'max', 
            'BookingCommodityCode': 'first'
        }).reset_index()

        #Creating a list of Cargo_Item objects
        for key, row in grouped_data.iterrows():
                self.cargo.append(Cargo_Item(key, int(row.BookingAirWaybillSerialNumber), int(row.BookingSegmentPieceCount), float(row.BookingSegmentWeight), 
                                        row.IsCRT, row.IsCOL, row.IsDangerousGoods, row.BookingCommodityCode))

        #Creating individual items for serialnumber with multiple pieces and assigning them with a new serialnumber
        individual_items = []
        key = 0
        for i in self.cargo: 
            if i.number_of_pieces > 1:
                individual_weight = float(i.weight / i.number_of_pieces)
                for n in range(i.number_of_pieces):
                    new_serial = f"{i.serialnumber}-{n+1}"
                    individual_item = Cargo_Item(i.index, new_serial, 1, individual_weight, i.CRT, i.COL, i.dangerous, i.commodity)
                    individual_items.append(individual_item)
            else:
                individual_items.append(i)

        individual = []
        for key, i in enumerate(individual_items):
            indivi = Cargo_Item(key, i.serialnumber, i.number_of_pieces, i.weight, i.CRT, i.COL, i.dangerous, i.commodity)
            individual.append(indivi)
        self.items = individual

        data['BookingLinePieceHeight'] = pd.to_numeric(data['BookingLinePieceHeight'], errors='coerce')
        data['BookingLinePieceWidth'] = pd.to_numeric(data['BookingLinePieceWidth'], errors='coerce')
        data['BookingLinePieceLength'] = pd.to_numeric(data['BookingLinePieceLength'], errors='coerce')

        #Assigning the dimensions and characteristics of the cargo items to the Cargo_Item objects
        for i in self.items:
            serial_str = str(i.serialnumber)
            if '-' in serial_str:
                index_piece = int(serial_str.split('-')[1]) - 1
            else:
                index_piece = 0
            short_serial = re.sub(r'^74[0]*|-.*$', '', serial_str)

            relevant_rows = data[data['BookingAirWaybillSerialNumber'] == int(short_serial)]

            filtered_rows = relevant_rows[relevant_rows['BookingLinePieceIsInformational'] == True]

            if not filtered_rows.empty:
                heights = []
                widths = []
                lengths = []
                rotatables = []
                stackables = []

                for _, row in filtered_rows.iterrows():
                    n = int(row['BookingSegmentPiecesCount'])
                    heights += n * [row['BookingLinePieceHeight']]
                    widths += n * [row['BookingLinePieceWidth']]
                    lengths += n * [row['BookingLinePieceLength']]
                    rotatables += n * [row['BookingSegmentPiecesTurnable']]
                    stackables += n * [row['BookingSegmentPiecesStackable']]

                if index_piece < len(heights):
                    i.height = int(heights[index_piece])
                    i.width = int(widths[index_piece])
                    i.length = int(lengths[index_piece])
                    i.volume = i.height * i.length * i.width
                    i.density = i.weight / i.volume
                    if rotatables[index_piece] == True:
                        i.rotate = 1
                    else:
                        i.rotate = 0
                    if stackables[index_piece] == True:
                        i.stack = 0
                    else:
                        i.stack = 1

            else:
                if not relevant_rows.empty:
                    row = relevant_rows.iloc[0]
                    i.volume = int((row['BookingSegmentVolume'] * 1000000) / row['BookingSegmentPieceCount'])
                    i.height = int((i.volume ** (1/3)) * data_analysis.dimensions_proportions_per_commodity(arrival_airport, i.commodity, 'HeightProportion'))
                    i.width = int((i.volume ** (1/3)) * data_analysis.dimensions_proportions_per_commodity(arrival_airport, i.commodity, 'WidthProportion'))
                    i.length = int((i.volume ** (1/3)) * data_analysis.dimensions_proportions_per_commodity(arrival_airport, i.commodity, 'LengthProportion'))
                    i.density = i.weight / i.volume
                    i.rotate = 1
                    i.stack = 0

        corrected_data = []
        for i in self.items:
            if i.height is not None:
                corrected_data.append(i)
        self.items = corrected_data

        return

    
    def define_parameters_ULD(self):
        """
        Define the parameters of the ULD types
        """
        for i in self.uld:
            if i.type == 'PMC':
                i.max_weight = int(5102)
                i.volume = int(12.5 * 1000000)
                i.height = int(1.62 * 100)
                i.length = int(3.18 * 100)
                i.width = int(2.44 * 100)
                i.a = 0
                i.b = 0
                i.cut_a = 0

            if i.type == 'AKE':
                i.max_weight = int(1587)
                i.volume = int(2.8 * 1000000)
                i.height = int(1.62 * 100)
                i.length = int(1.56 * 100)
                i.width = int(1.53 * 100)
                i.a = 42
                i.b = 53
                i.cut_a = i.b / i.a

            if i.type == 'PAG':
                i.max_weight = int(4676)
                i.volume = int(11.4 * 1000000)
                i.height = int(1.62 * 100)
                i.length = int(3.18 * 100)
                i.width = int(2.24 * 100)
                i.a = 0
                i.b = 0
                i.cut_a = 0


    def define_uld_cut_sets(self):
        """
        Define the set of ULDs that have a cut and the set of ULDs that do not have a cut
        """
        for i in self.uld:
            if i.type == 'PMC' or i.type == 'PAG':
                self.uld_nocut.append(i)
            if i.type == 'AKE':
                self.uld_cut.append(i)


    def define_ulds_used_flight(self, filename_load_locations, filename_buildup_information):
        """
        Define the ULDs used in the flight

        Args:
            filename (str): The path to the Excel file containing the ULD data.
        """
        number_of_AKE = 0
        number_of_PMC = 0
        number_of_PAG = 0
        filtered_set_of_ulds = []
        weight_bax = []
        serialnumber_list = []

        #Reading the file and creating a list of ULD objects
        data_loadlocations = pd.read_csv(filename_load_locations)
        special_handling_filter = data_loadlocations['SpecialHandlingCode'].isin(['COL', 'CRT'])
        serialnumber_with_special_handling = data_loadlocations[special_handling_filter][['SerialNumber', 'SpecialHandlingCode']]
        special_handling_dict = serialnumber_with_special_handling.set_index('SerialNumber')['SpecialHandlingCode'].to_dict()


        data_buildup_information = pd.read_csv(filename_buildup_information)
        unique_buildup_information = data_buildup_information.drop_duplicates(subset=['ULD', 'AirWaybillNumber'])
        df_T_uld = unique_buildup_information.groupby('ULD').filter(lambda x: not x['IsBuildUpInVG3'].any())
        str_airwaybillserialnumber = df_T_uld['AirWaybillNumber'].astype(str)
        df_T_uld = df_T_uld[~ str_airwaybillserialnumber.isin(self.BUP_data['BookingAirWaybillNumber'].astype(str))]
        df_T_uld_information = pd.DataFrame()
        df_T_uld_information['SerialNumber'] = df_T_uld['ULD'].astype(str)
        df_T_uld_information = pd.merge(df_T_uld_information, data_loadlocations[['SerialNumber', 'Weight']], on='SerialNumber', how='left')
        df_T_uld_information = df_T_uld_information.dropna(subset=['Weight'])

        data_loadlocations = data_loadlocations[~data_loadlocations['SerialNumber'].isin(df_T_uld_information['SerialNumber'])]

        for row_number, row in enumerate(data_loadlocations.itertuples(index=False), start=0):
            serialnumber = row.SerialNumber

            if pd.notnull(serialnumber):
                if row.DeadloadType == 'C' and 'AKE' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_AKE += 1
                    uld_type = 'AKE'
                    serial = f'{uld_type}-{row_number}'
                    uld = ULD(row_number, uld_type, serial)
                    uld.weight = row.UldGrossWeight
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'COL'):
                        uld.COL = 1
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'CRT'):
                        uld.CRT = 1
                    filtered_set_of_ulds.append(uld)
                    serialnumber_list.append(serialnumber)

                elif row.DeadloadType == 'C' and 'AKY' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_AKE += 1
                    uld_type = 'AKE'
                    serial = f'{uld_type}-{row_number}'
                    uld = ULD(row_number, uld_type, serial)
                    uld.weight = row.UldGrossWeight
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'COL'):
                        uld.COL = 1
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'CRT'):
                        uld.CRT = 1
                    filtered_set_of_ulds.append(uld)
                    serialnumber_list.append(serialnumber)

                elif row.DeadloadType == 'C' and 'RKN' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_AKE += 1
                    uld_type = 'AKE'
                    serial = f'{uld_type}-{row_number}'
                    uld = ULD(row_number, uld_type, serial)
                    uld.weight = row.UldGrossWeight
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'COL'):
                        uld.COL = 1
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'CRT'):
                        uld.CRT = 1
                    filtered_set_of_ulds.append(uld)
                    serialnumber_list.append(serialnumber)
                
                elif row.DeadloadType == 'C' and 'AKN' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_AKE += 1
                    uld_type = 'AKE'
                    serial = f'{uld_type}-{row_number}'
                    uld = ULD(row_number, uld_type, serial)
                    uld.weight = row.UldGrossWeight
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'COL'):
                        uld.COL = 1
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'CRT'):
                        uld.CRT = 1
                    filtered_set_of_ulds.append(uld)
                    serialnumber_list.append(serialnumber)

                elif row.DeadloadType == 'C' and 'PMC' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_PMC += 1
                    uld_type = 'PMC'
                    serial = f'{uld_type}-{row_number}'
                    uld = ULD(row_number, uld_type, serial)
                    uld.weight = row.UldGrossWeight
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'COL'):
                        uld.COL = 1
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'CRT'):
                        uld.CRT = 1
                    filtered_set_of_ulds.append(uld)
                    serialnumber_list.append(serialnumber)

                elif row.DeadloadType == 'C' and 'AAP' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_PMC += 1
                    uld_type = 'PMC'
                    serial = f'{uld_type}-{row_number}'
                    uld = ULD(row_number, uld_type, serial)
                    uld.weight = row.UldGrossWeight
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'COL'):
                        uld.COL = 1
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'CRT'):
                        uld.CRT = 1
                    filtered_set_of_ulds.append(uld)
                    serialnumber_list.append(serialnumber)

                elif row.DeadloadType == 'C' and 'PAG' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_PAG += 1
                    uld_type = 'PAG'
                    serial = f'{uld_type}-{row_number}'
                    uld = ULD(row_number, uld_type, serial)
                    uld.weight = row.UldGrossWeight
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'COL'):
                        uld.COL = 1
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'CRT'):
                        uld.CRT = 1
                    filtered_set_of_ulds.append(uld)
                    serialnumber_list.append(serialnumber)

                elif row.DeadloadType == 'C' and 'PLB' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_PAG += 1
                    uld_type = 'PAG'
                    serial = f'{uld_type}-{row_number}'
                    uld = ULD(row_number, uld_type, serial)
                    uld.weight = row.UldGrossWeight
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'COL'):
                        uld.COL = 1
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'CRT'):
                        uld.CRT = 1
                    filtered_set_of_ulds.append(uld)
                    serialnumber_list.append(serialnumber)

                elif row.DeadloadType == 'C' and 'PLA' in serialnumber and serialnumber not in serialnumber_list:
                    number_of_PAG += 1
                    uld_type = 'PAG'
                    serial = f'{uld_type}-{row_number}'
                    uld = ULD(row_number, uld_type, serial)
                    uld.weight = row.UldGrossWeight
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'COL'):
                        uld.COL = 1
                    if (serialnumber in special_handling_dict.keys()) and (special_handling_dict[serialnumber] == 'CRT'):
                        uld.CRT = 1
                    filtered_set_of_ulds.append(uld)
                    serialnumber_list.append(serialnumber)

                elif row.DeadloadType == 'B' and 'AKE' in serialnumber and serialnumber not in serialnumber_list:
                    uld_type = 'AKE'
                    name_bax = 'BAX'
                    serial = f'{name_bax}-{row_number}'
                    filtered_set_of_ulds.append(ULD(row_number, uld_type, serial))
                    weight_bax.append(row.UldGrossWeight)
                    serialnumber_list.append(serialnumber)

        self.uld = filtered_set_of_ulds

        #Assigning the weight of the BAX ULDs
        bax_index = 0
        for j in self.uld:
            if 'BAX' in j.serialnumber:
                j.weight = weight_bax[bax_index]
                bax_index += 1

        #Assigning the weight and number of the BUP ULDs
        if len(self.BUP_data) > 0:
            for key, row_bup in self.BUP_data.iterrows():
                row_number = max(uld.index for uld in self.uld) + 1
                uld_type = 'PMC'
                name_bax = 'BUP'
                serial = f'{name_bax}-{row_number}'
                self.uld.append(ULD(row_number, uld_type, serial))

        for j in self.uld:
            if 'BUP' in j.serialnumber:
                j.weight = self.BUP_data['BookingTotalWeight'].max()

        #Assigning T-ULDs
        serialnumber_list = []
        if len(df_T_uld_information) > 0:
            for index, row in df_T_uld_information.iterrows():
                uld = row['SerialNumber']
                weight = row['Weight']
                COL = False
                CRT = False
                
                if uld not in serialnumber_list and ((str(uld[0]) != 'B') and (str(uld[0]) != 'K')):
                    row_number = max(uld.index for uld in self.uld) + 1
                    uld_type = str(uld[0:3])

                    if uld in special_handling_dict:
                        if special_handling_dict[uld] == 'COL':
                            COL = True
                        if special_handling_dict[uld] == 'CRT':
                            CRT = True
                    if uld_type not in ['PMC', 'PAG', 'AKE']:
                        if uld_type == 'PLB':
                            uld_type = 'PAG'
                        elif uld_type == 'PLA':
                            uld_type = 'PAG'
                        elif uld_type == 'AAP':
                            uld_type = 'PMC'
                        elif uld_type == 'RKN':
                            uld_type = 'AKE'
                        elif uld_type == 'AKY':
                            uld_type = 'AKE'
                        elif uld_type == 'AKN':
                            uld_type = 'AKE'
                        else:
                            uld_type = 'AKE'
                        
                    name_T = 'T'
                    serial = f'{name_T}-{row_number}'
                    T_uld = ULD(row_number, uld_type, serial)
                    T_uld.weight = weight
                    if COL:
                        T_uld.COL = 1
                    if CRT:
                        T_uld.CRT = 1
                    self.uld.append(T_uld)
                    serialnumber_list.append(uld)

    def filter_items_test(self, amount):
        """
        Filter the items to a specific number of items

        Args:
            amount (int): The number of items to filter
        """
        filter_items = random.sample(self.items, amount)
        self.items = filter_items

        # filter_items = self.items[:amount]
        # self.items = filter_items


    def get_prefix_groups(self):
        """
        Get the serialnumber prefix groups of the items
        """
        prefix_groups = {}
        for i in self.items:
            if i.prefix_serialnumber not in prefix_groups:
                prefix_groups[i.prefix_serialnumber] = []
            prefix_groups[i.prefix_serialnumber].append(i)

        return prefix_groups

In [ ]:
class Aircraft():
    def __init__(self):
        self.aircraft_filename = None
        self.data_fuel_filename = None
        self.main_filename = None

        self.loadlocations = []
        self.loadlocations_left = []
        self.loadlocations_right = []
        self.loadlocations_pallet = []
        self.loadlocations_pairs = []
        self.loadlocations_C1 = []
        self.loadlocations_C2 = []
        self.loadlocations_C3 = []
        self.loadlocations_C4 = []
        self.loadlocations_C1_C2 = []
        self.loadlocations_C3_C4 = []

        self.aircraft_type = None
        self.aircraft_registration = None
        self.flight_number = None
        self.departure_airport = None
        self.arrival_airport = None
        self.date = None
        self.MTOW = None
        self.MLW = None
        self.MZFW = None
        self.AZFW = None
        self.OEW = None
        self.TOF = None
        self.TripF = None
        self.cost_index = None
        self.DOI = None
        self.fuel_index = None
        self.ZFW = None
        self.TOW = None
        self.LW = None

        self.max_weight_C1 = None
        self.max_weight_C2 = None
        self.max_weight_C3 = None
        self.max_weight_C4 = None
        self.max_weight_C1_C2 = None
        self.max_weight_C3_C4 = None

        self.actual_MAC_ZFW = None
        self.C = None
        self.K = None
        self.reference_arm = None
        self.lemac = None
        self.mac_formula = None
        self.delta_index_cargo_C1 = None
        self.delta_index_cargo_C2 = None
        self.delta_index_cargo_C3 = None
        self.delta_index_cargo_C4 = None
        self.delta_index_cargo_C5 = None
        self.delta_index_pax_0A = None
        self.delta_index_pax_0B = None
        self.delta_index_pax_0C = None
        self.delta_index_pax_0D = None
        self.delta_index_pax_0E = None

        self.pax_0A = None
        self.pax_0B = None
        self.pax_0C = None
        self.pax_0D = None
        self.pax_0E = None
        self.pax_0F = None
        self.pax_0G = None
        self.total_PAX = None


    def define_aircraft(self, filename, AC_type, restricted_locations):
        """ 
        Define the aircraft type and the load locations of the aircraft

        Args:
            filename (str): The path to the CSV file containing the aircraft data.
            AC_type (str): The aircraft type.
        """
        #Reding the file and creating a list of LoadLocations objects
        self.aircraft_filename = filename
        data = pd.read_csv(filename)
        data = data[data['ACType'] == str(AC_type)]
        locations = data.LoadLocation
        locations = locations[~locations.isin(restricted_locations)]
        
        for key, i in enumerate(locations):
            self.loadlocations.append(LoadLocations(key, i.strip()))
        
        #Creating lists of load locations for the left, right and pallet positions
        for i in self.loadlocations:
            if 'L' in i.location:
                self.loadlocations_left.append(i)
            elif 'R' in i.location:
                self.loadlocations_right.append(i)
            elif 'P' in i.location:
                self.loadlocations_pallet.append(i)
        
        #Creating lists of load locations for the cargo compartments
        for i in self.loadlocations:
            if '11' in i.location or '12' in i.location or '13' in i.location or '14' in i.location or '15' in i.location:
                self.loadlocations_C1.append(i)
                self.loadlocations_C1_C2.append(i)
            if '21' in i.location or '22' in i.location or '23' in i.location or '24' in i.location or '25' in i.location or '26' in i.location or '27' in i.location or '28' in i.location:
                self.loadlocations_C2.append(i)
                self.loadlocations_C1_C2.append(i)
            if '31' in i.location or '32' in i.location or '33' in i.location or '34' in i.location or '35' in i.location or '36' in i.location:
                self.loadlocations_C3.append(i)
                self.loadlocations_C3_C4.append(i)
            if '41' in i.location or '42' in i.location or '43' in i.location or '44' in i.location or '45' in i.location:
                self.loadlocations_C4.append(i)
                self.loadlocations_C3_C4.append(i)

        #Creating a list of pairs of load locations for the left and right positions
        for i in range(len(self.loadlocations_left)):
            self.loadlocations_pairs.append((self.loadlocations_left[i].location, self.loadlocations_right[i].location))
        
        
    def define_overlapping_positions(self, position):
        """
        Define the overlapping positions of a specific position

        Args:
            position (LoadLocations): The position for which the overlapping positions are defined.
        """
        #Reading the file and creating a list of overlapping LoadLocations objects
        overlapping = []
        data = pd.read_csv(self.aircraft_filename, converters={'Overlapping': lambda x: x.split(',')})
        data = data[data['ACType'] == str(self.aircraft_type)]
        list_pos = data[data.LoadLocation == position.location]['Overlapping']
        for overlapping_list in list_pos:
            for overlapping_position in overlapping_list:
                for i in self.loadlocations:
                    if i.location == overlapping_position.strip():
                        overlapping.append(i)

        return overlapping
    

    def define_forbidden_positions_for_ULD(self, uld):
        """
        Define the forbidden positions for a specific ULD type

        Args:
            uld (ULD): ULD class object.
        """
        #Creating a list of forbidden LoadLocations objects for PMC ULDs
        forbidden = []
        if uld.type == 'PMC':
            for i in self.loadlocations:
                if i not in self.loadlocations_pallet:
                    forbidden.append(i)

        #Creating a list of forbidden LoadLocations objects for AKE ULDs
        elif uld.type == 'AKE':
            for i in self.loadlocations:
                if i in self.loadlocations_pallet:
                    forbidden.append(i)

        #Creating a list of forbidden LoadLocations objects for PAG ULDs
        elif uld.type == 'PAG':
            for i in self.loadlocations:
                if i not in self.loadlocations_pallet:
                    forbidden.append(i)
        return forbidden
    

    def define_positions_for_ULD(self, uld):
        """
        Define the positions for a specific ULD type

        Args:
            uld (ULD): ULD class object.
        """
        #Creating a list of LoadLocations objects for PMC ULDs
        positions = []
        if uld.type == 'PMC':
            for i in self.loadlocations_pallet:
                positions.append(i)

        #Creating a list of LoadLocations objects for AKE ULDs
        elif uld.type == 'AKE':
            for i in self.loadlocations:
                if i not in self.loadlocations_pallet:
                    positions.append(i)

        #Creating a list of LoadLocations objects for PAG ULDs
        elif uld.type == 'PAG':
            for i in self.loadlocations_pallet:
                positions.append(i)
        return positions
    

    def define_max_weight_postion(self, position):
        """
        Define the maximum weight for a specific position

        Args:
            position (LoadLocations): The position for which the maximum weight is defined.
        """
        if position in self.loadlocations_pallet:
            return 5102
        if position not in self.loadlocations_pallet:
            return 1587
        

    def define_MPL(self):
        """
        Define Maximum Payload for the aircraft
        """ 
        limit_1 = self.MTOW - self.OEW - self.TOF
        limit_2 = self.MLW - self.OEW - (self.TOF - self.TripF)
        limit_3 = self.MZFW - self.OEW
        MPL = min(limit_1, limit_2, limit_3)
        return MPL
        

    def define_proximity_score_loadlocation(self, position):
        """
        Define the proximity score for a specific position with respect of the door of its compartment

        Args:
            position (LoadLocations): The position for which the proximity score is defined.
        """
        data = pd.read_csv(self.aircraft_filename)
        data = data[data['ACType'] == str(self.aircraft_type)]

        proximity_score = data[data.LoadLocation == position.location]['ProximityScore'].iloc[0]

        return int(proximity_score)
    

    def define_flight_information(self, filename):
        """
        Define the flight information of the aircraft

        Args:
            filename (str): The path to the Excel file containing the flight data.
        """
        data = pd.read_csv(filename)

        self.aircraft_type = data['AircraftType'].iloc[0]
        self.aircraft_registration = data['AircraftRegistration'].iloc[0]
        self.flight_number = str(data['Airline'].iloc[0]) + str(data['FlightNumber'].iloc[0])
        self.departure_airport = data['DepartureAirport'].iloc[0]
        self.arrival_airport = data['ArrivalAirport'].iloc[0]
        self.date = pd.to_datetime(data['LegDepartureDateUtc'].iloc[0]).strftime("%d %b %y").upper()
        self.AZFW = data['ActualZeroFuelWeight'].iloc[0]
        self.OEW = data['DryOperatingWeight'].iloc[0]
        self.actual_MAC_ZFW = data['MacZFW'].iloc[0]
        if 1 < float(self.actual_MAC_ZFW) < 10:
            self.actual_MAC_ZFW = float(self.actual_MAC_ZFW) * 10
        if float(self.actual_MAC_ZFW) < 1:
            self.actual_MAC_ZFW = float(self.actual_MAC_ZFW) * 100
        self.cost_index =data['CostIndex'].iloc[0]
        self.TOF = data['TakeOffFuel'].iloc[0]
        self.TripF = data['TripFuel'].iloc[0]
        self.cost_index = data['CostIndex'].iloc[0]


    def define_max_weight_information(self, filename, AC_type):
        """
        Define the maximum weight information of the aircraft

        Args:
            filename (str): The path to the CSV file containing the maximum weight data.
            AC_type (str): The aircraft type.
        """
        data = pd.read_csv(filename)
        self.main_filename = filename

        self.MZFW = data[data['ACType'] == str(AC_type)]['MaxZeroFuelWeight'].max()
        self.MTOW = data[data['ACType'] == str(AC_type)]['MaxTakeOffWeight'].max()
        self.MLW = data[data['ACType'] == str(AC_type)]['MaxLandingWeight'].max()
        self.max_weight_C1 = data[data['ACType'] == str(AC_type)]['MaxWeightC1'].max()
        self.max_weight_C2 = data[data['ACType'] == str(AC_type)]['MaxWeightC2'].max()
        self.max_weight_C3 = data[data['ACType'] == str(AC_type)]['MaxWeightC3'].max()
        self.max_weight_C4 = data[data['ACType'] == str(AC_type)]['MaxWeightC4'].max()
        self.max_weight_C1_C2 = data[data['ACType'] == str(AC_type)]['MaxWeightC1C2'].max()
        self.max_weight_C3_C4 = data[data['ACType'] == str(AC_type)]['MaxWeightC3C4'].max()


    def define_envelope_information(self, AC_type):
        """
        Define the CG envelope information of the aircraft

        Args:
            AC_type (str): The aircraft type.
        """
        data = pd.read_csv(self.main_filename)
        data = data[data['ACType'] == str(AC_type)]


        self.C = data['C'].iloc[0]
        self.K = data['K'].iloc[0]
        self.reference_arm = data['ReferenceArm'].iloc[0]
        self.lemac = data['LEMAC'].iloc[0]
        self.mac_formula = data['MACFormula'].iloc[0]
        self.delta_index_cargo_C1 = data['DeltaIndexCargoC1'].iloc[0]
        self.delta_index_cargo_C2 = data['DeltaIndexCargoC2'].iloc[0]
        self.delta_index_cargo_C3 = data['DeltaIndexCargoC3'].iloc[0]
        self.delta_index_cargo_C4 = data['DeltaIndexCargoC4'].iloc[0]
        self.delta_index_cargo_C5 = data['DeltaIndexCargoC5'].iloc[0]
        self.delta_index_pax_0A = data['DeltaIndexPax0A'].iloc[0]
        self.delta_index_pax_0B = data['DeltaIndexPax0B'].iloc[0]
        self.delta_index_pax_0C = data['DeltaIndexPax0C'].iloc[0]
        self.delta_index_pax_0D = data['DeltaIndexPax0D'].iloc[0]
        self.delta_index_pax_0E = data['DeltaIndexPax0E'].iloc[0]
        self.delta_index_pax_0F = data['DeltaIndexPax0F'].iloc[0]
        self.delta_index_pax_0G = data['DeltaIndexPax0G'].iloc[0]


    def define_fuel_index(self, filename, AC_type):
        """
        Define the fuel index of the aircraft

        Args:
            filename (str): The path to the CSV file containing the fuel index data.
            AC_type (str): The aircraft type.
        """
        #Reading the file and interpolating to find the closest fuel index
        self.data_fuel_filename = filename
        data_fuel_index = pd.read_csv(filename)
        data_fuel_index = data_fuel_index[data_fuel_index['ACType'] == str(AC_type)]

        closest_TOF =  (data_fuel_index['TakeOffFuel'] - self.TOF).abs().idxmin()
        closest_TOF_row = data_fuel_index.loc[closest_TOF]
        self.fuel_index = closest_TOF_row['FuelIndex']

    def define_DOI(self, filename, AC_type):
        """
        Define the DOI of the aircraft

        Args:
            filename (str): The path to the CSV file containing the DOI data.
            AC_type (str): The aircraft type.
        """
        #Reading the file and interpolating to find the closest DOI
        data = pd.read_csv(filename)
        data = data[(data['ACType'] == str(AC_type)) & (data['ACRegistration'] == str(self.aircraft_registration))]
        self.DOI = data['DOI'].iloc[0]
        # self.OEW = data['OEW'].iloc[0]


    def define_INDEX_ZFW_fwd(self, AC_type):
        """
        Define the forward ZFW index limit of the aircraft

        Args:
            AC_type (str): The aircraft type.
        """
        #Reading the file and filtering the information needed
        df = pd.read_csv(self.main_filename)
        df = df[df['ACType'] == str(AC_type)][['ZFWWeightFwd', 'ZFWIndexFwd']].reset_index(drop=True)
        df = df.dropna()

        #Finding the range of the ZFW
        pivot_index = df[df['ZFWWeightFwd'] <= self.ZFW].index[-1]

        #If pivot index is the last index, the range is from the last index to 0
        if pivot_index == df.index[-1]:
            zero_row = pd.DataFrame([[0, 0]], columns=['ZFWWeightFwd', 'ZFWIndexFwd'])
            pivot_row = df.iloc[[pivot_index]][['ZFWWeightFwd', 'ZFWIndexFwd']]
            rows = pd.concat([pivot_row, zero_row], ignore_index=True)
        #If pivot index is not the last index, the range is from the pivot index to the next index
        else:
            rows = df.iloc[[pivot_index, pivot_index + 1]][['ZFWWeightFwd', 'ZFWIndexFwd']]

        #Calculating the ZFW fwt index limit
        ZFW_fwt_limit = rows['ZFWIndexFwd'].iloc[0] + ((rows['ZFWIndexFwd'].iloc[1] - rows['ZFWIndexFwd'].iloc[0]) / (rows['ZFWWeightFwd'].iloc[1] - rows['ZFWWeightFwd'].iloc[0])) * (self.ZFW - rows['ZFWWeightFwd'].iloc[0])

        return float(ZFW_fwt_limit)


    def define_INDEX_ZFW_aft(self, AC_type):
        """
        Define the aft ZFW index limit of the aircraft

        Args:
            AC_type (str): The aircraft type.
        """
        #Reading the file and filtering the information needed
        df = pd.read_csv(self.main_filename)
        df = df[df['ACType'] == str(AC_type)][['ZFWWeightAft', 'ZFWIndexAft']].reset_index(drop=True)
        df = df.dropna()

        #Finding the range of the ZFW
        pivot_index = df[df['ZFWWeightAft'] <= self.ZFW].index[-1]

        #If pivot index is the last index, the range is from the last index to 0
        if pivot_index == df.index[-1]:
            zero_row = pd.DataFrame([[0, 0]], columns=['ZFWWeightAft', 'ZFWIndexAft'])
            pivot_row = df.iloc[[pivot_index]][['ZFWWeightAft', 'ZFWIndexAft']]
            rows = pd.concat([pivot_row, zero_row], ignore_index=True)
        #If pivot index is not the last index, the range is from the pivot index to the next index
        else:
            rows = df.iloc[[pivot_index, pivot_index + 1]][['ZFWWeightAft', 'ZFWIndexAft']]

        #Calculating the ZFW aft index limit
        ZFW_aft_limit = rows['ZFWIndexAft'].iloc[0] + ((rows['ZFWIndexAft'].iloc[1] - rows['ZFWIndexAft'].iloc[0]) / (rows['ZFWWeightAft'].iloc[1] - rows['ZFWWeightAft'].iloc[0])) * (self.ZFW - rows['ZFWWeightAft'].iloc[0])

        return float(ZFW_aft_limit)
   

    def define_INDEX_TOW_fwd(self, AC_type):
        """
        Define the forward TOW index limit of the aircraft

        Args:
            AC_type (str): The aircraft type.
        """
        #Reading the file and filtering the information needed
        df = pd.read_csv(self.main_filename)
        df = df[df['ACType'] == str(AC_type)][['TOWWeightFwd', 'TOWIndexFwd']].reset_index(drop=True)
        df = df.dropna()
        
        #Finding the range of the TOW
        pivot_index = df[df['TOWWeightFwd'] <= self.TOW].index[-1]

        #If pivot index is the last index, the range is from the last index to 0
        if pivot_index == df.index[-1]:
            zero_row = pd.DataFrame([[0, 0]], columns=['TOWWeightFwd', 'TOWIndexFwd'])
            pivot_row = df.iloc[[pivot_index]][['TOWWeightFwd', 'TOWIndexFwd']]
            rows = pd.concat([pivot_row, zero_row], ignore_index=True)
        #If pivot index is not the last index, the range is from the pivot index to the next index
        else:
            rows = df.iloc[[pivot_index, pivot_index + 1]][['TOWWeightFwd', 'TOWIndexFwd']]

        #Calculating the TOW fwt index limit
        TOW_fwt_limit = rows['TOWIndexFwd'].iloc[0] + ((rows['TOWIndexFwd'].iloc[1] - rows['TOWIndexFwd'].iloc[0]) / (rows['TOWWeightFwd'].iloc[1] - rows['TOWWeightFwd'].iloc[0])) * (self.TOW - rows['TOWWeightFwd'].iloc[0])

        return float(TOW_fwt_limit)
        
        
    def define_INDEX_TOW_aft(self, AC_type):
        """
        Define the aft TOW index limit of the aircraft

        Args:
            AC_type (str): The aircraft type.
        """
        #Reading the file and filtering the information needed
        df = pd.read_csv(self.main_filename)
        df = df[df['ACType'] == str(AC_type)][['TOWWeightAft', 'TOWIndexAft']].reset_index(drop=True)
        df = df.dropna()

        #Finding the range of the TOW
        pivot_index = df[df['TOWWeightAft'] <= self.TOW].index[-1]

        #If pivot index is the last index, the range is from the last index to 0
        if pivot_index == df.index[-1]:
            zero_row = pd.DataFrame([[0, 0]], columns=['TOWWeightAft', 'TOWIndexAft'])
            pivot_row = df.iloc[[pivot_index]][['TOWWeightAft', 'TOWIndexAft']]
            rows = pd.concat([pivot_row, zero_row], ignore_index=True)
        #If pivot index is not the last index, the range is from the pivot index to the next index
        else:
            rows = df.iloc[[pivot_index, pivot_index + 1]][['TOWWeightAft', 'TOWIndexAft']]

        #Calculating the TOW aft index limit
        TOW_aft_limit = rows['TOWIndexAft'].iloc[0] + ((rows['TOWIndexAft'].iloc[1] - rows['TOWIndexAft'].iloc[0]) / (rows['TOWWeightAft'].iloc[1] - rows['TOWWeightAft'].iloc[0])) * (self.TOW - rows['TOWWeightAft'].iloc[0])

        return float(TOW_aft_limit)


    def define_INDEX_LW_fwd(self, AC_type):
        """
        Define the forward LW index limit of the aircraft

        Args:
            AC_type (str): The aircraft type.
        """
        #Reading the file and filtering the information needed
        df = pd.read_csv(self.main_filename)
        df = df[df['ACType'] == str(AC_type)].reset_index(drop=True)

        #Finding the range of the LW
        pivot_index = df[df['LWWeightFwd'] <= self.LW].index[-1]

        #If pivot index is the last index, the range is from the last index to 0
        if pivot_index == df.index[-1]:
            missing_rows = df.iloc[pivot_index][['LWWeightFwd', 'LWIndexFwd']]
            rows = pd.concat([rows, missing_rows], ignore_index=True)
        #If pivot index is not the last index, the range is from the pivot index to the next index
        else:
            rows = df.iloc[[pivot_index, pivot_index + 1]][['LWWeightFwd', 'LWIndexFwd']]

        #Calculating the LW fwt index limit
        LW_fwt_limit = rows['LWIndexFwd'].iloc[0] + ((rows['LWIndexFwd'].iloc[1] - rows['LWIndexFwd'].iloc[0]) / (rows['LWWeightFwd'].iloc[1] - rows['LWWeightFwd'].iloc[0])) * (self.LW - rows['LWWeightFwd'].iloc[0])

        return float(LW_fwt_limit)
        

    def define_INDEX_LW_aft(self, AC_type):
        """
        Define the aft LW index limit of the aircraft

        Args:
            AC_type (str): The aircraft type.
        """
        #Reading the file and filtering the information needed
        df = pd.read_csv(self.main_filename)
        df = df[df['ACType'] == str(AC_type)].reset_index(drop=True)

        #Finding the range of the LW
        pivot_index = df[df['LWWeightAft'] <= self.LW].index[-1]

        #If pivot index is the last index, the range is from the last index to 0
        if pivot_index == df.index[-1]:
            missing_rows = df.iloc[pivot_index][['LWWeightAft', 'LWIndexAft']]
            rows = pd.concat([rows, missing_rows], ignore_index=True)
        #If pivot index is not the last index, the range is from the pivot index to the next index
        else:
            rows = df.iloc[[pivot_index, pivot_index + 1]][['LWWeightAft', 'LWIndexAft']]

        #Calculating the LW aft index limit
        LW_aft_limit = rows['LWIndexAft'].iloc[0] + ((rows['LWIndexAft'].iloc[1] - rows['LWIndexAft'].iloc[0]) / (rows['LWWeightAft'].iloc[1] - rows['LWWeightAft'].iloc[0])) * (self.LW - rows['LWWeightAft'].iloc[0])

        return float(LW_aft_limit)

        
    def define_INDEX_LW(self, ZFW_index):
        """
        Define the LW index of the aircraft

        Args:
            ZFW_index (float): The ZFW index of the aircraft.
        """
        data = pd.read_csv(self.data_fuel_filename)
        
        data['corrected_weight'] = data['TakeOffFuel'] + self.ZFW
        closest_corrected_weight =  (data['corrected_weight'] - self.LW).abs().idxmin()
        closest_corrected_weight_row = data.loc[closest_corrected_weight]
        LW_INDEX = closest_corrected_weight_row['FuelIndex'] +  ZFW_index

        return LW_INDEX
    

    def define_axis_ZFW(self, AC_type):
        """
        Define the ZFW axis of the aircraft for the graph

        Args:
            AC_type (str): The aircraft type.
        """
        df = pd.read_csv(self.main_filename)
        df = df[df['ACType'] == str(AC_type)]

        index_axis = [x for x in df['ZFWIndexFwd'].tolist() if not pd.isna(x)] + [x for x in df['ZFWIndexAft'].iloc[::-1].tolist() if not pd.isna(x)]
        weight_axis = [x for x in df['ZFWWeightFwd'].tolist() if not pd.isna(x)] + [x for x in df['ZFWWeightAft'].iloc[::-1].tolist() if not pd.isna(x)]


        return index_axis, weight_axis
    

    def define_axis_TOW(self, AC_type):
        """
        Define the TOW axis of the aircraft for the graph

        Args:
            AC_type (str): The aircraft type.
        """
        df = pd.read_csv(self.main_filename)
        df = df[df['ACType'] == str(AC_type)]

        index_axis = [x for x in df['TOWIndexFwd'].tolist() if not pd.isna(x)] + [x for x in df['TOWIndexAft'].iloc[::-1].tolist() if not pd.isna(x)]
        weight_axis = [x for x in df['TOWWeightFwd'].tolist() if not pd.isna(x)] + [x for x in df['TOWWeightAft'].iloc[::-1].tolist() if not pd.isna(x)]

        return index_axis, weight_axis
    

    def define_axis_LW(self, AC_type):
        """
        Define the LW axis of the aircraft for the graph

        Args:
            AC_type (str): The aircraft type.
        """
        df = pd.read_csv(self.main_filename)
        df = df[df['ACType'] == str(AC_type)]

        index_axis = [x for x in df['LWIndexFwd'].tolist() if not pd.isna(x)] + [x for x in df['LWIndexAft'].iloc[::-1].tolist() if not pd.isna(x)]
        weight_axis = [x for x in df['LWWeightFwd'].tolist() if not pd.isna(x)] + [x for x in df['LWWeightAft'].iloc[::-1].tolist() if not pd.isna(x)]

        return index_axis, weight_axis
    
    def define_PAX_allocation(self, pax_filename):
        df = pd.read_csv(pax_filename)

        self.pax_0A = df[df['CenterOfGravityCabin'] == 'CABIN0A']['PassengerCount'].iloc[0]
        self.pax_0B = df[df['CenterOfGravityCabin'] == 'CABIN0B']['PassengerCount'].iloc[0]
        self.pax_0C = df[df['CenterOfGravityCabin'] == 'CABIN0C']['PassengerCount'].iloc[0]
        self.pax_0D = df[df['CenterOfGravityCabin'] == 'CABIN0D']['PassengerCount'].iloc[0]
        self.pax_0E = df[df['CenterOfGravityCabin'] == 'CABIN0E']['PassengerCount'].iloc[0]
        if 'CABIN0F' in df['CenterOfGravityCabin'].to_list() and 'CABIN0G' in df['CenterOfGravityCabin'].to_list():
            self.pax_0F = df[df['CenterOfGravityCabin'] == 'CABIN0F']['PassengerCount'].iloc[0]
            self.pax_0G = df[df['CenterOfGravityCabin'] == 'CABIN0G']['PassengerCount'].iloc[0]
        else:
            self.pax_0F = 0
            self.pax_0G = 0
            
        self.total_PAX = self.pax_0A + self.pax_0B + self.pax_0C + self.pax_0D + self.pax_0E + self.pax_0F + self.pax_0G


    def define_INDEX_PAX(self):
        """
        Define the PAX index of the aicraft
        """
        #Calculating the PAX index per compartment
        index_0A = self.pax_0A * 84 * self.delta_index_pax_0A
        index_0B = self.pax_0B * 84 * self.delta_index_pax_0B
        index_0C = self.pax_0C * 84 * self.delta_index_pax_0C
        index_0D = self.pax_0D * 84 * self.delta_index_pax_0D
        index_0E = self.pax_0E * 84 * self.delta_index_pax_0E
        index_0F = self.pax_0F * 84 * self.delta_index_pax_0F
        index_0G = self.pax_0G * 84 * self.delta_index_pax_0G

        #Calculating the total PAX index
        index_PAX = index_0A + index_0B + index_0C + index_0D + index_0E + index_0F + index_0G

        return float(index_PAX)
    

    def define_PAX_weight(self):
        """
        Define the PAX weight of the aircraft
        """
        #Calculating the PAX weight, IATA assumption of 84 kg per PAX
        weight_pax = (self.pax_0A + self.pax_0B + self.pax_0C + self.pax_0D + self.pax_0E + self.pax_0F + self.pax_0G) * 84
        return int(weight_pax)
    
    
    def define_ff_increment_MAC_ZFW(self, MAC_ZFW):
        """
        Define the fuel efficiency increment for the MAC ZFW

        Args:
            MAC_ZFW (float): The MAC ZFW of the aircraft.
        """
        actual_increment = None 
        model_increment = None
        
        #Defining the fuel efficiency brackets for the different aircraft types
        if str(self.aircraft_type) == '789':
            fuel_efficiency_brackets = np.array([
                (0, 1.9),
                (16, 1.9),
                (18, 1.7),
                (20, 1.3),
                (22, 1.1),
                (24, 0.8),
                (26, 0.5),
                (28, 0.3),
                (32, 0),
                (34, -0.2),
                (36, -0.3),
                (40, -0.3),
                (43, -0.3)
            ])

        elif str(self.aircraft_type) == '781':
            fuel_efficiency_brackets = np.array([
                (0, 2.2),
                (16, 2.2),
                (18, 2),
                (20, 1.6),
                (22, 1.3),
                (24, 1),
                (26, 0.6),
                (28, 0.4),
                (32, 0),
                (34, -0.3),
                (36, -0.4),
                (40, -0.5),
                (43, -0.5)
            ])

        elif str(self.aircraft_type) == '772':
            fuel_efficiency_brackets = np.array([
                (0, 1.3),
                (16, 1.3),
                (18, 1.3),
                (20, 1.2),
                (22, 0.9),
                (24, 0.6),
                (26, 0.4),
                (28, 0.2),
                (32, 0),
                (34, -0.4),
                (36, -0.5),
                (40, -0.7),
                (43, -0.7)
            ])

        elif str(self.aircraft_type) == '77W':
            fuel_efficiency_brackets = np.array([
                (0, 1.7), 
                (16, 1.7),
                (18, 1.5),
                (20, 1.2),
                (22, 0.9),
                (24, 0.6), 
                (26, 0.4),
                (28, 0.2), 
                (32, 0),
                (34, -0.4),
                (36, -0.5),
                (40, -0.6),
                (43, -0.6)
            ])


        for i in range(len(fuel_efficiency_brackets) - 1):
            if fuel_efficiency_brackets[i][0] <= MAC_ZFW <= fuel_efficiency_brackets[i + 1][0]:
                x1, y1 = fuel_efficiency_brackets[i]
                x2, y2 = fuel_efficiency_brackets[i + 1]
                model_increment = y1 + (y2 - y1) * (MAC_ZFW - x1) / (x2 - x1)

            if fuel_efficiency_brackets[i][0] <= self.actual_MAC_ZFW <= fuel_efficiency_brackets[i + 1][0]:
                x1, y1 = fuel_efficiency_brackets[i]
                x2, y2 = fuel_efficiency_brackets[i + 1]
                actual_increment = y1 + (y2 - y1) * (self.actual_MAC_ZFW - x1) / (x2 - x1)

        if actual_increment is not None and model_increment is not None:
            fuel_saving = model_increment - actual_increment
            return fuel_saving




In [8]:
class Extreme_Points():
    def __init__(self) -> None:
        pass
    
    
    def get_color_map(self, items):
        """
        Generate a color map for items based on their prefix_serialnumber

        Args:
            items (list): List of items to generate a color map for.
        """
        # Get unique prefixes_serialnumber
        prefixes_serialnumber = sorted(
            list(set([item.prefix_serialnumber for item in items]))
        )
        
        # Generate a color map using a colormap from matplotlib
        cmap = matplotlib.colormaps.get_cmap('tab20')
        colors = np.linspace(0, 1, len(prefixes_serialnumber))
        color_map = {prefix: cmap(color) for prefix, color in zip(prefixes_serialnumber, colors)}
        
        return color_map


    def get_item_bounds(self, item, placed_items):
        """
        Get the bounds of an item based on its position and orientation

        Args:
            item (Item): The item to get the bounds for.
            placed_items (dict): Dictionary of placed items.
        """
        x, y, z, length, width, height, weight, stack = placed_items[item]
    
        return {'x_min': x,'x_max': x + length,
                'y_min': y,'y_max': y + width,
                'z_min': z,'z_max': z + height
                }
    

    def get_item_edges(self, item_bounds):
        """
        Get the edges of an item based on its bounds

        Args:
            item_bounds (dict): Dictionary of item bounds.
        """
        edges = {
            'left': {(item_bounds['x_min'], y) for y in range(item_bounds['y_min'], item_bounds['y_max'] + 1)},
            'right': {(item_bounds['x_max'], y) for y in range(item_bounds['y_min'], item_bounds['y_max'] + 1)},
            'front': {(x, item_bounds['y_min']) for x in range(item_bounds['x_min'], item_bounds['x_max'] + 1)},
            'back': {(x, item_bounds['y_max']) for x in range(item_bounds['x_min'], item_bounds['x_max'] + 1)},
        }
        return edges
    

    def get_base_corners(self, ep, orientation):
        """
        Get the corners of the base of an item based on its extreme point and orientation

        Args:
            ep (tuple): The extreme point of the item.
            orientation (tuple): The orientation of the item.
        """
        length, width, height = orientation

        corners = [
            (ep[0], ep[1], ep[2]),
            (ep[0] + length, ep[1], ep[2]),
            (ep[0], ep[1] + width, ep[2]),
            (ep[0] + length, ep[1] + width, ep[2]),
        ]

        return corners
    

    def get_starting_extreme_points(self, j):
        """
        Get the starting extreme points for the ULD types

        Args:
            j (ULD): The ULD to get the starting extreme points for.
        """
        if 'AKE' in j.serialnumber:
            return [(j.a, 0, 0)]
        else:
            return [(0, 0, 0)]
            

    def update_extreme_points(self, extreme_points, ep, i, j):
        """
        Update the extreme points based on the placed item

        Args:
            extreme_points (list): List of extreme points to update.
            ep (tuple): The extreme point of the placed item.
            i (Item): The placed item.
        """
        # The farthest corner of the placed item
        new_point = (ep[0] + i.length, ep[1] + i.width, ep[2] + i.height)

        # Add the new extreme point if it's inside the container and not already present
        if new_point not in extreme_points:
            extreme_points.append(new_point)

        # Additional potential extreme points generated by the placed item
        potential_points = [
            (ep[0] + i.length, ep[1], ep[2]),
            (ep[0], ep[1] + i.width, ep[2]),
            (ep[0], ep[1], ep[2] + i.height),
            (ep[0] + i.length, ep[1] + i.width, ep[2]),
            (ep[0] + i.length, ep[1], ep[2] + i.height),
            (ep[0], ep[1] + i.width, ep[2] + i.height)
        ]

        # Add each potential extreme point if it's inside the bounds and not already present
        for point in potential_points:
            if point not in extreme_points:
                extreme_points.append(point)

        # Remove the extreme point where the item was placed, as it's no longer 'extreme'
        if ep in extreme_points:
            extreme_points.remove(ep)

        return extreme_points


    def get_orientations(self, item):
        """
        Get the possible orientations of an item

        Args:
            item (Item): The item to get the orientations for.
        """
        if item.rotate == 1:
            dimensions = [
                (item.length, item.width, item.height),
                (item.length, item.height, item.width),
                (item.width, item.length, item.height),
                (item.width, item.height, item.length),
                (item.height, item.length, item.width),
                (item.height, item.width, item.length)

            ]

        else:
            dimensions = [
                (item.length, item.width, item.height),
            ]
        
        return dimensions
    

    def check_collision(self, ep, length, width, height, placed_items, stack_item):
        """
        Check for collisions between the placed item and other items

        Args:
            ep (tuple): The extreme point of the placed item.
            length (float): The length of the placed item.
            width (float): The width of the placed item.
            height (float): The height of the placed item.
            placed_items (dict): Dictionary of placed items.
        """
        for other_item in placed_items:
            x_other_item, y_other_item, z_other_item, length_other_item, width_other_item, height_other_item, weight_other_item, stack_other_item = placed_items[other_item]
            if (ep[0] < x_other_item + length_other_item and ep[0] + length > x_other_item and
                ep[1] < y_other_item + width_other_item and ep[1] + width > y_other_item and
                ep[2] < z_other_item + height_other_item and ep[2] + height > z_other_item):
                return True
            
            if stack_item == 1:
                if (ep[2] + height > z_other_item):
                    return True
            
        return False
    

    def is_point_supported(self, point, placed_items):
        """
        Check if a point is supported by other items

        Args:
            point (tuple): The point to check for support.
            placed_items (dict): Dictionary of placed items.
        """
        x, y, z = point

        for _, (item_x, item_y, item_z, length, width, height, weight, stack) in placed_items.items():
            if (item_x <= x <= item_x + length) and (item_y <= y <= item_y + width) and z == item_z + height:
                return True
        return False
        

    def check_full_base_coverage(self, j, ep, orientation, placed_items):
        """
        Check if the base of the placed item is fully supported

        Args:
            ep (tuple): The extreme point of the placed item.
            orientation (tuple): The orientation of the placed item.
            placed_items (dict): Dictionary of placed items.
        """
        length, width, height = orientation
        base_corners = self.get_base_corners(ep, orientation)

        key_points = base_corners + [
            (ep[0] + length / 2, ep[1] + width / 2, ep[2]),
            (ep[0] + length / 2, ep[1], ep[2]),
            (ep[0], ep[1] + width / 2, ep[2]),
            (ep[0] + length, ep[1] + width / 2, ep[2]),
            (ep[0] + length / 2, ep[1] + width, ep[2]),
        ]

        supported_points = 0

        for point in key_points:
            if self.is_point_supported(point, placed_items):
                supported_points += 1
      

        return supported_points == len(key_points)


    def is_supported_by_uld_side(self, ep, j):
        """
        Check if the placed item is supported by the ULD side

        Args:
            ep (tuple): The extreme point of the placed item.
            j (ULD): The ULD to check for support.
        """
        x_support = (ep[0] == 0 or ep[0] == j.length)
        y_support = (ep[1] == 0 or ep[1] == j.width)
        
        uld_support_count = int(x_support) + int(y_support) 

        return uld_support_count
    

    def is_side_contact(self, ep, orientation, placed_items):
        """
        Check if the placed item has side contact with other items

        Args:
            ep (tuple): The extreme point of the placed item.
            orientation (tuple): The orientation of the placed item.
            placed_items (dict): Dictionary of placed items.
        """
        contact_counts = {'left': 0, 'right': 0, 'front': 0, 'back': 0}

        item_bounds = {
            'x_min': ep[0], 'x_max': ep[0] + orientation[0],
            'y_min': ep[1], 'y_max': ep[1] + orientation[1],
            'z_min': ep[2], 'z_max': ep[2] + orientation[2],
                    }

        for other_item_serial, other_item_info in placed_items.items():
            other_item_bounds = self.get_item_bounds(other_item_serial, placed_items)
            
            if (item_bounds['y_min'] < other_item_bounds['y_max'] and
                item_bounds['y_max'] > other_item_bounds['y_min'] and
                item_bounds['z_min'] < other_item_bounds['z_max'] and
                item_bounds['z_max'] > other_item_bounds['z_min']):
                if item_bounds['x_min'] == other_item_bounds['x_max']:
                    contact_counts['left'] = 1
                elif item_bounds['x_max'] == other_item_bounds['x_min']:
                    contact_counts['right'] = 1

            if (item_bounds['x_min'] < other_item_bounds['x_max'] and
                item_bounds['x_max'] > other_item_bounds['x_min'] and
                item_bounds['z_min'] < other_item_bounds['z_max'] and
                item_bounds['z_max'] > other_item_bounds['z_min']):
                if item_bounds['y_min'] == other_item_bounds['y_max']:
                    contact_counts['front'] = 1
                elif item_bounds['y_max'] == other_item_bounds['y_min']:
                    contact_counts['back'] = 1

        side_support_count = sum(contact_counts.values())

        return side_support_count


    def check_side_support(self, ep, orientation, placed_items, j):
        """
        Check if the placed item is supported by the ULD side and has side contact with other items

        Args:
            ep (tuple): The extreme point of the placed item.
            orientation (tuple): The orientation of the placed item.
            placed_items (dict): Dictionary of placed items.
            j (ULD): The ULD to check for support.
        """
        side_support_count = 0

        side_support = self.is_side_contact(ep, orientation, placed_items)
        side_support_count += side_support

        uld_support = self.is_supported_by_uld_side(ep, j)
        side_support_count += uld_support
                
        return side_support_count
    

    def check_supported_by_cut(self, ep, orientation, j, placed_items):
        """
        Check if the placed item is supported by the AKE ULD cut

        Args:
            ep (tuple): The extreme point of the placed item.
            orientation (tuple): The orientation of the placed item.
            j (ULD): The ULD to check for support.
            placed_items (dict): Dictionary of placed items.
        """
        if 'AKE' in j.serialnumber:
            item_bounds = {
                'x_min': ep[0], 'x_max': ep[0] + orientation[0],
                'y_min': ep[1], 'y_max': ep[1] + orientation[1],
                'z_min': ep[2], 'z_max': ep[2] + orientation[2],
                    }


            expected_z_on_cut = -j.cut_a * ep[0] + j.b


            touches_cut = False
            if (item_bounds['z_min'] == expected_z_on_cut):
                touches_cut = True

            lateral_support = False
            for other_item_serial, other_item_info in placed_items.items():
                other_item_bounds = self.get_item_bounds(other_item_serial, placed_items)

                is_adjacent = (other_item_bounds['x_max'] == item_bounds['x_min'] or
                            other_item_bounds['x_min'] == item_bounds['x_max'])

                is_at_cut_height = (other_item_bounds['z_max'] >= expected_z_on_cut)

                if is_adjacent and is_at_cut_height:
                    lateral_support = True
                    break

            if touches_cut and lateral_support:
                return True

        return False
    

    def is_fully_supported(self, ep, orientation, placed_items, j): 
        """
        Check if the placed item is fully supported, support count must be equal or greater than 3

        Args:
            ep (tuple): The extreme point of the placed item.
            orientation (tuple): The orientation of the placed item.
            placed_items (dict): Dictionary of placed items.
            j (ULD): The ULD to check for support.
        """
        support_count = 0
        ground_supported = False
        base_supported = False

        if ep[2] == 0:
            ground_support = 3
            support_count += ground_support
            ground_supported = True

        if self.check_full_base_coverage(j, ep, orientation, placed_items):
            base_support = 2
            support_count += base_support
            base_supported = True

        if not ground_supported and not base_supported:
            return False, support_count

        if base_supported or ground_supported:
            side_support = self.check_side_support(ep, orientation, placed_items, j)
            support_count += side_support

        if self.check_supported_by_cut(ep, orientation, j, placed_items):
            cut_support = 3
            support_count += cut_support

        if support_count >= 3:
            return True, support_count
        
        return False, support_count
    

    def calculate_cog_deviation(self, placed_items, j):
        x_cog, y_cog = self.calculate_cog(placed_items)
        center_x, center_y = j.length / 2, j.width / 2

        cog_deviation = ((x_cog - center_x) ** 2 + (y_cog - center_y) ** 2) ** 0.5

        return cog_deviation


    def calculate_cog(self, placed_items):
        total_weight = sum(item[6] for item in placed_items.values())
        x_cog = sum((item[0] + item[3] / 2) * item[6] for item in placed_items.values()) / total_weight
        y_cog = sum((item[1] + item[4] / 2) * item[6] for item in placed_items.values()) / total_weight

        return x_cog, y_cog
    

    def calculate_cog_deviation(self, placed_items, j):
        x_cog, y_cog = self.calculate_cog(placed_items)

        cog_devaition = max(abs(x_cog - j.length / 2) - 0.2 * j.length, 0) +  max(abs(y_cog - j.width / 2) - 0.2 * j.width, 0)

        return cog_devaition


    def place_item(self, item, ep, placed_items, orientation, current_extreme_points):
        """
        Place an item in the ULD

        Args:
            item (Item): The item to place.
            ep (tuple): The extreme point to place the item at.
            placed_items (dict): Dictionary of placed items.
            orientation (tuple): The orientation of the item.
            current_extreme_points (list): List of current extreme points.
        """
        item.x, item.y, item.z = ep
        item.length, item.width, item.height = orientation
        placed_items[item.serialnumber] = [item.x, item.y, item.z, item.length, item.width, item.height, item.weight, item.stack] 

        removed_points = []
        for point in current_extreme_points:
            if (item.x <= point[0] <= item.x + item.length and
                item.y <= point[1] <= item.y + item.width and
                item.z <= point[2] <= item.z + item.height):
                removed_points.append(point)

        # Identify added points
        added_points = [
            (item.x + item.length, item.y, item.z),  
            (item.x, item.y + item.width, item.z),  
            (item.x, item.y, item.z + item.height),  
            (item.x + item.length, item.y + item.width, item.z),  
            (item.x + item.length, item.y, item.z + item.height),  
            (item.x, item.y + item.width, item.z + item.height), 
            (item.x + item.length, item.y + item.width, item.z + item.height),  #
        ]

        added_points_by_item = [point for point in added_points if point not in current_extreme_points]

        return placed_items, added_points_by_item, removed_points
    

    def calculate_residual_space(self, ep, placed_items, j):
        """
        Calculate the residual space in the ULD

        Args:
            ep (tuple): The extreme point to calculate the residual space from.
            placed_items (dict): Dictionary of placed items.
            j (ULD): The ULD to calculate the residual space for.
        """
        # Initialize residual space in all directions
        rs_left = ep[0]
        rs_right = j.length - ep[0]
        rs_front = ep[1]
        rs_back = j.width - ep[1]
        rs_up = j.height - ep[2]

        for _, (x, y, z, length, width, height, _, _) in placed_items.items():
            # Check space to the left
            if x + length <= ep[0]:
                rs_left = min(rs_left, ep[0] - (x + length))
            
            # Check space to the right
            if x >= ep[0]:
                rs_right = min(rs_right, x - ep[0])

            # Check space in front
            if y + width <= ep[1]:
                rs_front = min(rs_front, ep[1] - (y + width))

            # Check space in back
            if y >= ep[1]:
                rs_back = min(rs_back, y - ep[1])

            # Check space above
            if z >= ep[2]:
                rs_up = min(rs_up, z - ep[2])

        # Ensure that residual space is not negative
        rs_left = max(rs_left, 0)
        rs_right = max(rs_right, 0)
        rs_front = max(rs_front, 0)
        rs_back = max(rs_back, 0)
        rs_up = max(rs_up, 0)

        return rs_left, rs_right, rs_front, rs_back, rs_up

    
    def does_item_fit_uld(self, ep, orientation, j):
        """
        Check if the item fits in the ULD

        Args:
            ep (tuple): The extreme point to place the item at.
            orientation (tuple): The orientation of the item.
            j (ULD): The ULD to check if the item fits in.
        """
        return (ep[0] + orientation[0] <= j.length and
                ep[1] + orientation[1] <= j.width and
                ep[2] + orientation[2] <= j.height)


    def find_best_next_item_and_placement(self, items, j, placed_items, extreme_points):
        """
        Find the best next item and placement for this item in the ULD based on merit

        Args:
            item (Item): The item to find the best placement for.
            j (ULD): The ULD to find the best placement for the item in.
            placed_items (dict): Dictionary of placed items.
            extreme_points (list): List of extreme points.
        """
        best_overall_merit = float('inf')
        best_item_for_placement = None
        best_placement_details = None

        stackables = [i for i in items if i.stack == 0]
        non_stackables = [i for i in items if i.stack == 1]


        for prioritized_group in [stackables, non_stackables]:
            for item in prioritized_group:
                best_merit_for_item = float('inf')
                best_ep_for_item = None
                best_orientation_for_item = None
                best_support_count_for_item = float('inf')
                item_defer_reason = 'No feasible placement found'

                ground_eps = [ep for ep in extreme_points if ep[2] == 0]
                non_ground_eps = [ep for ep in extreme_points if ep[2] != 0]
            
                def try_place(eps):
                    nonlocal best_merit_for_item, best_ep_for_item, best_orientation_for_item, best_support_count_for_item, item_defer_reason
                    for ep in eps:
                        orientations = self.get_orientations(item)
                        for orientation in orientations:
                            length, width, height = orientation

                            if not self.does_item_fit_uld(ep, orientation, j):
                                item_defer_reason = 'item does not fit in ULD'
                                continue

                            rs_left, rs_right, rs_front, rs_back, rs_up = self.calculate_residual_space(ep, placed_items, j)
                            
                            if self.check_collision(ep, length, width, height, placed_items, item.stack):
                                item_defer_reason = 'item collides with other items'
                                continue
                            
                            # cut_merit = compute_cut_merit(ep, j)
                            merit = (max(rs_left - length, 0) + max(rs_right - length, 0) +
                                        max(rs_front - width, 0) + max(rs_back - width, 0) +
                                        max(rs_up - height, 0))
                                
                            stable, support_count = self.is_fully_supported(ep, orientation, placed_items, j)
                            if not stable:
                                item_defer_reason = 'item is not fully supported'
                                continue

                            if stable and (merit < best_merit_for_item or (merit == best_merit_for_item and support_count > best_support_count_for_item)):
                                best_merit_for_item = merit
                                best_ep_for_item = ep
                                best_orientation_for_item = orientation
                                best_support_count_for_item = support_count
                                item_defer_reason = ''

                try_place(ground_eps)

                if best_merit_for_item == float('inf'):
                    try_place(non_ground_eps)

                
                if best_merit_for_item < best_overall_merit:
                    best_overall_merit = best_merit_for_item
                    best_item_for_placement = item
                    best_placement_details = (best_ep_for_item, best_orientation_for_item, best_merit_for_item, best_support_count_for_item)

        return best_item_for_placement, best_placement_details, item_defer_reason

In [9]:
class Plot():
    def __init__(self) -> None:
        pass
    
    def BPP(self, cargo, results_1D_BPP, placed_items, extreme_points, color_map, folder_path):
        """ 
        Plot 3D BPP

        Args:
            cargo (Cargo): The cargo object.
            results_1D_BPP (dict): Dictionary of 1D BPP results.
            placed_items (dict): Dictionary of placed items.
            extreme_points (dict): Dictionary of extreme points.
            color_map (dict): Dictionary of colors for items.
            folder_path (str): The folder path to save the plots to.
        """
        for j, items in results_1D_BPP.items():
            uld_serial = j.serialnumber
            fig = plt.figure(figsize=(10, 10))
            ax = fig.add_subplot(111, projection='3d')
            ax.set_title(f'{uld_serial}', fontsize=15)
        
            #Plane Plot
            plane_coords = [(j.a, 0, 0), (0, 0, j.b), (0, j.width, j.b), (j.a, j.width, 0)]
            plane = Poly3DCollection([plane_coords], alpha=0.3, edgecolor='black', facecolor='black')
            ax.add_collection3d(plane)

            #Items Plot
            for serialnumber in placed_items[j].keys():
                item_info = placed_items[j][serialnumber]
                prefix_serialnumber = str(serialnumber).split('-')[0] if '-' in str(serialnumber) else str(serialnumber)
                item_color = color_map[prefix_serialnumber]
                x, y, z, length, width, height, weight, stack = item_info
                ax.bar3d(x, y, z, length, width, height, color = item_color, alpha=0.3, edgecolor='black')
                ax.text(x + length / 2, y + width / 2, z + height / 2, f"{serialnumber}", fontsize=11, ha='center', va='center')

            #Extreme Points Plot
            for ep in extreme_points[j]:
                ax.scatter(ep[0], ep[1], ep[2], color='red')

            #Legend
            placed_count = {}
            prefix_groups = cargo.get_prefix_groups()

            for serialnumber in placed_items[j].keys():
                prefix_serialnumber = str(serialnumber).split('-')[0]
                placed_count[prefix_serialnumber] = placed_count.get(prefix_serialnumber, 0) + 1

            legend_patches = []
            for serialnumber in placed_items[j].keys():
                prefix_serialnumber = str(serialnumber).split('-')[0] if '-' in str(serialnumber) else str(serialnumber)
                item_color = color_map[prefix_serialnumber]
                label = f'{prefix_serialnumber}: {placed_count[prefix_serialnumber]}/{len(prefix_groups[prefix_serialnumber])}'
                if item_color not in [patch.get_facecolor() for patch in legend_patches]:
                        legend_patches.append(Patch(color = item_color, label = label ))

            #Special Handeling Patches 
            COL_items = [i.serialnumber for i in items if i.COL == 1]
            CRT_items = [i.serialnumber for i in items if i.CRT == 1]
            stack_items = [i.serialnumber for i in items if i.stack == 1]

            # patch_size_length = 20 
            # patch_size_width = 20  
            # patch_height = 0

            for serialnumber in COL_items:
                if serialnumber in placed_items[j]:
                    item_info = placed_items[j][serialnumber]
                    x, y, z, length, width, height, weight, stack = item_info
                    patch_size_length = length * 0.1
                    patch_size_width = width * 0.1  
                    patch_height = 0
                    fontsize = 10 
                    patch_x = x
                    patch_y = y + width - patch_size_width
                    patch_z = z + height
                    ax.bar3d(patch_x, patch_y, patch_z, patch_size_length, patch_size_width, patch_height, color='aqua', alpha=0.4, edgecolor='white')
                    ax.text(patch_x + patch_size_length / 2, patch_y + patch_size_width / 2, patch_z, "COL", color='black', ha='center', va='center', weight = 'bold', fontsize=fontsize, alpha = 1)

            for serialnumber in CRT_items:
                if serialnumber in placed_items[j]:
                    item_info = placed_items[j][serialnumber]
                    x, y, z, length, width, height, weight, stack = item_info
                    patch_size_length = length * 0.1
                    patch_size_width = width * 0.1  
                    patch_height = 0
                    fontsize = 10
                    patch_x = x
                    patch_y = y + width - patch_size_width
                    patch_z = z + height
                    ax.bar3d(patch_x, patch_y, patch_z, patch_size_length, patch_size_width, patch_height, color='lime', alpha=0.4, edgecolor='white')
                    ax.text(patch_x + patch_size_length / 2, patch_y + patch_size_width / 2, patch_z, "CRT", color='black', ha='center', va='center', weight = 'bold', fontsize=fontsize, alpha = 1)


            for serialnumber in stack_items:
                if serialnumber in placed_items[j]:
                    item_info = placed_items[j][serialnumber]
                    x, y, z, length, width, height, weight, stack = item_info
                    patch_size_length = length * 0.1
                    patch_size_width = width * 0.1  
                    patch_height = 0
                    fontsize = 10
                    patch_x = x
                    patch_y = y + width - patch_size_width
                    patch_z = z + height
                    ax.bar3d(patch_x, patch_y, patch_z, patch_size_length, patch_size_width, patch_height, color='gold', alpha=0.4, edgecolor='white')
                    ax.text(patch_x + patch_size_length / 2, patch_y + patch_size_width / 2, patch_z, "STK", color='black', ha='center', va='center', weight = 'bold', fontsize=fontsize, alpha = 1)

            #Axis and Limit Plot
            ax.set_xlabel('Length [cm]', fontsize=11)
            ax.set_ylabel('Width [cm]', fontsize=11)
            ax.set_zlabel('Height [cm]', fontsize=11)
            ax.set_xlim(0, j.length)
            ax.set_ylim(0, j.width)
            ax.set_zlim(0, j.height)
            ax.legend(handles=legend_patches, title="Serialnumbers", bbox_to_anchor=(1.05, 1))
            ax.set_box_aspect([1, 1, 1])

            save_path = os.path.join(folder_path, f'{uld_serial}.png')
            plt.savefig(save_path, bbox_inches='tight')
            plt.show()


    def WB(self, aircraft, ZFW_index, TOW_index, folder_path):
        """ 
        Plot CG Envelope

        Args:
            aircraft (Aircraft): The aircraft to plot the CG envelope for.
            ZFW_index (float): The index for the Zero Fuel Weight.
            TOW_index (float): The index for the Takeoff Weight.
            folder_path (str): The folder path to save the plot.
        """

        #Plotting the CG envelope for flight
        index_axis_ZFW, weight_axis_ZFW = aircraft.define_axis_ZFW(aircraft.aircraft_type) 
        index_axis_TOW, weight_axis_TOW = aircraft.define_axis_TOW(aircraft.aircraft_type)
        index_axis_LW, weight_axis_LW = aircraft.define_axis_LW(aircraft.aircraft_type) 

        fig, ax = plt.subplots(figsize=(10, 10)) 

        ax.plot(ZFW_index, aircraft.ZFW, marker='X', color='blue', label='ZFW')
        ax.text(ZFW_index, aircraft.ZFW + 30, f'ZFW: {ZFW_index:.1f}', fontsize=8, ha = 'center', va='bottom', fontweight = 'bold')
        ax.plot(aircraft.define_INDEX_ZFW_fwd(aircraft.aircraft_type), aircraft.ZFW, marker='_', color='blue', label='ZFW fwd')
        ax.text(aircraft.define_INDEX_ZFW_fwd(aircraft.aircraft_type) - 10, aircraft.ZFW, f'ZFW fwd: {aircraft.define_INDEX_ZFW_fwd(aircraft.aircraft_type):.1f}', fontsize=8, ha='right', fontstyle = 'italic')
        ax.plot(aircraft.define_INDEX_ZFW_aft(aircraft.aircraft_type), aircraft.ZFW, marker='_', color='blue', label='ZFW aft')
        ax.text(aircraft.define_INDEX_ZFW_aft(aircraft.aircraft_type) + 10, aircraft.ZFW, f'ZFW aft: {aircraft.define_INDEX_ZFW_aft(aircraft.aircraft_type):.1f}', fontsize=8, ha='left', fontstyle = 'italic')

        ax.plot(TOW_index, aircraft.TOW, marker='D', color='red', label='TOW')
        ax.text(TOW_index, aircraft.TOW + 30, f'TOW: {TOW_index:.1f}', fontsize=8, ha = 'center', va='bottom', fontweight = 'bold')
        ax.plot(aircraft.define_INDEX_TOW_fwd(aircraft.aircraft_type), aircraft.TOW, marker='_', color='red', label='TOW fwd')
        ax.text(aircraft.define_INDEX_TOW_fwd(aircraft.aircraft_type) - 5, aircraft.TOW, f'TOW fwd: {aircraft.define_INDEX_TOW_fwd(aircraft.aircraft_type):.1f}', fontsize=8, ha='right', fontstyle = 'italic')
        ax.plot(aircraft.define_INDEX_TOW_aft(aircraft.aircraft_type), aircraft.TOW, marker='_', color='red', label='TOW aft')
        ax.text(aircraft.define_INDEX_TOW_aft(aircraft.aircraft_type) + 5, aircraft.TOW, f'TOW aft: {aircraft.define_INDEX_TOW_aft(aircraft.aircraft_type):.1f}', fontsize=8, ha='left', fontstyle = 'italic')

        ax.plot(aircraft.define_INDEX_LW(ZFW_index), aircraft.LW, marker='s', color='green', label='LW')
        ax.text(aircraft.define_INDEX_LW(ZFW_index), aircraft.LW + 30, f'LW: {aircraft.define_INDEX_LW(ZFW_index):.1f}', fontsize=8, ha = 'center', va='bottom', fontweight = 'bold')
        ax.plot(aircraft.define_INDEX_LW_fwd(aircraft.aircraft_type), aircraft.LW, marker='_', color='green', label='LW fwd')
        ax.text(aircraft.define_INDEX_LW_fwd(aircraft.aircraft_type) - 10, aircraft.LW, f'LW fwd: {aircraft.define_INDEX_LW_fwd(aircraft.aircraft_type):.1f}', fontsize=8, ha='right', fontstyle = 'italic')
        ax.plot(aircraft.define_INDEX_LW_aft(aircraft.aircraft_type), aircraft.LW, marker='_', color='green', label='LW aft')
        ax.text(aircraft.define_INDEX_LW_aft(aircraft.aircraft_type) + 10, aircraft.LW, f'LW aft: {aircraft.define_INDEX_LW_aft(aircraft.aircraft_type):.1f}', fontsize=8, ha='left', fontstyle = 'italic')

        ax.plot(index_axis_ZFW, weight_axis_ZFW, linestyle='-', color='blue', label='ZFW')
        ax.plot(index_axis_TOW, weight_axis_TOW, linestyle='-', color='red', label='TOW')
        ax.plot(index_axis_LW, weight_axis_LW, linestyle='-', color='green', label='LW')

        ax.set_xlim(0, 200)
        if aircraft.aircraft_type == '789' and aircraft.aircraft_type == '781':
            ax.set_ylim(100000, 300000)

        elif aircraft.aircraft_type == '77W':
            ax.set_ylim(100000, 400000)

        elif aircraft.aircraft_type == '772':
            ax.set_y_lim(100000, 350000)

        ax.set_xlabel('Index', fontsize=11)
        ax.set_ylabel('Weight [kg]', fontsize=11)
        ax.set_title('CG Envelope', fontsize=15)

        ax.grid(axis='y', linestyle='--', color='gray', alpha=0.7)

        save_path = os.path.join(folder_path, f'CG_envelope.png')
        plt.savefig(save_path, bbox_inches='tight')
        plt.show()


In [ ]:
class ProjectSetup:
    def __init__(self) -> None:
        self.logger = logging.getLogger(__name__)

    def setup_logger(self, directory_path, log_filename='General_Information.txt'):
        """
        Setup the logger

        Args:
            directory_path (str): The directory path to save the log file.
            log_filename (str): The name of the log file.
        """
        log_file_path = os.path.join(directory_path, log_filename)
        
        # Clear any existing handlers
        self.logger.handlers = []
        
        # Create a file handler for writing to file
        file_handler = logging.FileHandler(log_file_path, 'w')
        file_formatter = logging.Formatter('%(message)s')
        file_handler.setFormatter(file_formatter)
        self.logger.addHandler(file_handler)
        
        # Create a stream handler for printing to console
        stream_handler = logging.StreamHandler(sys.stdout)
        stream_formatter = logging.Formatter('%(message)s')
        stream_handler.setFormatter(stream_formatter)
        self.logger.addHandler(stream_handler)
        
        self.logger.setLevel(logging.INFO)

    def setup_project_directory(self, flight_number, date, departure, arrival, baseline, optimized_actual, BAX_fixed=False):
        """ 
        Setup the project directory

        Args:
            flight_number (str): The flight number.
            date (str): The date of the flight.
            departure (str): The departure airport.
            arrival (str): The arrival airport.
        """
        ## esto antes estaba con los paths de windows, los tuve que cambiar para que funcione en mac, asi como estos son 26 paths mas
        if baseline:
            base_path = 'Results_Baseline'  # Changed from Windows path
        if optimized_actual:
            base_path = 'Results_Optimized_Actual'  # Changed from Windows path
        if BAX_fixed:
            base_path = 'Results_BAX_Fixed'  # Changed from Windows path
        if not baseline and not optimized_actual and not BAX_fixed:
            base_path = 'Results'  # Changed from Windows path

        date_split = date.split(' ')
        folder_name = f'Results {departure}{arrival} {date_split[1]} 20{date_split[2]}'

        base_path = os.path.join(base_path, folder_name)
        directory_name = f"Flight {flight_number} {departure}{arrival} {date}"
        full_path = os.path.join(base_path, directory_name)

        if not os.path.exists(full_path):
            os.makedirs(full_path)
        return full_path
    
    def setup_project(self, piece_information_csv, flight_information_csv, load_locations_csv, pax_information_csv, buildup_information_csv, arrival_airport, restricted_locations, baseline = False, optimized_actual = False, BAX_fixed = False):
        """
        Setup the project

        Args:
            piece_information_csv (str): The path to the piece information csv file.
            flight_information_csv (str): The path to the flight information xlsx file.
            load_locations_csv (str): The path to the load locations xlsx file.
            pax_information_csv (str): The path to the pax information csv file.
        """
        data_analysis = Data_Analysis()
        if optimized_actual:
            cargo = CargoActual()
        if not optimized_actual:
            cargo = Cargo() # ver la diff 
        aircraft = Aircraft()
        EP = Extreme_Points()
        plot = Plot()
        
        cargo.read_cargo_pieces(piece_information_csv, buildup_information_csv, arrival_airport)
        aircraft.define_flight_information(flight_information_csv)
        
        #tambien cambie esto para q los paths sean relativos:
        aircraft.define_aircraft('Inputfiles/LoadLocations.csv', aircraft.aircraft_type, restricted_locations)  # Changed from Windows path
        aircraft.define_fuel_index('Inputfiles/Fuel_Index.csv', aircraft.aircraft_type)  # Changed from Windows path
        aircraft.define_max_weight_information('Inputfiles/Main.csv', aircraft.aircraft_type)  # Changed from Windows path
        aircraft.define_DOI('Inputfiles/Dry_Operating_Index.csv', aircraft.aircraft_type)  # Changed from Windows path
        aircraft.define_envelope_information(aircraft.aircraft_type)
        aircraft.define_PAX_allocation(pax_information_csv)

        cargo.define_ulds_used_flight(load_locations_csv, buildup_information_csv)
        cargo.define_parameters_ULD()
        cargo.define_uld_cut_sets()
        cargo.total_number_of_build_ULDs = len([j for j in cargo.uld if j.isNeitherBAXnorBUPnorT])

        items_not_fit = [item for item in cargo.items if not any(item.length <= j.length and item.width <= j.width and item.height <= j.height for j in cargo.uld)]
        total_items = len(cargo.items)
        
        aircraft.ZFW = aircraft.OEW + sum([i.weight for i in cargo.items]) + sum([j.weight for j in cargo.uld if j.isBAXorBUPorT]) + aircraft.define_PAX_weight() - sum([i.weight for i in items_not_fit])
        aircraft.TOW = aircraft.ZFW + aircraft.TOF
        aircraft.LW = aircraft.TOW - aircraft.TripF

        number_of_BAX_ULDs = len([j for j in cargo.uld if j.isBAX])
        number_of_BUP_ULDs = len([j for j in cargo.uld if 'BUP' in j.serialnumber])
        number_of_T_ULDs = len([j for j in cargo.uld if 'T' in j.serialnumber])
        number_of_cargo_ULDs = len([j for j in cargo.uld if j.isNeitherBAXnorBUPnorT])

        directory_path = self.setup_project_directory(aircraft.flight_number, aircraft.date, aircraft.departure_airport, aircraft.arrival_airport, baseline, optimized_actual, BAX_fixed)
        self.setup_logger(directory_path)

        '''Print Statements'''
        self.logger.info('The model will run the following flight:')
        self.logger.info('--------------------------------------------------------------------------------------------------------')
        self.logger.info(f'Flight Number: {aircraft.flight_number} {aircraft.aircraft_registration} - Aircraft Type: {aircraft.aircraft_type} - Departure: {aircraft.departure_airport} - Arrival: {aircraft.arrival_airport} - Date: {aircraft.date}')
        self.logger.info('')
        self.logger.info(f'ZFW: {aircraft.ZFW:.1f} kg - TOW: {aircraft.TOW:.1f} kg - LW: {aircraft.LW:.1f} kg - OEW: {aircraft.OEW:.1f} kg')
        self.logger.info('')
        self.logger.info(f'Fuel: {aircraft.TOF} kg - Trip Fuel: {aircraft.TripF} kg')
        self.logger.info('')
        self.logger.info(f'Number of Cargo ULDs used: {number_of_cargo_ULDs}')
        self.logger.info('')
        self.logger.info(f'Number of BAX ULDs: {number_of_BAX_ULDs} - Number of BUP ULDs: {number_of_BUP_ULDs} - Number of T ULDs: {number_of_T_ULDs}')
        self.logger.info('--------------------------------------------------------------------------------------------------------')
        self.logger.info('ULD set available for loading:')
        for j in cargo.uld:
            if j.isNeitherBAXnorBUPnorT:
                self.logger.info(f'{j.serialnumber}, {j.type}')

        self.logger.info('--------------------------------------------------------------------------------------------------------')

        self.logger.info('BAX and BUP set for loading:')
        for j in cargo.uld:   
            if j.isBAXorBUPorT:
                self.logger.info(f'{j.serialnumber}, {j.type}')
        self.logger.info('--------------------------------------------------------------------------------------------------------')
        self.logger.info('PAX allocation:')
        self.logger.info(f'PAX-A: {aircraft.pax_0A}')
        self.logger.info(f'PAX-B: {aircraft.pax_0B}')
        self.logger.info(f'PAX-C: {aircraft.pax_0C}')
        self.logger.info(f'PAX-D: {aircraft.pax_0D}')
        self.logger.info(f'PAX-E: {aircraft.pax_0E}')
        self.logger.info(f'PAX-F: {aircraft.pax_0F}')
        self.logger.info(f'PAX-G: {aircraft.pax_0G}')
        self.logger.info(f'Total PAX: {aircraft.total_PAX}')
        self.logger.info('--------------------------------------------------------------------------------------------------------')
        self.logger.info('Check if all items fit the ULDs dimensions:')
        if items_not_fit:
            self.logger.info('--------------------------------------------------------------------------------------------------------')
            self.logger.info('The following items do not fit the dimensions of the ULDs:')
            for item in items_not_fit:
                self.logger.info(f'Serialnumber: {item.serialnumber}')
                self.logger.info(f'Dimensions: {item.length, item.width, item.height}')
                cargo.items.remove(item)
            self.logger.info('--------------------------------------------------------------------------------------------------------')
        self.logger.info('')
        self.logger.info(f'{len(cargo.items)} / {total_items} items fit the dimensions of the ULDs')
        self.logger.info('--------------------------------------------------------------------------------------------------------')

        return cargo, aircraft, EP, plot, data_analysis

In [11]:
class DataExtraction():
    def __init__(self):
        self.df_flight_details = 'Inputfiles/FlightDetailsSpotfire.csv'  # Changed from Windows path
        self.df_load_locations = 'Inputfiles/LoadLocationsSpotfire.csv'  # Changed from Windows path
        self.df_piece_information = 'Inputfiles/PieceInformationSpotfire.csv'  # Changed from Windows path
        self.df_pax_information = 'Inputfiles/PaxInformationSpotfire.csv'  # Changed from Windows path
        self.df_buildup_information = 'Inputfiles/BuildUpInformationSpotfire.csv'  # Changed from Windows path
    def custom_date_parser(self, date_string):
        """
        Parses date strings that are in either 'd/m/YYYY' or 'd-m-YYYY' format.

        Args:
            date_string (str): The date string to parse.

        Returns:
            datetime: The parsed date as a datetime object.
        """
        try:
            return datetime.strptime(date_string, '%d/%m/%Y')
        except ValueError:
            return datetime.strptime(date_string, '%d-%m-%Y')
        
        
    def standardize_decimal(self, value):
        if isinstance(value, str):
            cleaned_value = value.replace(',', '.')
            try:
                return float(cleaned_value)
            except ValueError:
                pass 
        return value
    
        
    def setup_project_directory(self, flight_number, date, departure, arrival):
        """ 
        Setup the project directory

        Args:
            flight_number (str): The flight number.
            date (str): The date of the flight.
            departure (str): The departure airport.
            arrival (str): The arrival airport.
        """

        base_path = 'Data_Common_102' ## lo cambie yo
        date_split = date.split(' ')
        folder_name = f'Flights {departure}{arrival} {date_split[1]} {date_split[2]}'

        base_path = os.path.join(base_path, folder_name)
        directory_name = f"Flight {flight_number} {departure}{arrival} {date}"
        full_path = os.path.join(base_path, directory_name)
        if not os.path.exists(full_path):
            os.makedirs(full_path)
        return full_path
    

    def extract_individual_flight_details(self):
        """ 
        Extract the flight details data from spotfire csv file
        """
        flight_details_data = pd.read_csv(self.df_flight_details, parse_dates=['LegDepartureDateUtc'], date_parser = self.custom_date_parser)
        flight_details_data['MacZFW'] = flight_details_data['MacZFW'].apply(lambda x: round(float(str(x).replace(',', '')) / 10**7, 1))
        unique_flights = flight_details_data.drop_duplicates(subset=['FlightNumber', 'LegDepartureDateUtc', 'DepartureAirport', 'ArrivalAirport'])

        for index, flight in unique_flights.iterrows():
            flight_number = flight['FlightNumber']
            flight_number_folder = f'KL0{flight_number}'
            date = flight['LegDepartureDateUtc']
            date_folder = date.strftime('%d %b %Y').upper()
            departure = flight['DepartureAirport']
            arrival = flight['ArrivalAirport']    

            directory = self.setup_project_directory(flight_number_folder, date_folder, departure, arrival)
            
            flight_details = flight_details_data[(flight_details_data['FlightNumber'] == flight_number) & 
                                                 (flight_details_data['LegDepartureDateUtc'] == date) &
                                                 (flight_details_data['DepartureAirport'] == departure) & 
                                                 (flight_details_data['ArrivalAirport'] == arrival)]
            
            flight_details.to_csv(os.path.join(directory, 'FlightInformation.csv'), index=False)


    def extract_individual_load_locations(self):
        """ 
        Extract the load locations data from spotfire csv file
        """
        load_locations_data = pd.read_csv(self.df_load_locations)
        unique_flights = load_locations_data.drop_duplicates(subset=['FlightLegDepartureKey'])

        for index, flight in unique_flights.iterrows():
            parts = flight['FlightLegDepartureKey'].split('|')
            flight_number = parts[2]
            flight_number_folder = f'KL{flight_number}'
            date = pd.to_datetime(parts[0])
            date_folder = date.strftime('%d %b %Y').upper()
            departure = flight['DeadloadOrigin']
            arrival = flight['DeadloadDestination']

            load_locations = load_locations_data[load_locations_data['FlightLegDepartureKey'] == flight['FlightLegDepartureKey']]

            directory = self.setup_project_directory(flight_number_folder, date_folder, departure, arrival)            
            load_locations.to_csv(os.path.join(directory, 'LoadLocations.csv'), index=False)

    def extract_individual_piece_information(self):
        """ 
        Extract the piece information data from spotfire csv file
        """
        converters = {'BookingTotalVolume': self.standardize_decimal,
                'BookingTotalWeight': self.standardize_decimal,
                'BookingSegmentVolume': self.standardize_decimal,
                'BookingSegmentWeight': self.standardize_decimal,
                'BookingSegmentPiecesVolume': self.standardize_decimal,
                'BookingSegmentPiecesWeight': self.standardize_decimal,
                'BookingLinePieceVolume': self.standardize_decimal,
                'BookingLinePieceWeight': self.standardize_decimal,
                'BookingLinePieceHeight': self.standardize_decimal,
                'BookingLinePieceWidth': self.standardize_decimal,
                'BookingLinePieceLength': self.standardize_decimal
                }
        piece_information_data = pd.read_csv(self.df_piece_information, parse_dates = ['BookingSegmentFlightDateLT'], date_parser = self.custom_date_parser, converters=converters)
        unique_flights = piece_information_data.drop_duplicates(subset=['BookingSegmentFlightNumber', 'BookingSegmentFlightDateLT',
                                                                        'BookingSegmentBoardPointStationCode', 'BookingSegmentOffPointStationCode'])

        for index, flight in unique_flights.iterrows():
            flight_number = flight['BookingSegmentFlightNumber']
            flight_number_folder = f'KL0{flight_number}'
            date = pd.to_datetime(flight['BookingSegmentFlightDateLT'])
            date_folder = date.strftime('%d %b %Y').upper()
            departure = flight['BookingSegmentBoardPointStationCode']
            arrival = flight['BookingSegmentOffPointStationCode']

            piece_information = piece_information_data[(piece_information_data['BookingSegmentFlightNumber'] == flight_number) &
                                                       (piece_information_data['BookingSegmentFlightDateLT'] == flight['BookingSegmentFlightDateLT']) &
                                                       (piece_information_data['BookingSegmentBoardPointStationCode'] == departure) &
                                                       (piece_information_data['BookingSegmentOffPointStationCode'] == arrival)]


            directory = self.setup_project_directory(flight_number_folder, date_folder, departure, arrival)            
            piece_information.to_csv(os.path.join(directory, 'PieceInformation.csv'), index=False)

    def extract_individual_pax_information(self):
        """ 
        Extract the pax information data from spotfire csv file
        """
        pax_information_data = pd.read_csv(self.df_pax_information, parse_dates=['LocalDepartureDate'], date_parser = self.custom_date_parser)
        unique_flights = pax_information_data.drop_duplicates(subset=['FlightNumber', 'LocalDepartureDate', 'DepartureAirport', 'SegmentArrivalAirport'])

        for index, flight in unique_flights.iterrows():
            flight_number = flight['FlightNumber']
            flight_number_folder = f'KL0{flight_number}'
            date = pd.to_datetime(flight['LocalDepartureDate'])
            date_folder = date.strftime('%d %b %Y').upper()
            departure = flight['DepartureAirport']
            arrival = flight['SegmentArrivalAirport']

            pax_information = pax_information_data[(pax_information_data['FlightNumber'] == flight_number) &
                                                   (pax_information_data['LocalDepartureDate'] == flight['LocalDepartureDate']) &
                                                   (pax_information_data['DepartureAirport'] == departure) &
                                                   (pax_information_data['SegmentArrivalAirport'] == arrival)]

            directory = self.setup_project_directory(flight_number_folder, date_folder, departure, arrival)            
            pax_information.to_csv(os.path.join(directory, 'PaxInformation.csv'), index=False)

    def extract_individual_buildup_information(self):
        """  
        Extract the buildup information data from spotfire csv file
        """
        converters = {'TotalWeightOnAWB': self.standardize_decimal}

        buildup_information_data = pd.read_csv(self.df_buildup_information, parse_dates=['DepartureFlightDate'], date_parser = self.custom_date_parser, converters=converters)
        unique_flights = buildup_information_data.drop_duplicates(subset=['DepartureFlightNumber', 'DepartureFlightDate'])

        for index, flight in unique_flights.iterrows():
            flight_number = flight['DepartureFlightNumber']
            flight_number_folder = f'KL0{flight_number}'
            date = pd.to_datetime(flight['DepartureFlightDate'])
            date_folder = date.strftime('%d %b %Y').upper()
            arrival_airport = flight['DestinationStationCode']

            buildup_information = buildup_information_data[(buildup_information_data['DepartureFlightNumber'] == flight_number) &
                                                           (buildup_information_data['DepartureFlightDate'] == flight['DepartureFlightDate'])]

            directory = self.setup_project_directory(flight_number_folder, date_folder, 'AMS', arrival_airport)            
            buildup_information.to_csv(os.path.join(directory, 'BuildUpInformation.csv'), index=False)

    def delete_incorrect_folders(self):
        """ 
        Delete the incorrect folders that do not contain the 3 files
        """
        base_path = 'Data_Common_102'
        required_files = ['FlightInformation.csv', 'LoadLocations.csv', 'PieceInformation.csv', 'PaxInformation.csv', 'BuildUpInformation.csv']

        for months_folder in os.listdir(base_path):
            months_folder_path = os.path.join(base_path, months_folder)
            if not os.path.isdir(months_folder_path):
                continue
            for flight_folder in os.listdir(months_folder_path):
                folder_path = os.path.join(months_folder_path, flight_folder)
                if not os.path.isdir(folder_path):
                    continue
                if not all(os.path.exists(os.path.join(folder_path, file_name)) for file_name in required_files):
                    shutil.rmtree(folder_path)

In [12]:
class FilterData():
    def __init__(self):
        self.base_dir = 'Data_Common_102'  # Using relative path

    def per_month(self, departure_airport, arrival_airport, month, year):
        """
        Run the model for a specific month

        Args:
            departure_airport (str): The departure airport.
            arrival_airport (str): The arrival airport.
            month (str): The month to run the model for.
            year (str): The year to run the model for.
        """
        month = month.capitalize()
        year = str(year)
        month_folder_name = f'Flights {departure_airport}{arrival_airport} {month} {year}'

        month_folder = os.path.join(self.base_dir, month_folder_name)
        pattern = os.path.join(month_folder, 'Flight *')

        flights = glob.glob(pattern)

        print(f'{len(flights)} flights found for {departure_airport} to {arrival_airport} in {month} {year}')
        print('========================================================================================================')

        return flights
    
    def per_flight(self, flight_number, departure_airport, arrival_airport, day, month, year):
        """   
        Run the model for a specific flight

        Args:
            flight_number (str): The flight number.
            departure_airport (str): The departure airport.
            arrival_airport (str): The arrival airport.
            day (str): The day of the flight.
            month (str): The month of the flight.
            year (str): The year of the flight.
        """

        month = month.upper()
        year = str(year)
        flight_number = str(flight_number)
        day = str(day)
        month_folder_name = f'Flights {departure_airport}{arrival_airport} {month} {year}'
        flight_folder_name = f'Flight {flight_number} {departure_airport}{arrival_airport} {day} {month} {year}'

        flight_folder = os.path.join(self.base_dir, month_folder_name, flight_folder_name)
        print(f'Running the model for Flight {flight_number} {departure_airport}{arrival_airport} {day} {month} {year}')
        print('========================================================================================================')

        return flight_folder

In [13]:
class ResultsReader():
    def __init__(self, departure_airport = None, arrival_airport = None, month = None, year = None):
        self.departure_airport = departure_airport
        self.arrival_airport = arrival_airport
        self.month = month
        self.year = year
        self.aircraft_type = None
        self.aircraft_registration = None
        self.fuel_reduction_percentage = 0
        self.fuel_reduction_kg = 0
        self.uld_built_by_model = 0
        self.uld_actually_used = 0
        self.total_run_time = 0
        self.total_1D_BPP_WB_time = 0
        self.total_3D_BPP_time = 0
        self.MAC = 0
        self.MAC_actual = 0
        self.TOW = 0

    def read_all_results(self):
        """
        Read all the results from the results folder
        """
        results = []

        base_dir = 'Results'

        for root, dirs, files in os.walk(base_dir):
            for dir_name in dirs:
                if dir_name.startswith('Results '):  # Check if directory name starts with 'Results'
                    full_path = os.path.join(root, dir_name)
                    pattern = os.path.join(full_path, 'Flight *')
                    flights = glob.glob(pattern)

                    for flight in flights:
                        result = self.read_individual_flight_results(flight)
                        if result:
                            results.append(result)
        
        df = pd.DataFrame(results)

        sum_data = {
            'Flight Number': 'Total',
            'Flight Date': '-',
            'Fuel Deviation Percentage': '-',
            'Fuel Deviation': df['Fuel Deviation'].sum(),
            'TOW': df['TOW'].sum(),
            'ULDs Built by Model': df['ULDs Built by Model'].sum(),
            'ULDs Actually Used': df['ULDs Actually Used'].sum(),
            '%MAC ZFW': '-',
            'Actual %MAC ZFW': '-',
            'Total Run Time': df['Total Run Time'].sum(),
            'Total 1D BPP WB Time': df['Total 1D BPP WB Time'].sum(),
            'Total 3D BPP Time': df['Total 3D BPP Time'].sum(),
            'Number of BAX ULDs': df['Number of BAX ULDs'].sum(),
            'Number of BUP ULDs': df['Number of BUP ULDs'].sum(),
            'Number of T ULDs': df['Number of T ULDs'].sum(),
            'Number of PAX A': '-',
            'Number of PAX B': '-',
            'Number of PAX C': '-',
            'Number of PAX D': '-',
            'Number of PAX E': '-',
            'Number of PAX F': '-',
            'Number of PAX G': '-',
            'Total PAX': '-',
            'Number of items': df['Number of items'].sum(),
            'Weight in Comp 1': df['Weight in Comp 1'].sum(),
            'Weight in Comp 2': df['Weight in Comp 2'].sum(),
            'Weight in Comp 3': df['Weight in Comp 3'].sum(),
            'Weight in Comp 4': df['Weight in Comp 4'].sum(),
            'Number of ULDs in Comp 1': df['Number of ULDs in Comp 1'].sum(),
            'Number of BAX ULDs in Comp 1': df['Number of BAX ULDs in Comp 1'].sum(),
            'Weight ULDs in Comp 1': df['Weight ULDs in Comp 1'].sum(),
            'Weight BAX ULDs in Comp 1': df['Weight BAX ULDs in Comp 1'].sum(),
            'Number of ULDs in Comp 2': df['Number of ULDs in Comp 2'].sum(),
            'Number of BAX ULDs in Comp 2': df['Number of BAX ULDs in Comp 2'].sum(),
            'Weight ULDs in Comp 2': df['Weight ULDs in Comp 2'].sum(),
            'Weight BAX ULDs in Comp 2': df['Weight BAX ULDs in Comp 2'].sum(),
            'Number of ULDs in Comp 3': df['Number of ULDs in Comp 3'].sum(),
            'Number of BAX ULDs in Comp 3': df['Number of BAX ULDs in Comp 3'].sum(),
            'Weight ULDs in Comp 3': df['Weight ULDs in Comp 3'].sum(),
            'Weight BAX ULDs in Comp 3': df['Weight BAX ULDs in Comp 3'].sum(),
            'Number of ULDs in Comp 4': df['Number of ULDs in Comp 4'].sum(),
            'Number of BAX ULDs in Comp 4': df['Number of BAX ULDs in Comp 4'].sum(),
            'Weight ULDs in Comp 4': df['Weight ULDs in Comp 4'].sum(),
            'Weight BAX ULDs in Comp 4': df['Weight BAX ULDs in Comp 4'].sum(),
        }

        average_data = {
            'Flight Number': 'Average',
            'Flight Date': '-',
            'Fuel Deviation Percentage': df['Fuel Deviation Percentage'].mean(),
            'Fuel Deviation': df['Fuel Deviation'].mean(),
            'TOW': df['TOW'].mean(),
            'ULDs Built by Model': df['ULDs Built by Model'].mean(),
            'ULDs Actually Used': df['ULDs Actually Used'].mean(),
            '%MAC ZFW': df['%MAC ZFW'].mean(),
            'Actual %MAC ZFW': df['Actual %MAC ZFW'].mean(),
            'Total Run Time': df['Total Run Time'].mean(),
            'Total 1D BPP WB Time': df['Total 1D BPP WB Time'].mean(),
            'Total 3D BPP Time': df['Total 3D BPP Time'].mean(),
            'Number of BAX ULDs': df['Number of BAX ULDs'].mean(),
            'Number of BUP ULDs': df['Number of BUP ULDs'].mean(),
            'Number of T ULDs': df['Number of T ULDs'].mean(),
            'Number of PAX A': df['Number of PAX A'].mean(),
            'Number of PAX B': df['Number of PAX B'].mean(),
            'Number of PAX C': df['Number of PAX C'].mean(),
            'Number of PAX D': df['Number of PAX D'].mean(),
            'Number of PAX E': df['Number of PAX E'].mean(),
            'Number of PAX F': df['Number of PAX F'].mean(),
            'Number of PAX G': df['Number of PAX G'].mean(),
            'Total PAX': df['Total PAX'].mean(),
            'Number of items': df['Number of items'].mean(),
            'Weight in Comp 1': df['Weight in Comp 1'].mean(),
            'Weight in Comp 2': df['Weight in Comp 2'].mean(),
            'Weight in Comp 3': df['Weight in Comp 3'].mean(),
            'Weight in Comp 4': df['Weight in Comp 4'].mean(),
            'Number of ULDs in Comp 1': df['Number of ULDs in Comp 1'].mean(),
            'Number of BAX ULDs in Comp 1': df['Number of BAX ULDs in Comp 1'].mean(),
            'Weight ULDs in Comp 1': df['Weight ULDs in Comp 1'].mean(),
            'Weight BAX ULDs in Comp 1': df['Weight BAX ULDs in Comp 1'].mean(),
            'Number of ULDs in Comp 2': df['Number of ULDs in Comp 2'].mean(),
            'Number of BAX ULDs in Comp 2': df['Number of BAX ULDs in Comp 2'].mean(),
            'Weight ULDs in Comp 2': df['Weight ULDs in Comp 2'].mean(),
            'Weight BAX ULDs in Comp 2': df['Weight BAX ULDs in Comp 2'].mean(),
            'Number of ULDs in Comp 3': df['Number of ULDs in Comp 3'].mean(),
            'Number of BAX ULDs in Comp 3': df['Number of BAX ULDs in Comp 3'].mean(),
            'Weight ULDs in Comp 3': df['Weight ULDs in Comp 3'].mean(),
            'Weight BAX ULDs in Comp 3': df['Weight BAX ULDs in Comp 3'].mean(),
            'Number of ULDs in Comp 4': df['Number of ULDs in Comp 4'].mean(),
            'Number of BAX ULDs in Comp 4': df['Number of BAX ULDs in Comp 4'].mean(),
            'Weight ULDs in Comp 4': df['Weight ULDs in Comp 4'].mean(),
            'Weight BAX ULDs in Comp 4': df['Weight BAX ULDs in Comp 4'].mean()
        }

        sum_df = pd.DataFrame([sum_data])
        average_df = pd.DataFrame([average_data])

        df = pd.concat([df, sum_df, average_df], ignore_index=True)

        col_order = ['Flight Number', 'Flight Date', 'Aircraft Type', 'Aircraft Registration', '%MAC ZFW', 'Actual %MAC ZFW', 'Fuel Deviation Percentage',
        'Fuel Deviation', 'TOW', 'ULDs Built by Model', 'ULDs Actually Used', 'Total Run Time', 'Total 1D BPP WB Time', 'Total 3D BPP Time',
        'Number of BAX ULDs', 'Number of BUP ULDs','Number of T ULDs', 'Number of PAX A', 'Number of PAX B', 'Number of PAX C',
        'Number of PAX D', 'Number of PAX E', 'Number of PAX F', 'Number of PAX G', 'Total PAX', 'Number of items',
        'Weight in Comp 1', 'Weight in Comp 2', 'Weight in Comp 3', 'Weight in Comp 4', 'Number of ULDs in Comp 1',
        'Number of BAX ULDs in Comp 1',	'Weight ULDs in Comp 1', 'Weight BAX ULDs in Comp 1', 'Number of ULDs in Comp 2',
        'Number of BAX ULDs in Comp 2',	'Weight ULDs in Comp 2', 'Weight BAX ULDs in Comp 2', 'Number of ULDs in Comp 3',
        'Number of BAX ULDs in Comp 3', 'Weight ULDs in Comp 3', 'Weight BAX ULDs in Comp 3', 'Number of ULDs in Comp 4',	
        'Number of BAX ULDs in Comp 4', 'Weight ULDs in Comp 4', 'Weight BAX ULDs in Comp 4'
        ]

        df = df[col_order]

        return df

    def read_results(self):
        """
        Read the results from the results folder and display them in a DataFrame
        """

        base_dir = 'Results'
        month_folder_name = f'Results {self.departure_airport}{self.arrival_airport} {self.month} {self.year}'
        month_folder = os.path.join(base_dir, month_folder_name)
        pattern = os.path.join(month_folder, 'Flight *')

        flights = glob.glob(pattern)
        results = []


        for flight in flights:
            result = self.read_individual_flight_results(flight)
            if result is not None:
                results.append(result)

        df = pd.DataFrame(results)

    
        sum_data = {
            'Flight Number': 'Total',
            'Flight Date': '-',
            'Fuel Deviation Percentage': '-',
            'Fuel Deviation': df['Fuel Deviation'].sum(),
            'TOW': df['TOW'].sum(),
            'ULDs Built by Model': df['ULDs Built by Model'].sum(),
            'ULDs Actually Used': df['ULDs Actually Used'].sum(),
            '%MAC ZFW': '-',
            'Actual %MAC ZFW': '-',
            'Total Run Time': df['Total Run Time'].sum(),
            'Total 1D BPP WB Time': df['Total 1D BPP WB Time'].sum(),
            'Total 3D BPP Time': df['Total 3D BPP Time'].sum(),
            'Number of BAX ULDs': df['Number of BAX ULDs'].sum(),
            'Number of BUP ULDs': df['Number of BUP ULDs'].sum(),
            'Number of T ULDs': df['Number of T ULDs'].sum(),
            'Number of PAX A': '-',
            'Number of PAX B': '-',
            'Number of PAX C': '-',
            'Number of PAX D': '-',
            'Number of PAX E': '-',
            'Number of PAX F': '-',
            'Number of PAX G': '-',
            'Total PAX': '-',
            'Number of items': df['Number of items'].sum(),
            'Weight in Comp 1': df['Weight in Comp 1'].sum(),
            'Weight in Comp 2': df['Weight in Comp 2'].sum(),
            'Weight in Comp 3': df['Weight in Comp 3'].sum(),
            'Weight in Comp 4': df['Weight in Comp 4'].sum(),
            'Number of ULDs in Comp 1': df['Number of ULDs in Comp 1'].sum(),
            'Number of BAX ULDs in Comp 1': df['Number of BAX ULDs in Comp 1'].sum(),
            'Weight ULDs in Comp 1': df['Weight ULDs in Comp 1'].sum(),
            'Weight BAX ULDs in Comp 1': df['Weight BAX ULDs in Comp 1'].sum(),
            'Number of ULDs in Comp 2': df['Number of ULDs in Comp 2'].sum(),
            'Number of BAX ULDs in Comp 2': df['Number of BAX ULDs in Comp 2'].sum(),
            'Weight ULDs in Comp 2': df['Weight ULDs in Comp 2'].sum(),
            'Weight BAX ULDs in Comp 2': df['Weight BAX ULDs in Comp 2'].sum(),
            'Number of ULDs in Comp 3': df['Number of ULDs in Comp 3'].sum(),
            'Number of BAX ULDs in Comp 3': df['Number of BAX ULDs in Comp 3'].sum(),
            'Weight ULDs in Comp 3': df['Weight ULDs in Comp 3'].sum(),
            'Weight BAX ULDs in Comp 3': df['Weight BAX ULDs in Comp 3'].sum(),
            'Number of ULDs in Comp 4': df['Number of ULDs in Comp 4'].sum(),
            'Number of BAX ULDs in Comp 4': df['Number of BAX ULDs in Comp 4'].sum(),
            'Weight ULDs in Comp 4': df['Weight ULDs in Comp 4'].sum(),
            'Weight BAX ULDs in Comp 4': df['Weight BAX ULDs in Comp 4'].sum()
        }

        average_data = {
            'Flight Number': 'Average',
            'Flight Date': '-',
            'Fuel Deviation Percentage': df['Fuel Deviation Percentage'].mean(),
            'Fuel Deviation': df['Fuel Deviation'].mean(),
            'TOW': df['TOW'].mean(),
            'ULDs Built by Model': df['ULDs Built by Model'].mean(),
            'ULDs Actually Used': df['ULDs Actually Used'].mean(),
            '%MAC ZFW': df['%MAC ZFW'].mean(),
            'Actual %MAC ZFW': df['Actual %MAC ZFW'].mean(),
            'Total Run Time': df['Total Run Time'].mean(),
            'Total 1D BPP WB Time': df['Total 1D BPP WB Time'].mean(),
            'Total 3D BPP Time': df['Total 3D BPP Time'].mean(),
            'Number of BAX ULDs': df['Number of BAX ULDs'].mean(),
            'Number of BUP ULDs': df['Number of BUP ULDs'].mean(),
            'Number of T ULDs': df['Number of T ULDs'].mean(),
            'Number of PAX A': df['Number of PAX A'].mean(),
            'Number of PAX B': df['Number of PAX B'].mean(),
            'Number of PAX C': df['Number of PAX C'].mean(),
            'Number of PAX D': df['Number of PAX D'].mean(),
            'Number of PAX E': df['Number of PAX E'].mean(),
            'Number of PAX F': df['Number of PAX F'].mean(),
            'Number of PAX G': df['Number of PAX G'].mean(),
            'Total PAX': df['Total PAX'].mean(),
            'Number of items': df['Number of items'].mean(),
            'Weight in Comp 1': df['Weight in Comp 1'].mean(),
            'Weight in Comp 2': df['Weight in Comp 2'].mean(),
            'Weight in Comp 3': df['Weight in Comp 3'].mean(),
            'Weight in Comp 4': df['Weight in Comp 4'].mean(),
            'Number of ULDs in Comp 1': df['Number of ULDs in Comp 1'].mean(),
            'Number of BAX ULDs in Comp 1': df['Number of BAX ULDs in Comp 1'].mean(),
            'Weight ULDs in Comp 1': df['Weight ULDs in Comp 1'].mean(),
            'Weight BAX ULDs in Comp 1': df['Weight BAX ULDs in Comp 1'].mean(),
            'Number of ULDs in Comp 2': df['Number of ULDs in Comp 2'].mean(),
            'Number of BAX ULDs in Comp 2': df['Number of BAX ULDs in Comp 2'].mean(),
            'Weight ULDs in Comp 2': df['Weight ULDs in Comp 2'].mean(),
            'Weight BAX ULDs in Comp 2': df['Weight BAX ULDs in Comp 2'].mean(),
            'Number of ULDs in Comp 3': df['Number of ULDs in Comp 3'].mean(),
            'Number of BAX ULDs in Comp 3': df['Number of BAX ULDs in Comp 3'].mean(),
            'Weight ULDs in Comp 3': df['Weight ULDs in Comp 3'].mean(),
            'Weight BAX ULDs in Comp 3': df['Weight BAX ULDs in Comp 3'].mean(),
            'Number of ULDs in Comp 4': df['Number of ULDs in Comp 4'].mean(),
            'Number of BAX ULDs in Comp 4': df['Number of BAX ULDs in Comp 4'].mean(),
            'Weight ULDs in Comp 4': df['Weight ULDs in Comp 4'].mean(),
            'Weight BAX ULDs in Comp 4': df['Weight BAX ULDs in Comp 4'].mean()
        }

        sum_df = pd.DataFrame([sum_data])
        average_df = pd.DataFrame([average_data])

        df = pd.concat([df, sum_df, average_df], ignore_index=True)

        col_order = ['Flight Number', 'Flight Date', 'Aircraft Type', 'Aircraft Registration', '%MAC ZFW', 'Actual %MAC ZFW', 'Fuel Deviation Percentage',
        'Fuel Deviation', 'TOW', 'ULDs Built by Model', 'ULDs Actually Used', 'Total Run Time', 'Total 1D BPP WB Time', 'Total 3D BPP Time',
        'Number of BAX ULDs', 'Number of BUP ULDs','Number of T ULDs', 'Number of PAX A', 'Number of PAX B', 'Number of PAX C',
        'Number of PAX D', 'Number of PAX E', 'Number of PAX F', 'Number of PAX G', 'Total PAX', 'Number of items',
        'Weight in Comp 1', 'Weight in Comp 2', 'Weight in Comp 3', 'Weight in Comp 4', 'Number of ULDs in Comp 1',
        'Number of BAX ULDs in Comp 1',	'Weight ULDs in Comp 1', 'Weight BAX ULDs in Comp 1', 'Number of ULDs in Comp 2',
        'Number of BAX ULDs in Comp 2',	'Weight ULDs in Comp 2', 'Weight BAX ULDs in Comp 2', 'Number of ULDs in Comp 3',
        'Number of BAX ULDs in Comp 3', 'Weight ULDs in Comp 3', 'Weight BAX ULDs in Comp 3', 'Number of ULDs in Comp 4',	
        'Number of BAX ULDs in Comp 4', 'Weight ULDs in Comp 4', 'Weight BAX ULDs in Comp 4'
        ]

        df = df[col_order]

        return df
    
    def style_results(self, df):
        """
        Style the results DataFrame

        Args:
            df (pd.DataFrame): The DataFrame to style.
        """
        
        last_two_indices = df.tail(2).index

        df.replace('-', np.nan, inplace=True)

        styled_df = (df.style
                    .apply(lambda x: ['font-weight: bold;' if x.name in last_two_indices else '' for _ in x], axis=1)
                    .format({
                        'Fuel Deviation Percentage': '{:.3%}',
                        'Fuel Deviation': '{:,.3f} kg',
                        'TOW': '{:,.1f} kg',
                        'Total Run Time': '{:.2f} s',
                        'Total 1D BPP WB Time': '{:.2f} s',
                        'Total 3D BPP Time': '{:.2f} s',
                        '%MAC ZFW': '{:.4f}',
                        'Actual %MAC ZFW': '{:.4f}',
                        'ULDs Built by Model': '{:,.0f}',
                        'ULDs Actually Used': '{:,.0f}',
                        'Number of BAX ULDs': '{:,.0f}',
                        'Number of BUP ULDs': '{:,.0f}',
                        'Number of T ULDs': '{:,.0f}',
                        'Number of PAX A': '{:,.0f}',
                        'Number of PAX B': '{:,.0f}',
                        'Number of PAX C': '{:,.0f}',
                        'Number of PAX D': '{:,.0f}',
                        'Number of PAX E': '{:,.0f}',
                        'Number of PAX F': '{:,.0f}',
                        'Number of PAX G': '{:,.0f}',
                        'Total PAX': '{:,.0f}',
                        'Number of items': '{:,.0f}',
                        'Weight in Comp 1': '{:,.1f} kg',
                        'Weight in Comp 2': '{:,.1f} kg',
                        'Weight in Comp 3': '{:,.1f} kg',
                        'Weight in Comp 4': '{:,.1f} kg',
                        'Number of ULDs in Comp 1': '{:,.0f}',
                        'Number of BAX ULDs in Comp 1': '{:,.0f}',
                        'Weight ULDs in Comp 1': '{:,.1f} kg',
                        'Weight BAX ULDs in Comp 1': '{:,.1f} kg',
                        'Number of ULDs in Comp 2': '{:,.0f}',
                        'Number of BAX ULDs in Comp 2': '{:,.0f}',
                        'Weight ULDs in Comp 2': '{:,.1f} kg',
                        'Weight BAX ULDs in Comp 2': '{:,.1f} kg',
                        'Number of ULDs in Comp 3': '{:,.0f}',
                        'Number of BAX ULDs in Comp 3': '{:,.0f}',
                        'Weight ULDs in Comp 3': '{:,.1f} kg',
                        'Weight BAX ULDs in Comp 3': '{:,.1f} kg',
                        'Number of ULDs in Comp 4': '{:,.0f}',
                        'Number of BAX ULDs in Comp 4': '{:,.0f}',
                        'Weight ULDs in Comp 4': '{:,.1f} kg',
                        'Weight BAX ULDs in Comp 4': '{:,.1f} kg'
                    }, na_rep='')
                    .hide(axis='index')
                    # Apply bar only to numeric columns that do not have 'Total' or 'Average' in the index
                    .set_table_styles({
                        'A': [{'selector': 'th', 'props': [('text-align', 'left')]}],
                        'B': [{'selector': 'td', 'props': [('text-align', 'center')]}]
                    }, overwrite=False)
                    .set_properties(subset=['Flight Number'], **{'text-align': 'left'})
                    .set_table_attributes('style="font-size: 13px; border: 1px solid black;"')
                    .set_caption(f"Results"))
        
            
        return styled_df
    

    def read_individual_flight_results(self, flight_folder):
        """
        Read the results for an individual flight

        Args:
            flight_folder (str): The folder path to the flight results.
        """

        model_info_file = os.path.join(flight_folder, 'Model_Information.txt')
        if not os.path.exists(model_info_file):
            return None

        flight_number = os.path.basename(flight_folder).split(' ')[1]
        flight_date = os.path.basename(flight_folder).split(' ')[3:6]
        flight_date = ' '.join(flight_date)


        results = {
            'Flight Number': flight_number,
            'Flight Date': flight_date,
        }

        results_file = os.path.join(flight_folder, 'Results.txt')
        model_info_file = os.path.join(flight_folder, 'Model_Information.txt')
        general_info_file = os.path.join(flight_folder, 'General_Information.txt')
        CG_info_file = os.path.join(flight_folder, 'CG_envelope.png')

        with open(general_info_file, 'r') as file:
            content = file.read()
            aircraft_type_match = re.search(r'Aircraft Type: ([\d\w]+) ', content)
            if aircraft_type_match:
                self.aircraft_type = aircraft_type_match.group(1)

            aircraft_registration_match = re.search(r'Flight Number: ([\w\d]+) ([\w\d]+)', content)
            if aircraft_registration_match:
                self.aircraft_registration = aircraft_registration_match.group(2)

            TOW_match = re.search(r'TOW: ([\d.]+) kg', content)
            if TOW_match:
                self.TOW = float(TOW_match.group(1))

            results['Number of BAX ULDs'] = int(re.search(r'Number of BAX ULDs: (\d+)', content).group(1))
            results['Number of BUP ULDs'] = int(re.search(r'Number of BUP ULDs: (\d+)', content).group(1))
            results['Number of T ULDs'] = int(re.search(r'Number of T ULDs: (\d+)', content).group(1))
            
            results['Number of PAX A'] = int(re.search(r'PAX-A: (\d+)', content).group(1))
            results['Number of PAX B'] = int(re.search(r'PAX-B: (\d+)', content).group(1))
            results['Number of PAX C'] = int(re.search(r'PAX-C: (\d+)', content).group(1))
            results['Number of PAX D'] = int(re.search(r'PAX-D: (\d+)', content).group(1))
            results['Number of PAX E'] = int(re.search(r'PAX-E: (\d+)', content).group(1))
            results['Number of PAX F'] = int(re.search(r'PAX-F: (\d+)', content).group(1))
            results['Number of PAX G'] = int(re.search(r'PAX-G: (\d+)', content).group(1))
            total_pax = int(re.search(r'Total PAX: (\d+)', content).group(1))
            results['Total PAX'] = total_pax

            items_fit = int(re.search(r'(\d+) / \d+ items fit the dimensions of the ULDs', content).group(1))
            results['Number of items'] = items_fit

        # Read and process the Results.txt file
        with open(results_file, 'r') as file:
            content = file.read()

            fuel_data = re.search(r"Resulting in a fuel deviation of ([\-\d.]+)% or ([\-\d.]+) kg", content)
            if fuel_data:
                fuel_percentage = float(fuel_data.group(1)) / 100
                fuel_kg = float(fuel_data.group(2))

                self.fuel_reduction_percentage = fuel_percentage
                self.fuel_reduction_kg = fuel_kg

            uld_data = re.search(r"(\d+) ULDs are built by the model\n(\d+) ULDs were actually built", content)
            if uld_data:
                self.uld_built_by_model = int(uld_data.group(1))
                self.uld_actually_used = int(uld_data.group(2))

            mac_data = re.search(r"%MAC ZFW is ([\d.]+)", content)
            if mac_data:
                self.MAC = float(mac_data.group(1))

            mac_actual = re.search(r'The actual %MAC ZFW for this flight was ([\d.]+)', content)
            if mac_actual:
                self.MAC_actual = float(mac_actual.group(1))

            weight_uld_comp_1 = 0
            weight_uld_comp_2 = 0
            weight_uld_comp_3 = 0
            weight_uld_comp_4 = 0
            count_uld_comp_1 = 0
            count_uld_comp_2 = 0
            count_uld_comp_3 = 0
            count_uld_comp_4 = 0

            weight_bax_comp_1 = 0
            weight_bax_comp_2 = 0
            weight_bax_comp_3 = 0
            weight_bax_comp_4 = 0
            count_bax_comp_1 = 0
            count_bax_comp_2 = 0
            count_bax_comp_3 = 0
            count_bax_comp_4 = 0

            pattern = r'ULD (\w+-?\d*) with weight ([\d.]+) kg.*position (\d+)'

            for match in re.finditer(pattern, content):
                uld_type = match.group(1)
                weight = float(match.group(2))
                compartment = int(match.group(3)[0])

                if 'BAX' not in str(uld_type):
                    if compartment == 1:
                        count_uld_comp_1 += 1
                        weight_uld_comp_1 += weight
                    elif compartment == 2:
                        count_uld_comp_2 += 1
                        weight_uld_comp_2 += weight
                    elif compartment == 3:
                        count_uld_comp_3 += 1
                        weight_uld_comp_3 += weight
                    elif compartment == 4:
                        count_uld_comp_4 += 1
                        weight_uld_comp_4 += weight

                if 'BAX' in str(uld_type):
                    if compartment == 1:
                        count_bax_comp_1 += 1
                        weight_bax_comp_1 += weight
                    elif compartment == 2:
                        count_bax_comp_2 += 1
                        weight_bax_comp_2 += weight
                    elif compartment == 3:
                        count_bax_comp_3 += 1
                        weight_bax_comp_3 += weight
                    elif compartment == 4:
                        count_bax_comp_4 += 1
                        weight_bax_comp_4 += weight

            results['Aircraft Type'] = self.aircraft_type
            results['Aircraft Registration'] = self.aircraft_registration
            results['%MAC ZFW'] = self.MAC
            results['Actual %MAC ZFW'] = self.MAC_actual
            results['Fuel Deviation Percentage'] = self.fuel_reduction_percentage
            results['Fuel Deviation'] = self.fuel_reduction_kg
            results['TOW'] = self.TOW
            results['ULDs Built by Model'] = self.uld_built_by_model
            results['ULDs Actually Used'] = self.uld_actually_used
            results['Weight in Comp 1'] = float(re.search(r'Weight in Compartment 1: ([\d.]+)', content).group(1))
            results['Weight in Comp 2'] = float(re.search(r'Weight in Compartment 2: ([\d.]+)', content).group(1))
            results['Weight in Comp 3'] = float(re.search(r'Weight in Compartment 3: ([\d.]+)', content).group(1))
            results['Weight in Comp 4'] = float(re.search(r'Weight in Compartment 4: ([\d.]+)', content).group(1))
            results['Number of ULDs in Comp 1'] = count_uld_comp_1
            results['Number of BAX ULDs in Comp 1'] = count_bax_comp_1
            results['Weight ULDs in Comp 1'] = weight_uld_comp_1
            results['Weight BAX ULDs in Comp 1'] = weight_bax_comp_1
            results['Number of ULDs in Comp 2'] = count_uld_comp_2
            results['Number of BAX ULDs in Comp 2'] = count_bax_comp_2
            results['Weight ULDs in Comp 2'] = weight_uld_comp_2
            results['Weight BAX ULDs in Comp 2'] = weight_bax_comp_2
            results['Number of ULDs in Comp 3'] = count_uld_comp_3
            results['Number of BAX ULDs in Comp 3'] = count_bax_comp_3
            results['Weight ULDs in Comp 3'] = weight_uld_comp_3
            results['Weight BAX ULDs in Comp 3'] = weight_bax_comp_3
            results['Number of ULDs in Comp 4'] = count_uld_comp_4
            results['Number of BAX ULDs in Comp 4'] = count_bax_comp_4
            results['Weight ULDs in Comp 4'] = weight_uld_comp_4
            results['Weight BAX ULDs in Comp 4'] = weight_bax_comp_4
        


        with open(model_info_file, 'r') as file:
            content = file.read()

            total_time_data = re.search(r"Total time: ([\d.]+) seconds", content)
            if total_time_data:
                self.total_run_time = float(total_time_data.group(1))

            total_1D_BPP_WB_data = re.search(r"Total time 1D BPP WB: ([\d.]+) seconds", content)
            if total_1D_BPP_WB_data:
                self.total_1D_BPP_WB_time = float(total_1D_BPP_WB_data.group(1))

            total_3D_BPP_data = re.search(r"Total time 3D BPP: ([\d.]+) seconds", content)
            if total_3D_BPP_data:
                self.total_3D_BPP_time = float(total_3D_BPP_data.group(1))

        results['Total Run Time'] = self.total_run_time
        results['Total 1D BPP WB Time'] = self.total_1D_BPP_WB_time
        results['Total 3D BPP Time'] = self.total_3D_BPP_time

        return results

In [14]:
class ResultsReaderBaseline():
    def __init__(self, departure_airport = None, arrival_airport = None, month = None, year = None):
        self.departure_airport = departure_airport
        self.arrival_airport = arrival_airport
        self.month = month
        self.year = year
        self.aircraft_type = None
        self.aircraft_registration = None
        self.fuel_reduction_percentage = 0
        self.fuel_reduction_kg = 0
        self.uld_built_by_model = 0
        self.uld_actually_used = 0
        self.total_run_time = 0
        self.total_1D_BPP_time = 0
        self.total_3D_BPP_time = 0
        self.total_WB_time = 0
        self.MAC = 0
        self.MAC_actual = 0
        self.TOW = 0

    def read_all_results(self):
        """
        Read all the results from the results folder
        """
        results = []

        base_dir = 'Results_Baseline'

        for root, dirs, files in os.walk(base_dir):
            for dir_name in dirs:
                if dir_name.startswith('Results '):  # Check if directory name starts with 'Results'
                    full_path = os.path.join(root, dir_name)
                    pattern = os.path.join(full_path, 'Flight *')
                    flights = glob.glob(pattern)

                    for flight in flights:
                        result = self.read_individual_flight_results(flight)
                        if result:
                            results.append(result)
        
        df = pd.DataFrame(results)

        sum_data = {
            'Flight Number': 'Total',
            'Flight Date': '-',
            'Fuel Deviation Percentage': '-',
            'Fuel Deviation': df['Fuel Deviation'].sum(),
            'TOW': df['TOW'].sum(),
            'ULDs Built by Model': df['ULDs Built by Model'].sum(),
            'ULDs Actually Used': df['ULDs Actually Used'].sum(),
            '%MAC ZFW': '-',
            'Actual %MAC ZFW': '-',
            'Total Run Time': df['Total Run Time'].sum(),
            'Total 1D BPP Time': df['Total 1D BPP Time'].sum(),
            'Total 3D BPP Time': df['Total 3D BPP Time'].sum(),
            'Total WB Time': df['Total WB Time'].sum(),
            'Number of BAX ULDs': df['Number of BAX ULDs'].sum(),
            'Number of BUP ULDs': df['Number of BUP ULDs'].sum(),
            'Number of T ULDs': df['Number of T ULDs'].sum(),
            'Number of PAX A': '-',
            'Number of PAX B': '-',
            'Number of PAX C': '-',
            'Number of PAX D': '-',
            'Number of PAX E': '-',
            'Number of PAX F': '-',
            'Number of PAX G': '-',
            'Total PAX': '-',
            'Number of items': df['Number of items'].sum(),
            'Weight in Comp 1': df['Weight in Comp 1'].sum(),
            'Weight in Comp 2': df['Weight in Comp 2'].sum(),
            'Weight in Comp 3': df['Weight in Comp 3'].sum(),
            'Weight in Comp 4': df['Weight in Comp 4'].sum(),
            'Number of ULDs in Comp 1': df['Number of ULDs in Comp 1'].sum(),
            'Number of BAX ULDs in Comp 1': df['Number of BAX ULDs in Comp 1'].sum(),
            'Weight ULDs in Comp 1': df['Weight ULDs in Comp 1'].sum(),
            'Weight BAX ULDs in Comp 1': df['Weight BAX ULDs in Comp 1'].sum(),
            'Number of ULDs in Comp 2': df['Number of ULDs in Comp 2'].sum(),
            'Number of BAX ULDs in Comp 2': df['Number of BAX ULDs in Comp 2'].sum(),
            'Weight ULDs in Comp 2': df['Weight ULDs in Comp 2'].sum(),
            'Weight BAX ULDs in Comp 2': df['Weight BAX ULDs in Comp 2'].sum(),
            'Number of ULDs in Comp 3': df['Number of ULDs in Comp 3'].sum(),
            'Number of BAX ULDs in Comp 3': df['Number of BAX ULDs in Comp 3'].sum(),
            'Weight ULDs in Comp 3': df['Weight ULDs in Comp 3'].sum(),
            'Weight BAX ULDs in Comp 3': df['Weight BAX ULDs in Comp 3'].sum(),
            'Number of ULDs in Comp 4': df['Number of ULDs in Comp 4'].sum(),
            'Number of BAX ULDs in Comp 4': df['Number of BAX ULDs in Comp 4'].sum(),
            'Weight ULDs in Comp 4': df['Weight ULDs in Comp 4'].sum(),
            'Weight BAX ULDs in Comp 4': df['Weight BAX ULDs in Comp 4'].sum()
        }

        average_data = {
            'Flight Number': 'Average',
            'Flight Date': '-',
            'Fuel Deviation Percentage': df['Fuel Deviation Percentage'].mean(),
            'Fuel Deviation': df['Fuel Deviation'].mean(),
            'TOW': df['TOW'].mean(),
            'ULDs Built by Model': df['ULDs Built by Model'].mean(),
            'ULDs Actually Used': df['ULDs Actually Used'].mean(),
            '%MAC ZFW': df['%MAC ZFW'].mean(),
            'Actual %MAC ZFW': df['Actual %MAC ZFW'].mean(),
            'Total Run Time': df['Total Run Time'].mean(),
            'Total 1D BPP Time': df['Total 1D BPP Time'].mean(),
            'Total 3D BPP Time': df['Total 3D BPP Time'].mean(),
            'Total WB Time': df['Total WB Time'].mean(),
            'Number of BAX ULDs': df['Number of BAX ULDs'].mean(),
            'Number of BUP ULDs': df['Number of BUP ULDs'].mean(),
            'Number of T ULDs': df['Number of T ULDs'].mean(),
            'Number of PAX A': df['Number of PAX A'].mean(),
            'Number of PAX B': df['Number of PAX B'].mean(),
            'Number of PAX C': df['Number of PAX C'].mean(),
            'Number of PAX D': df['Number of PAX D'].mean(),
            'Number of PAX E': df['Number of PAX E'].mean(),
            'Number of PAX F': df['Number of PAX F'].mean(),
            'Number of PAX G': df['Number of PAX G'].mean(),
            'Total PAX': df['Total PAX'].mean(),
            'Number of items': df['Number of items'].mean(),
            'Weight in Comp 1': df['Weight in Comp 1'].mean(),
            'Weight in Comp 2': df['Weight in Comp 2'].mean(),
            'Weight in Comp 3': df['Weight in Comp 3'].mean(),
            'Weight in Comp 4': df['Weight in Comp 4'].mean(),
            'Number of ULDs in Comp 1': df['Number of ULDs in Comp 1'].mean(),
            'Number of BAX ULDs in Comp 1': df['Number of BAX ULDs in Comp 1'].mean(),
            'Weight ULDs in Comp 1': df['Weight ULDs in Comp 1'].mean(),
            'Weight BAX ULDs in Comp 1': df['Weight BAX ULDs in Comp 1'].mean(),
            'Number of ULDs in Comp 2': df['Number of ULDs in Comp 2'].mean(),
            'Number of BAX ULDs in Comp 2': df['Number of BAX ULDs in Comp 2'].mean(),
            'Weight ULDs in Comp 2': df['Weight ULDs in Comp 2'].mean(),
            'Weight BAX ULDs in Comp 2': df['Weight BAX ULDs in Comp 2'].mean(),
            'Number of ULDs in Comp 3': df['Number of ULDs in Comp 3'].mean(),
            'Number of BAX ULDs in Comp 3': df['Number of BAX ULDs in Comp 3'].mean(),
            'Weight ULDs in Comp 3': df['Weight ULDs in Comp 3'].mean(),
            'Weight BAX ULDs in Comp 3': df['Weight BAX ULDs in Comp 3'].mean(),
            'Number of ULDs in Comp 4': df['Number of ULDs in Comp 4'].mean(),
            'Number of BAX ULDs in Comp 4': df['Number of BAX ULDs in Comp 4'].mean(),
            'Weight ULDs in Comp 4': df['Weight ULDs in Comp 4'].mean(),
            'Weight BAX ULDs in Comp 4': df['Weight BAX ULDs in Comp 4'].mean()
        }

        sum_df = pd.DataFrame([sum_data])
        average_df = pd.DataFrame([average_data])

        df = pd.concat([df, sum_df, average_df], ignore_index=True)

        col_order = ['Flight Number', 'Flight Date', 'Aircraft Type', 'Aircraft Registration', '%MAC ZFW', 'Actual %MAC ZFW', 'Fuel Deviation Percentage',
        'Fuel Deviation', 'TOW', 'ULDs Built by Model', 'ULDs Actually Used', 'Total Run Time', 'Total 1D BPP Time', 'Total 3D BPP Time', 'Total WB Time',
        'Number of BAX ULDs', 'Number of BUP ULDs','Number of T ULDs', 'Number of PAX A', 'Number of PAX B', 'Number of PAX C',
        'Number of PAX D', 'Number of PAX E', 'Number of PAX F', 'Number of PAX G', 'Total PAX', 'Number of items',
        'Weight in Comp 1', 'Weight in Comp 2', 'Weight in Comp 3', 'Weight in Comp 4', 'Number of ULDs in Comp 1',
        'Number of BAX ULDs in Comp 1',	'Weight ULDs in Comp 1', 'Weight BAX ULDs in Comp 1', 'Number of ULDs in Comp 2',
        'Number of BAX ULDs in Comp 2',	'Weight ULDs in Comp 2', 'Weight BAX ULDs in Comp 2', 'Number of ULDs in Comp 3',
        'Number of BAX ULDs in Comp 3', 'Weight ULDs in Comp 3', 'Weight BAX ULDs in Comp 3', 'Number of ULDs in Comp 4',	
        'Number of BAX ULDs in Comp 4', 'Weight ULDs in Comp 4', 'Weight BAX ULDs in Comp 4'
        ]

        df = df[col_order]

        return df


    def read_results(self):
        """
        Read the results from the results folder and display them in a DataFrame
        """

        base_dir = 'Results_Baseline'
        month_folder_name = f'Results {self.departure_airport}{self.arrival_airport} {self.month} {self.year}'
        month_folder = os.path.join(base_dir, month_folder_name)
        pattern = os.path.join(month_folder, 'Flight *')

        flights = glob.glob(pattern)
        results = []


        for flight in flights:
            result = self.read_individual_flight_results(flight)
            if result is not None:
                results.append(result)


        df = pd.DataFrame(results)

    
        sum_data = {
            'Flight Number': 'Total',
            'Flight Date': '-',
            'Fuel Deviation Percentage': '-',
            'Fuel Deviation': df['Fuel Deviation'].sum(),
            'TOW': df['TOW'].sum(),
            'ULDs Built by Model': df['ULDs Built by Model'].sum(),
            'ULDs Actually Used': df['ULDs Actually Used'].sum(),
            '%MAC ZFW': '-',
            'Actual %MAC ZFW': '-',
            'Total Run Time': df['Total Run Time'].sum(),
            'Total 1D BPP Time': df['Total 1D BPP Time'].sum(),
            'Total 3D BPP Time': df['Total 3D BPP Time'].sum(),
            'Total WB Time': df['Total WB Time'].sum(),
            'Number of BAX ULDs': df['Number of BAX ULDs'].sum(),
            'Number of BUP ULDs': df['Number of BUP ULDs'].sum(),
            'Number of T ULDs': df['Number of T ULDs'].sum(),
            'Number of PAX A': '-',
            'Number of PAX B': '-',
            'Number of PAX C': '-',
            'Number of PAX D': '-',
            'Number of PAX E': '-',
            'Number of PAX F': '-',
            'Number of PAX G': '-',
            'Total PAX': '-',
            'Number of items': df['Number of items'].sum(),
            'Weight in Comp 1': df['Weight in Comp 1'].sum(),
            'Weight in Comp 2': df['Weight in Comp 2'].sum(),
            'Weight in Comp 3': df['Weight in Comp 3'].sum(),
            'Weight in Comp 4': df['Weight in Comp 4'].sum(),
            'Number of ULDs in Comp 1': df['Number of ULDs in Comp 1'].sum(),
            'Number of BAX ULDs in Comp 1': df['Number of BAX ULDs in Comp 1'].sum(),
            'Weight ULDs in Comp 1': df['Weight ULDs in Comp 1'].sum(),
            'Weight BAX ULDs in Comp 1': df['Weight BAX ULDs in Comp 1'].sum(),
            'Number of ULDs in Comp 2': df['Number of ULDs in Comp 2'].sum(),
            'Number of BAX ULDs in Comp 2': df['Number of BAX ULDs in Comp 2'].sum(),
            'Weight ULDs in Comp 2': df['Weight ULDs in Comp 2'].sum(),
            'Weight BAX ULDs in Comp 2': df['Weight BAX ULDs in Comp 2'].sum(),
            'Number of ULDs in Comp 3': df['Number of ULDs in Comp 3'].sum(),
            'Number of BAX ULDs in Comp 3': df['Number of BAX ULDs in Comp 3'].sum(),
            'Weight ULDs in Comp 3': df['Weight ULDs in Comp 3'].sum(),
            'Weight BAX ULDs in Comp 3': df['Weight BAX ULDs in Comp 3'].sum(),
            'Number of ULDs in Comp 4': df['Number of ULDs in Comp 4'].sum(),
            'Number of BAX ULDs in Comp 4': df['Number of BAX ULDs in Comp 4'].sum(),
            'Weight ULDs in Comp 4': df['Weight ULDs in Comp 4'].sum(),
            'Weight BAX ULDs in Comp 4': df['Weight BAX ULDs in Comp 4'].sum()
        }

        average_data = {
            'Flight Number': 'Average',
            'Flight Date': '-',
            'Fuel Deviation Percentage': df['Fuel Deviation Percentage'].mean(),
            'Fuel Deviation': df['Fuel Deviation'].mean(),
            'TOW': df['TOW'].mean(),
            'ULDs Built by Model': df['ULDs Built by Model'].mean(),
            'ULDs Actually Used': df['ULDs Actually Used'].mean(),
            '%MAC ZFW': df['%MAC ZFW'].mean(),
            'Actual %MAC ZFW': df['Actual %MAC ZFW'].mean(),
            'Total Run Time': df['Total Run Time'].mean(),
            'Total 1D BPP Time': df['Total 1D BPP Time'].mean(),
            'Total 3D BPP Time': df['Total 3D BPP Time'].mean(),
            'Total WB Time': df['Total WB Time'].mean(),
            'Number of BAX ULDs': df['Number of BAX ULDs'].mean(),
            'Number of BUP ULDs': df['Number of BUP ULDs'].mean(),
            'Number of T ULDs': df['Number of T ULDs'].mean(),
            'Number of PAX A': df['Number of PAX A'].mean(),
            'Number of PAX B': df['Number of PAX B'].mean(),
            'Number of PAX C': df['Number of PAX C'].mean(),
            'Number of PAX D': df['Number of PAX D'].mean(),
            'Number of PAX E': df['Number of PAX E'].mean(),
            'Number of PAX F': df['Number of PAX F'].mean(),
            'Number of PAX G': df['Number of PAX G'].mean(),
            'Total PAX': df['Total PAX'].mean(),
            'Number of items': df['Number of items'].mean(),
            'Weight in Comp 1': df['Weight in Comp 1'].mean(),
            'Weight in Comp 2': df['Weight in Comp 2'].mean(),
            'Weight in Comp 3': df['Weight in Comp 3'].mean(),
            'Weight in Comp 4': df['Weight in Comp 4'].mean(),
            'Number of ULDs in Comp 1': df['Number of ULDs in Comp 1'].mean(),
            'Number of BAX ULDs in Comp 1': df['Number of BAX ULDs in Comp 1'].mean(),
            'Weight ULDs in Comp 1': df['Weight ULDs in Comp 1'].mean(),
            'Weight BAX ULDs in Comp 1': df['Weight BAX ULDs in Comp 1'].mean(),
            'Number of ULDs in Comp 2': df['Number of ULDs in Comp 2'].mean(),
            'Number of BAX ULDs in Comp 2': df['Number of BAX ULDs in Comp 2'].mean(),
            'Weight ULDs in Comp 2': df['Weight ULDs in Comp 2'].mean(),
            'Weight BAX ULDs in Comp 2': df['Weight BAX ULDs in Comp 2'].mean(),
            'Number of ULDs in Comp 3': df['Number of ULDs in Comp 3'].mean(),
            'Number of BAX ULDs in Comp 3': df['Number of BAX ULDs in Comp 3'].mean(),
            'Weight ULDs in Comp 3': df['Weight ULDs in Comp 3'].mean(),
            'Weight BAX ULDs in Comp 3': df['Weight BAX ULDs in Comp 3'].mean(),
            'Number of ULDs in Comp 4': df['Number of ULDs in Comp 4'].mean(),
            'Number of BAX ULDs in Comp 4': df['Number of BAX ULDs in Comp 4'].mean(),
            'Weight ULDs in Comp 4': df['Weight ULDs in Comp 4'].mean(),
            'Weight BAX ULDs in Comp 4': df['Weight BAX ULDs in Comp 4'].mean()
        }

        sum_df = pd.DataFrame([sum_data])
        average_df = pd.DataFrame([average_data])

        df = pd.concat([df, sum_df, average_df], ignore_index=True)

        col_order = ['Flight Number', 'Flight Date', 'Aircraft Type', 'Aircraft Registration', '%MAC ZFW', 'Actual %MAC ZFW', 'Fuel Deviation Percentage',
        'Fuel Deviation', 'TOW', 'ULDs Built by Model', 'ULDs Actually Used', 'Total Run Time', 'Total 1D BPP Time', 'Total 3D BPP Time', 'Total WB Time',
        'Number of BAX ULDs', 'Number of BUP ULDs','Number of T ULDs', 'Number of PAX A', 'Number of PAX B', 'Number of PAX C',
        'Number of PAX D', 'Number of PAX E', 'Number of PAX F', 'Number of PAX G', 'Total PAX', 'Number of items',
        'Weight in Comp 1', 'Weight in Comp 2', 'Weight in Comp 3', 'Weight in Comp 4', 'Number of ULDs in Comp 1',
        'Number of BAX ULDs in Comp 1',	'Weight ULDs in Comp 1', 'Weight BAX ULDs in Comp 1', 'Number of ULDs in Comp 2',
        'Number of BAX ULDs in Comp 2',	'Weight ULDs in Comp 2', 'Weight BAX ULDs in Comp 2', 'Number of ULDs in Comp 3',
        'Number of BAX ULDs in Comp 3', 'Weight ULDs in Comp 3', 'Weight BAX ULDs in Comp 3', 'Number of ULDs in Comp 4',	
        'Number of BAX ULDs in Comp 4', 'Weight ULDs in Comp 4', 'Weight BAX ULDs in Comp 4'
        ]

        df = df[col_order]

        return df
    
    def style_baseline_results(self, df):
        """  
        Style the results DataFrame

        Args:
            df (pd.DataFrame): The DataFrame to style.
        """
        
        last_two_indices = df.tail(2).index

        df.replace('-', np.nan, inplace=True)

        styled_df = (df.style
                    .apply(lambda x: ['font-weight: bold;' if x.name in last_two_indices else '' for _ in x], axis=1)
                    .format({
                        'Fuel Deviation Percentage': '{:.3%}',
                        'Fuel Deviation': '{:,.3f} kg',
                        'TOW': '{:,.1f} kg',
                        'Total Run Time': '{:.2f} s',
                        'Total 1D BPP Time': '{:.2f} s',
                        'Total 3D BPP Time': '{:.2f} s',
                        'Total WB Time': '{:.2f} s',
                        '%MAC ZFW': '{:.4f}',
                        'Actual %MAC ZFW': '{:.4f}',
                        'ULDs Built by Model': '{:,.0f}',
                        'ULDs Actually Used': '{:,.0f}',
                        'Number of BAX ULDs': '{:,.0f}',
                        'Number of BUP ULDs': '{:,.0f}',
                        'Number of T ULDs': '{:,.0f}',
                        'Number of PAX A': '{:,.0f}',
                        'Number of PAX B': '{:,.0f}',
                        'Number of PAX C': '{:,.0f}',
                        'Number of PAX D': '{:,.0f}',
                        'Number of PAX E': '{:,.0f}',
                        'Number of PAX F': '{:,.0f}',
                        'Number of PAX G': '{:,.0f}',
                        'Total PAX': '{:,.0f}',
                        'Number of items': '{:,.0f}',
                        'Weight in Comp 1': '{:,.1f} kg',
                        'Weight in Comp 2': '{:,.1f} kg',
                        'Weight in Comp 3': '{:,.1f} kg',
                        'Weight in Comp 4': '{:,.1f} kg',
                        'Number of ULDs in Comp 1': '{:,.0f}',
                        'Number of BAX ULDs in Comp 1': '{:,.0f}',
                        'Weight ULDs in Comp 1': '{:,.1f} kg',
                        'Weight BAX ULDs in Comp 1': '{:,.1f} kg',
                        'Number of ULDs in Comp 2': '{:,.0f}',
                        'Number of BAX ULDs in Comp 2': '{:,.0f}',
                        'Weight ULDs in Comp 2': '{:,.1f} kg',
                        'Weight BAX ULDs in Comp 2': '{:,.1f} kg',
                        'Number of ULDs in Comp 3': '{:,.0f}',
                        'Number of BAX ULDs in Comp 3': '{:,.0f}',
                        'Weight ULDs in Comp 3': '{:,.1f} kg',
                        'Weight BAX ULDs in Comp 3': '{:,.1f} kg',
                        'Number of ULDs in Comp 4': '{:,.0f}',
                        'Number of BAX ULDs in Comp 4': '{:,.0f}',
                        'Weight ULDs in Comp 4': '{:,.1f} kg',
                        'Weight BAX ULDs in Comp 4': '{:,.1f} kg'
                    }, na_rep='')
                    .hide(axis='index')
                    # Apply bar only to numeric columns that do not have 'Total' or 'Average' in the index
                    .set_table_styles({
                        'A': [{'selector': 'th', 'props': [('text-align', 'left')]}],
                        'B': [{'selector': 'td', 'props': [('text-align', 'center')]}]
                    }, overwrite=False)
                    .set_properties(subset=['Flight Number'], **{'text-align': 'left'})
                    .set_table_attributes('style="font-size: 13px; border: 1px solid black;"')
                    .set_caption(f"Results for {self.departure_airport} to {self.arrival_airport} in {self.month} {self.year}"))
        
            
        return styled_df
    

    def read_individual_flight_results(self, flight_folder):
        """
        Read the results for an individual flight

        Args:
            flight_folder (str): The folder path to the flight results.
        """
        model_info_file = os.path.join(flight_folder, 'Model_Information.txt')
        if not os.path.exists(model_info_file):
            return None
        
        flight_number = os.path.basename(flight_folder).split(' ')[1]
        flight_date = os.path.basename(flight_folder).split(' ')[3:6]
        flight_date = ' '.join(flight_date)


        results = {
            'Flight Number': flight_number,
            'Flight Date': flight_date,
        }

        results_file = os.path.join(flight_folder, 'Results.txt')
        model_info_file = os.path.join(flight_folder, 'Model_Information.txt')
        general_info_file = os.path.join(flight_folder, 'General_Information.txt')
        CG_info_file = os.path.join(flight_folder, 'CG_info.png')

        with open(general_info_file, 'r') as file:
            content = file.read()
            aircraft_type_match = re.search(r'Aircraft Type: ([\d\w]+) ', content)
            if aircraft_type_match:
                self.aircraft_type = aircraft_type_match.group(1) 

            aircraft_registration_match = re.search(r'Flight Number: ([\w\d]+) ([\w\d]+)', content)
            if aircraft_registration_match:
                self.aircraft_registration = aircraft_registration_match.group(2)

            TOW_match = re.search(r'TOW: ([\d.]+) kg', content)
            if TOW_match:
                self.TOW = float(TOW_match.group(1))

            results['Number of BAX ULDs'] = int(re.search(r'Number of BAX ULDs: (\d+)', content).group(1))
            results['Number of BUP ULDs'] = int(re.search(r'Number of BUP ULDs: (\d+)', content).group(1))
            results['Number of T ULDs'] = int(re.search(r'Number of T ULDs: (\d+)', content).group(1))
            
            results['Number of PAX A'] = int(re.search(r'PAX-A: (\d+)', content).group(1))
            results['Number of PAX B'] = int(re.search(r'PAX-B: (\d+)', content).group(1))
            results['Number of PAX C'] = int(re.search(r'PAX-C: (\d+)', content).group(1))
            results['Number of PAX D'] = int(re.search(r'PAX-D: (\d+)', content).group(1))
            results['Number of PAX E'] = int(re.search(r'PAX-E: (\d+)', content).group(1))
            results['Number of PAX F'] = int(re.search(r'PAX-F: (\d+)', content).group(1))
            results['Number of PAX G'] = int(re.search(r'PAX-G: (\d+)', content).group(1))
            total_pax = int(re.search(r'Total PAX: (\d+)', content).group(1))
            results['Total PAX'] = total_pax

            items_fit = int(re.search(r'(\d+) / \d+ items fit the dimensions of the ULDs', content).group(1))
            results['Number of items'] = items_fit

        # Read and process the Results.txt file
        with open(results_file, 'r') as file:
            content = file.read()

            fuel_data = re.search(r"Resulting in a fuel deviation of ([\-\d.]+)% or ([\-\d.]+) kg", content)
            if fuel_data:
                fuel_percentage = float(fuel_data.group(1)) / 100
                fuel_kg = float(fuel_data.group(2))

                self.fuel_reduction_percentage = fuel_percentage
                self.fuel_reduction_kg = fuel_kg

            uld_data = re.search(r"(\d+) ULDs are built by the model\n(\d+) ULDs were actually built", content)
            if uld_data:
                self.uld_built_by_model = int(uld_data.group(1))
                self.uld_actually_used = int(uld_data.group(2))

            mac_data = re.search(r"%MAC ZFW is ([\d.]+)", content)
            if mac_data:
                self.MAC = float(mac_data.group(1))

            mac_actual = re.search(r'The actual %MAC ZFW for this flight was ([\d.]+)', content)
            if mac_actual:
                self.MAC_actual = float(mac_actual.group(1))

            weight_uld_comp_1 = 0
            weight_uld_comp_2 = 0
            weight_uld_comp_3 = 0
            weight_uld_comp_4 = 0
            count_uld_comp_1 = 0
            count_uld_comp_2 = 0
            count_uld_comp_3 = 0
            count_uld_comp_4 = 0

            weight_bax_comp_1 = 0
            weight_bax_comp_2 = 0
            weight_bax_comp_3 = 0
            weight_bax_comp_4 = 0
            count_bax_comp_1 = 0
            count_bax_comp_2 = 0
            count_bax_comp_3 = 0
            count_bax_comp_4 = 0

            pattern = r'ULD (\w+-?\d*) with weight ([\d.]+) kg.*position (\d+)'

            for match in re.finditer(pattern, content):
                uld_type = match.group(1)
                weight = float(match.group(2))
                compartment = int(match.group(3)[0])

                if 'BAX' not in str(uld_type):
                    if compartment == 1:
                        count_uld_comp_1 += 1
                        weight_uld_comp_1 += weight
                    elif compartment == 2:
                        count_uld_comp_2 += 1
                        weight_uld_comp_2 += weight
                    elif compartment == 3:
                        count_uld_comp_3 += 1
                        weight_uld_comp_3 += weight
                    elif compartment == 4:
                        count_uld_comp_4 += 1
                        weight_uld_comp_4 += weight

                if 'BAX' in str(uld_type):
                    if compartment == 1:
                        count_bax_comp_1 += 1
                        weight_bax_comp_1 += weight
                    elif compartment == 2:
                        count_bax_comp_2 += 1
                        weight_bax_comp_2 += weight
                    elif compartment == 3:
                        count_bax_comp_3 += 1
                        weight_bax_comp_3 += weight
                    elif compartment == 4:
                        count_bax_comp_4 += 1
                        weight_bax_comp_4 += weight

        results['Aircraft Type'] = self.aircraft_type
        results['Aircraft Registration'] = self.aircraft_registration
        results['%MAC ZFW'] = self.MAC
        results['Actual %MAC ZFW'] = self.MAC_actual
        results['Fuel Deviation Percentage'] = self.fuel_reduction_percentage
        results['Fuel Deviation'] = self.fuel_reduction_kg
        results['TOW'] = self.TOW
        results['ULDs Built by Model'] = self.uld_built_by_model
        results['ULDs Actually Used'] = self.uld_actually_used
        results['Weight in Comp 1'] = float(re.search(r'Weight in Compartment 1: ([\d.]+)', content).group(1))
        results['Weight in Comp 2'] = float(re.search(r'Weight in Compartment 2: ([\d.]+)', content).group(1))
        results['Weight in Comp 3'] = float(re.search(r'Weight in Compartment 3: ([\d.]+)', content).group(1))
        results['Weight in Comp 4'] = float(re.search(r'Weight in Compartment 4: ([\d.]+)', content).group(1))
        results['Number of ULDs in Comp 1'] = count_uld_comp_1
        results['Number of BAX ULDs in Comp 1'] = count_bax_comp_1
        results['Weight ULDs in Comp 1'] = weight_uld_comp_1
        results['Weight BAX ULDs in Comp 1'] = weight_bax_comp_1
        results['Number of ULDs in Comp 2'] = count_uld_comp_2
        results['Number of BAX ULDs in Comp 2'] = count_bax_comp_2
        results['Weight ULDs in Comp 2'] = weight_uld_comp_2
        results['Weight BAX ULDs in Comp 2'] = weight_bax_comp_2
        results['Number of ULDs in Comp 3'] = count_uld_comp_3
        results['Number of BAX ULDs in Comp 3'] = count_bax_comp_3
        results['Weight ULDs in Comp 3'] = weight_uld_comp_3
        results['Weight BAX ULDs in Comp 3'] = weight_bax_comp_3
        results['Number of ULDs in Comp 4'] = count_uld_comp_4
        results['Number of BAX ULDs in Comp 4'] = count_bax_comp_4
        results['Weight ULDs in Comp 4'] = weight_uld_comp_4
        results['Weight BAX ULDs in Comp 4'] = weight_bax_comp_4

        with open(model_info_file, 'r') as file:
            content = file.read()

            total_time_data = re.search(r"Total time: ([\d.]+) seconds", content)
            if total_time_data:
                self.total_run_time = float(total_time_data.group(1))

            total_1D_BPP_data = re.search(r"Total time 1D BPP: ([\d.]+) seconds", content)
            if total_1D_BPP_data:
                self.total_1D_BPP_time = float(total_1D_BPP_data.group(1))

            total_3D_BPP_data = re.search(r"Total time 3D BPP: ([\d.]+) seconds", content)
            if total_3D_BPP_data:
                self.total_3D_BPP_time = float(total_3D_BPP_data.group(1))

            total_WB_data = re.search(r"Total time WB: ([\d.]+) seconds", content)
            if total_WB_data:
                self.total_WB_time = float(total_WB_data.group(1))

        results['Total Run Time'] = self.total_run_time
        results['Total 1D BPP Time'] = self.total_1D_BPP_time
        results['Total 3D BPP Time'] = self.total_3D_BPP_time
        results['Total WB Time'] = self.total_WB_time

        return results

In [15]:
class ResultsReaderOptimizedActual():
    def __init__(self, departure_airport = None, arrival_airport = None, month = None, year = None):
        self.departure_airport = departure_airport
        self.arrival_airport = arrival_airport
        self.month = month
        self.year = year
        self.aircraft_type = None
        self.aircraft_registration = None
        self.fuel_reduction_percentage = 0
        self.fuel_reduction_kg = 0
        self.uld_built_by_model = 0
        self.uld_actually_used = 0
        self.total_run_time = 0
        self.total_1D_BPP_time = 0
        self.total_3D_BPP_time = 0
        self.total_WB_time = 0
        self.MAC = 0
        self.MAC_actual = 0
        self.TOW = 0


    def read_all_results(self):
        """
        Read all the results from the results folder
        """
        results = []

        base_dir = 'Results_Optimized_Actual'

        for root, dirs, files in os.walk(base_dir):
            for dir_name in dirs:
                if dir_name.startswith('Results '):  # Check if directory name starts with 'Results'
                    full_path = os.path.join(root, dir_name)
                    pattern = os.path.join(full_path, 'Flight *')
                    flights = glob.glob(pattern)

                    for flight in flights:
                        result = self.read_individual_flight_results(flight)
                        if result:
                            results.append(result)
        
        df = pd.DataFrame(results)

        sum_data = {
            'Flight Number': 'Total',
            'Flight Date': '-',
            'Fuel Deviation Percentage': '-',
            'Fuel Deviation': df['Fuel Deviation'].sum(),
            'TOW': df['TOW'].sum(),
            'ULDs Built by Model': df['ULDs Built by Model'].sum(),
            'ULDs Actually Used': df['ULDs Actually Used'].sum(),
            '%MAC ZFW': '-',
            'Actual %MAC ZFW': '-',
            'Total Run Time': df['Total Run Time'].sum(),
            'Number of BAX ULDs': df['Number of BAX ULDs'].sum(),
            'Number of BUP ULDs': df['Number of BUP ULDs'].sum(),
            'Number of T ULDs': df['Number of T ULDs'].sum(),
            'Number of PAX A': '-',
            'Number of PAX B': '-',
            'Number of PAX C': '-',
            'Number of PAX D': '-',
            'Number of PAX E': '-',
            'Number of PAX F': '-',
            'Number of PAX G': '-',
            'Total PAX': '-',
            'Number of items': df['Number of items'].sum(),
            'Weight in Comp 1': df['Weight in Comp 1'].sum(),
            'Weight in Comp 2': df['Weight in Comp 2'].sum(),
            'Weight in Comp 3': df['Weight in Comp 3'].sum(),
            'Weight in Comp 4': df['Weight in Comp 4'].sum(),
            'Number of ULDs in Comp 1': df['Number of ULDs in Comp 1'].sum(),
            'Number of BAX ULDs in Comp 1': df['Number of BAX ULDs in Comp 1'].sum(),
            'Weight ULDs in Comp 1': df['Weight ULDs in Comp 1'].sum(),
            'Weight BAX ULDs in Comp 1': df['Weight BAX ULDs in Comp 1'].sum(),
            'Number of ULDs in Comp 2': df['Number of ULDs in Comp 2'].sum(),
            'Number of BAX ULDs in Comp 2': df['Number of BAX ULDs in Comp 2'].sum(),
            'Weight ULDs in Comp 2': df['Weight ULDs in Comp 2'].sum(),
            'Weight BAX ULDs in Comp 2': df['Weight BAX ULDs in Comp 2'].sum(),
            'Number of ULDs in Comp 3': df['Number of ULDs in Comp 3'].sum(),
            'Number of BAX ULDs in Comp 3': df['Number of BAX ULDs in Comp 3'].sum(),
            'Weight ULDs in Comp 3': df['Weight ULDs in Comp 3'].sum(),
            'Weight BAX ULDs in Comp 3': df['Weight BAX ULDs in Comp 3'].sum(),
            'Number of ULDs in Comp 4': df['Number of ULDs in Comp 4'].sum(),
            'Number of BAX ULDs in Comp 4': df['Number of BAX ULDs in Comp 4'].sum(),
            'Weight ULDs in Comp 4': df['Weight ULDs in Comp 4'].sum(),
            'Weight BAX ULDs in Comp 4': df['Weight BAX ULDs in Comp 4'].sum()
        }

        average_data = {
            'Flight Number': 'Average',
            'Flight Date': '-',
            'Fuel Deviation Percentage': df['Fuel Deviation Percentage'].mean(),
            'Fuel Deviation': df['Fuel Deviation'].mean(),
            'TOW': df['TOW'].mean(),
            'ULDs Built by Model': df['ULDs Built by Model'].mean(),
            'ULDs Actually Used': df['ULDs Actually Used'].mean(),
            '%MAC ZFW': df['%MAC ZFW'].mean(),
            'Actual %MAC ZFW': df['Actual %MAC ZFW'].mean(),
            'Total Run Time': df['Total Run Time'].mean(),
            'Number of BAX ULDs': df['Number of BAX ULDs'].mean(),
            'Number of BUP ULDs': df['Number of BUP ULDs'].mean(),
            'Number of T ULDs': df['Number of T ULDs'].mean(),
            'Number of PAX A': df['Number of PAX A'].mean(),
            'Number of PAX B': df['Number of PAX B'].mean(),
            'Number of PAX C': df['Number of PAX C'].mean(),
            'Number of PAX D': df['Number of PAX D'].mean(),
            'Number of PAX E': df['Number of PAX E'].mean(),
            'Number of PAX F': df['Number of PAX F'].mean(),
            'Number of PAX G': df['Number of PAX G'].mean(),
            'Total PAX': df['Total PAX'].mean(),
            'Number of items': df['Number of items'].mean(),
            'Weight in Comp 1': df['Weight in Comp 1'].mean(),
            'Weight in Comp 2': df['Weight in Comp 2'].mean(),
            'Weight in Comp 3': df['Weight in Comp 3'].mean(),
            'Weight in Comp 4': df['Weight in Comp 4'].mean(),
            'Number of ULDs in Comp 1': df['Number of ULDs in Comp 1'].mean(),
            'Number of BAX ULDs in Comp 1': df['Number of BAX ULDs in Comp 1'].mean(),
            'Weight ULDs in Comp 1': df['Weight ULDs in Comp 1'].mean(),
            'Weight BAX ULDs in Comp 1': df['Weight BAX ULDs in Comp 1'].mean(),
            'Number of ULDs in Comp 2': df['Number of ULDs in Comp 2'].mean(),
            'Number of BAX ULDs in Comp 2': df['Number of BAX ULDs in Comp 2'].mean(),
            'Weight ULDs in Comp 2': df['Weight ULDs in Comp 2'].mean(),
            'Weight BAX ULDs in Comp 2': df['Weight BAX ULDs in Comp 2'].mean(),
            'Number of ULDs in Comp 3': df['Number of ULDs in Comp 3'].mean(),
            'Number of BAX ULDs in Comp 3': df['Number of BAX ULDs in Comp 3'].mean(),
            'Weight ULDs in Comp 3': df['Weight ULDs in Comp 3'].mean(),
            'Weight BAX ULDs in Comp 3': df['Weight BAX ULDs in Comp 3'].mean(),
            'Number of ULDs in Comp 4': df['Number of ULDs in Comp 4'].mean(),
            'Number of BAX ULDs in Comp 4': df['Number of BAX ULDs in Comp 4'].mean(),
            'Weight ULDs in Comp 4': df['Weight ULDs in Comp 4'].mean(),
            'Weight BAX ULDs in Comp 4': df['Weight BAX ULDs in Comp 4'].mean()
        }

        sum_df = pd.DataFrame([sum_data])
        average_df = pd.DataFrame([average_data])

        df = pd.concat([df, sum_df, average_df], ignore_index=True)

        col_order = ['Flight Number', 'Flight Date', 'Aircraft Type', 'Aircraft Registration', '%MAC ZFW', 'Actual %MAC ZFW', 'Fuel Deviation Percentage',
        'Fuel Deviation', 'TOW', 'ULDs Built by Model', 'ULDs Actually Used', 'Total Run Time',
        'Number of BAX ULDs', 'Number of BUP ULDs','Number of T ULDs', 'Number of PAX A', 'Number of PAX B', 'Number of PAX C',
        'Number of PAX D', 'Number of PAX E', 'Number of PAX F', 'Number of PAX G', 'Total PAX', 'Number of items',
        'Weight in Comp 1', 'Weight in Comp 2', 'Weight in Comp 3', 'Weight in Comp 4', 'Number of ULDs in Comp 1',
        'Number of BAX ULDs in Comp 1',	'Weight ULDs in Comp 1', 'Weight BAX ULDs in Comp 1', 'Number of ULDs in Comp 2',
        'Number of BAX ULDs in Comp 2',	'Weight ULDs in Comp 2', 'Weight BAX ULDs in Comp 2', 'Number of ULDs in Comp 3',
        'Number of BAX ULDs in Comp 3', 'Weight ULDs in Comp 3', 'Weight BAX ULDs in Comp 3', 'Number of ULDs in Comp 4',	
        'Number of BAX ULDs in Comp 4', 'Weight ULDs in Comp 4', 'Weight BAX ULDs in Comp 4'
        ]

        df = df[col_order]

        return df


    def read_results(self):
        """
        Read the results from the results folder and display them in a DataFrame
        """

        base_dir = 'Results_Optimized_Actual'
        month_folder_name = f'Results {self.departure_airport}{self.arrival_airport} {self.month} {self.year}'
        month_folder = os.path.join(base_dir, month_folder_name)
        pattern = os.path.join(month_folder, 'Flight *')

        flights = glob.glob(pattern)
        results = []


        for flight in flights:
            result = self.read_individual_flight_results(flight)
            if result is not None:
                results.append(result)


        df = pd.DataFrame(results)

    
        sum_data = {
            'Flight Number': 'Total',
            'Flight Date': '-',
            'Fuel Deviation Percentage': '-',
            'Fuel Deviation': df['Fuel Deviation'].sum(),
            'TOW': df['TOW'].sum(),
            'ULDs Built by Model': df['ULDs Built by Model'].sum(),
            'ULDs Actually Used': df['ULDs Actually Used'].sum(),
            '%MAC ZFW': '-',
            'Actual %MAC ZFW': '-',
            'Total Run Time': df['Total Run Time'].sum(),
            'Number of BAX ULDs': df['Number of BAX ULDs'].sum(),
            'Number of BUP ULDs': df['Number of BUP ULDs'].sum(),
            'Number of T ULDs': df['Number of T ULDs'].sum(),
            'Number of PAX A': '-',
            'Number of PAX B': '-',
            'Number of PAX C': '-',
            'Number of PAX D': '-',
            'Number of PAX E': '-',
            'Number of PAX F': '-',
            'Number of PAX G': '-',
            'Total PAX': '-',
            'Number of items': df['Number of items'].sum(),
            'Weight in Comp 1': df['Weight in Comp 1'].sum(),
            'Weight in Comp 2': df['Weight in Comp 2'].sum(),
            'Weight in Comp 3': df['Weight in Comp 3'].sum(),
            'Weight in Comp 4': df['Weight in Comp 4'].sum(),
            'Number of ULDs in Comp 1': df['Number of ULDs in Comp 1'].sum(),
            'Number of BAX ULDs in Comp 1': df['Number of BAX ULDs in Comp 1'].sum(),
            'Weight ULDs in Comp 1': df['Weight ULDs in Comp 1'].sum(),
            'Weight BAX ULDs in Comp 1': df['Weight BAX ULDs in Comp 1'].sum(),
            'Number of ULDs in Comp 2': df['Number of ULDs in Comp 2'].sum(),
            'Number of BAX ULDs in Comp 2': df['Number of BAX ULDs in Comp 2'].sum(),
            'Weight ULDs in Comp 2': df['Weight ULDs in Comp 2'].sum(),
            'Weight BAX ULDs in Comp 2': df['Weight BAX ULDs in Comp 2'].sum(),
            'Number of ULDs in Comp 3': df['Number of ULDs in Comp 3'].sum(),
            'Number of BAX ULDs in Comp 3': df['Number of BAX ULDs in Comp 3'].sum(),
            'Weight ULDs in Comp 3': df['Weight ULDs in Comp 3'].sum(),
            'Weight BAX ULDs in Comp 3': df['Weight BAX ULDs in Comp 3'].sum(),
            'Number of ULDs in Comp 4': df['Number of ULDs in Comp 4'].sum(),
            'Number of BAX ULDs in Comp 4': df['Number of BAX ULDs in Comp 4'].sum(),
            'Weight ULDs in Comp 4': df['Weight ULDs in Comp 4'].sum(),
            'Weight BAX ULDs in Comp 4': df['Weight BAX ULDs in Comp 4'].sum()
        }

        average_data = {
            'Flight Number': 'Average',
            'Flight Date': '-',
            'Fuel Deviation Percentage': df['Fuel Deviation Percentage'].mean(),
            'Fuel Deviation': df['Fuel Deviation'].mean(),
            'TOW': df['TOW'].mean(),
            'ULDs Built by Model': df['ULDs Built by Model'].mean(),
            'ULDs Actually Used': df['ULDs Actually Used'].mean(),
            '%MAC ZFW': df['%MAC ZFW'].mean(),
            'Actual %MAC ZFW': df['Actual %MAC ZFW'].mean(),
            'Total Run Time': df['Total Run Time'].mean(),
            'Number of BAX ULDs': df['Number of BAX ULDs'].mean(),
            'Number of BUP ULDs': df['Number of BUP ULDs'].mean(),
            'Number of T ULDs': df['Number of T ULDs'].mean(),
            'Number of PAX A': df['Number of PAX A'].mean(),
            'Number of PAX B': df['Number of PAX B'].mean(),
            'Number of PAX C': df['Number of PAX C'].mean(),
            'Number of PAX D': df['Number of PAX D'].mean(),
            'Number of PAX E': df['Number of PAX E'].mean(),
            'Number of PAX F': df['Number of PAX F'].mean(),
            'Number of PAX G': df['Number of PAX G'].mean(),
            'Total PAX': df['Total PAX'].mean(),
            'Number of items': df['Number of items'].mean(),
            'Weight in Comp 1': df['Weight in Comp 1'].mean(),
            'Weight in Comp 2': df['Weight in Comp 2'].mean(),
            'Weight in Comp 3': df['Weight in Comp 3'].mean(),
            'Weight in Comp 4': df['Weight in Comp 4'].mean(),
            'Number of ULDs in Comp 1': df['Number of ULDs in Comp 1'].mean(),
            'Number of BAX ULDs in Comp 1': df['Number of BAX ULDs in Comp 1'].mean(),
            'Weight ULDs in Comp 1': df['Weight ULDs in Comp 1'].mean(),
            'Weight BAX ULDs in Comp 1': df['Weight BAX ULDs in Comp 1'].mean(),
            'Number of ULDs in Comp 2': df['Number of ULDs in Comp 2'].mean(),
            'Number of BAX ULDs in Comp 2': df['Number of BAX ULDs in Comp 2'].mean(),
            'Weight ULDs in Comp 2': df['Weight ULDs in Comp 2'].mean(),
            'Weight BAX ULDs in Comp 2': df['Weight BAX ULDs in Comp 2'].mean(),
            'Number of ULDs in Comp 3': df['Number of ULDs in Comp 3'].mean(),
            'Number of BAX ULDs in Comp 3': df['Number of BAX ULDs in Comp 3'].mean(),
            'Weight ULDs in Comp 3': df['Weight ULDs in Comp 3'].mean(),
            'Weight BAX ULDs in Comp 3': df['Weight BAX ULDs in Comp 3'].mean(),
            'Number of ULDs in Comp 4': df['Number of ULDs in Comp 4'].mean(),
            'Number of BAX ULDs in Comp 4': df['Number of BAX ULDs in Comp 4'].mean(),
            'Weight ULDs in Comp 4': df['Weight ULDs in Comp 4'].mean(),
            'Weight BAX ULDs in Comp 4': df['Weight BAX ULDs in Comp 4'].mean()
            
        }

        sum_df = pd.DataFrame([sum_data])
        average_df = pd.DataFrame([average_data])

        df = pd.concat([df, sum_df, average_df], ignore_index=True)

        col_order = ['Flight Number', 'Flight Date', 'Aircraft Type', 'Aircraft Registration', '%MAC ZFW', 'Actual %MAC ZFW', 'Fuel Deviation Percentage',
        'Fuel Deviation', 'TOW', 'ULDs Built by Model', 'ULDs Actually Used', 'Total Run Time',
        'Number of BAX ULDs', 'Number of BUP ULDs','Number of T ULDs', 'Number of PAX A', 'Number of PAX B', 'Number of PAX C',
        'Number of PAX D', 'Number of PAX E', 'Number of PAX F', 'Number of PAX G', 'Total PAX', 'Number of items',
        'Weight in Comp 1', 'Weight in Comp 2', 'Weight in Comp 3', 'Weight in Comp 4', 'Number of ULDs in Comp 1',
        'Number of BAX ULDs in Comp 1',	'Weight ULDs in Comp 1', 'Weight BAX ULDs in Comp 1', 'Number of ULDs in Comp 2',
        'Number of BAX ULDs in Comp 2',	'Weight ULDs in Comp 2', 'Weight BAX ULDs in Comp 2', 'Number of ULDs in Comp 3',
        'Number of BAX ULDs in Comp 3', 'Weight ULDs in Comp 3', 'Weight BAX ULDs in Comp 3', 'Number of ULDs in Comp 4',	
        'Number of BAX ULDs in Comp 4', 'Weight ULDs in Comp 4', 'Weight BAX ULDs in Comp 4'
        ]

        df = df[col_order]

        return df
    
    def style_baseline_results(self, df):
        """  
        Style the results DataFrame

        Args:
            df (pd.DataFrame): The DataFrame to style.
        """
        
        last_two_indices = df.tail(2).index

        df.replace('-', np.nan, inplace=True)

        styled_df = (df.style
                    .apply(lambda x: ['font-weight: bold;' if x.name in last_two_indices else '' for _ in x], axis=1)
                    .format({
                        'Fuel Deviation Percentage': '{:.3%}',
                        'Fuel Deviation': '{:,.3f} kg',
                        'TOW': '{:,.1f} kg',
                        'Total Run Time': '{:.2f} s',
                        '%MAC ZFW': '{:.4f}',
                        'Actual %MAC ZFW': '{:.4f}',
                        'ULDs Built by Model': '{:,.0f}',
                        'ULDs Actually Used': '{:,.0f}',
                        'Number of BAX ULDs': '{:,.0f}',
                        'Number of BUP ULDs': '{:,.0f}',
                        'Number of T ULDs': '{:,.0f}',
                        'Number of PAX A': '{:,.0f}',
                        'Number of PAX B': '{:,.0f}',
                        'Number of PAX C': '{:,.0f}',
                        'Number of PAX D': '{:,.0f}',
                        'Number of PAX E': '{:,.0f}',
                        'Number of PAX F': '{:,.0f}',
                        'Number of PAX G': '{:,.0f}',
                        'Total PAX': '{:,.0f}',
                        'Number of items': '{:,.0f}',
                        'Weight in Comp 1': '{:,.1f} kg',
                        'Weight in Comp 2': '{:,.1f} kg',
                        'Weight in Comp 3': '{:,.1f} kg',
                        'Weight in Comp 4': '{:,.1f} kg',
                        'Number of ULDs in Comp 1': '{:,.0f}',
                        'Number of BAX ULDs in Comp 1': '{:,.0f}',
                        'Weight ULDs in Comp 1': '{:,.1f} kg',
                        'Weight BAX ULDs in Comp 1': '{:,.1f} kg',
                        'Number of ULDs in Comp 2': '{:,.0f}',
                        'Number of BAX ULDs in Comp 2': '{:,.0f}',
                        'Weight ULDs in Comp 2': '{:,.1f} kg',
                        'Weight BAX ULDs in Comp 2': '{:,.1f} kg',
                        'Number of ULDs in Comp 3': '{:,.0f}',
                        'Number of BAX ULDs in Comp 3': '{:,.0f}',
                        'Weight ULDs in Comp 3': '{:,.1f} kg',
                        'Weight BAX ULDs in Comp 3': '{:,.1f} kg',
                        'Number of ULDs in Comp 4': '{:,.0f}',
                        'Number of BAX ULDs in Comp 4': '{:,.0f}',
                        'Weight ULDs in Comp 4': '{:,.1f} kg',
                        'Weight BAX ULDs in Comp 4': '{:,.1f} kg',
                    }, na_rep='')
                    .hide(axis='index')
                    # Apply bar only to numeric columns that do not have 'Total' or 'Average' in the index
                    .set_table_styles({
                        'A': [{'selector': 'th', 'props': [('text-align', 'left')]}],
                        'B': [{'selector': 'td', 'props': [('text-align', 'center')]}]
                    }, overwrite=False)
                    .set_properties(subset=['Flight Number'], **{'text-align': 'left'})
                    .set_table_attributes('style="font-size: 13px; border: 1px solid black;"')
                    .set_caption(f"Results for {self.departure_airport} to {self.arrival_airport} in {self.month} {self.year}"))
        
            
        return styled_df
    

    def read_individual_flight_results(self, flight_folder):
        """
        Read the results for an individual flight

        Args:
            flight_folder (str): The folder path to the flight results.
        """
        model_info_file = os.path.join(flight_folder, 'Model_Information.txt')
        if not os.path.exists(model_info_file):
            return None
        
        flight_number = os.path.basename(flight_folder).split(' ')[1]
        flight_date = os.path.basename(flight_folder).split(' ')[3:6]
        flight_date = ' '.join(flight_date)


        results = {
            'Flight Number': flight_number,
            'Flight Date': flight_date,
        }

        results_file = os.path.join(flight_folder, 'Results.txt')
        model_info_file = os.path.join(flight_folder, 'Model_Information.txt')
        general_info_file = os.path.join(flight_folder, 'General_Information.txt')
        CG_info_file = os.path.join(flight_folder, 'CG_envelope.png')

        with open(general_info_file, 'r') as file:
            content = file.read()
            aircraft_type_match = re.search(r'Aircraft Type: ([\d\w]+) ', content)
            if aircraft_type_match:
                self.aircraft_type = aircraft_type_match.group(1)

            aircraft_registration_match = re.search(r'Flight Number: ([\w\d]+) ([\w\d]+)', content)
            if aircraft_registration_match:
                self.aircraft_registration = aircraft_registration_match.group(2)

            TOW_match = re.search(r'TOW: ([\d.]+) kg', content)
            if TOW_match:
                self.TOW = float(TOW_match.group(1))
            
            results['Number of BAX ULDs'] = int(re.search(r'Number of BAX ULDs: (\d+)', content).group(1))
            results['Number of BUP ULDs'] = int(re.search(r'Number of BUP ULDs: (\d+)', content).group(1))
            results['Number of T ULDs'] = int(re.search(r'Number of T ULDs: (\d+)', content).group(1))
            
            results['Number of PAX A'] = int(re.search(r'PAX-A: (\d+)', content).group(1))
            results['Number of PAX B'] = int(re.search(r'PAX-B: (\d+)', content).group(1))
            results['Number of PAX C'] = int(re.search(r'PAX-C: (\d+)', content).group(1))
            results['Number of PAX D'] = int(re.search(r'PAX-D: (\d+)', content).group(1))
            results['Number of PAX E'] = int(re.search(r'PAX-E: (\d+)', content).group(1))
            results['Number of PAX F'] = int(re.search(r'PAX-F: (\d+)', content).group(1))
            results['Number of PAX G'] = int(re.search(r'PAX-G: (\d+)', content).group(1))
            total_pax = int(re.search(r'Total PAX: (\d+)', content).group(1))
            results['Total PAX'] = total_pax

            items_fit = int(re.search(r'(\d+) / \d+ items fit the dimensions of the ULDs', content).group(1))
            results['Number of items'] = items_fit

        # Read and process the Results.txt file
        with open(results_file, 'r') as file:
            content = file.read()

            fuel_data = re.search(r"Resulting in a fuel deviation of ([\-\d.]+)% or ([\-\d.]+) kg", content)
            if fuel_data:
                fuel_percentage = float(fuel_data.group(1)) / 100
                fuel_kg = float(fuel_data.group(2))

                self.fuel_reduction_percentage = fuel_percentage
                self.fuel_reduction_kg = fuel_kg

            uld_data = re.search(r"(\d+) ULDs are built by the model\n(\d+) ULDs were actually built", content)
            if uld_data:
                self.uld_built_by_model = int(uld_data.group(1))
                self.uld_actually_used = int(uld_data.group(2))

            mac_data = re.search(r"%MAC ZFW is ([\d.]+)", content)
            if mac_data:
                self.MAC = float(mac_data.group(1))

            mac_actual = re.search(r'The actual %MAC ZFW for this flight was ([\d.]+)', content)
            if mac_actual:
                self.MAC_actual = float(mac_actual.group(1))

            weight_uld_comp_1 = 0
            weight_uld_comp_2 = 0
            weight_uld_comp_3 = 0
            weight_uld_comp_4 = 0
            count_uld_comp_1 = 0
            count_uld_comp_2 = 0
            count_uld_comp_3 = 0
            count_uld_comp_4 = 0

            weight_bax_comp_1 = 0
            weight_bax_comp_2 = 0
            weight_bax_comp_3 = 0
            weight_bax_comp_4 = 0
            count_bax_comp_1 = 0
            count_bax_comp_2 = 0
            count_bax_comp_3 = 0
            count_bax_comp_4 = 0

            pattern = r'ULD (\w+-?\d*) with weight ([\d.]+) kg.*position (\d+)'

            for match in re.finditer(pattern, content):
                uld_type = match.group(1)
                weight = float(match.group(2))
                compartment = int(match.group(3)[0])

                if 'BAX' not in str(uld_type):
                    if compartment == 1:
                        count_uld_comp_1 += 1
                        weight_uld_comp_1 += weight
                    elif compartment == 2:
                        count_uld_comp_2 += 1
                        weight_uld_comp_2 += weight
                    elif compartment == 3:
                        count_uld_comp_3 += 1
                        weight_uld_comp_3 += weight
                    elif compartment == 4:
                        count_uld_comp_4 += 1
                        weight_uld_comp_4 += weight

                if 'BAX' in str(uld_type):
                    if compartment == 1:
                        count_bax_comp_1 += 1
                        weight_bax_comp_1 += weight
                    elif compartment == 2:
                        count_bax_comp_2 += 1
                        weight_bax_comp_2 += weight
                    elif compartment == 3:
                        count_bax_comp_3 += 1
                        weight_bax_comp_3 += weight
                    elif compartment == 4:
                        count_bax_comp_4 += 1
                        weight_bax_comp_4 += weight

            results['Aircraft Type'] = self.aircraft_type
            results['Aircraft Registration'] = self.aircraft_registration
            results['%MAC ZFW'] = self.MAC
            results['Actual %MAC ZFW'] = self.MAC_actual
            results['Fuel Deviation Percentage'] = self.fuel_reduction_percentage
            results['Fuel Deviation'] = self.fuel_reduction_kg
            results['TOW'] = self.TOW
            results['ULDs Built by Model'] = self.uld_built_by_model
            results['ULDs Actually Used'] = self.uld_actually_used
            results['Weight in Comp 1'] = float(re.search(r'Weight in Compartment 1: ([\d.]+)', content).group(1))
            results['Weight in Comp 2'] = float(re.search(r'Weight in Compartment 2: ([\d.]+)', content).group(1))
            results['Weight in Comp 3'] = float(re.search(r'Weight in Compartment 3: ([\d.]+)', content).group(1))
            results['Weight in Comp 4'] = float(re.search(r'Weight in Compartment 4: ([\d.]+)', content).group(1))
            results['Number of ULDs in Comp 1'] = count_uld_comp_1
            results['Number of BAX ULDs in Comp 1'] = count_bax_comp_1
            results['Weight ULDs in Comp 1'] = weight_uld_comp_1
            results['Weight BAX ULDs in Comp 1'] = weight_bax_comp_1
            results['Number of ULDs in Comp 2'] = count_uld_comp_2
            results['Number of BAX ULDs in Comp 2'] = count_bax_comp_2
            results['Weight ULDs in Comp 2'] = weight_uld_comp_2
            results['Weight BAX ULDs in Comp 2'] = weight_bax_comp_2
            results['Number of ULDs in Comp 3'] = count_uld_comp_3
            results['Number of BAX ULDs in Comp 3'] = count_bax_comp_3
            results['Weight ULDs in Comp 3'] = weight_uld_comp_3
            results['Weight BAX ULDs in Comp 3'] = weight_bax_comp_3
            results['Number of ULDs in Comp 4'] = count_uld_comp_4
            results['Number of BAX ULDs in Comp 4'] = count_bax_comp_4
            results['Weight ULDs in Comp 4'] = weight_uld_comp_4
            results['Weight BAX ULDs in Comp 4'] = weight_bax_comp_4

        with open(model_info_file, 'r') as file:
            content = file.read()

            total_time_data = re.search(r"Total time: ([\d.]+) seconds", content)
            if total_time_data:
                self.total_run_time = float(total_time_data.group(1))


        results['Total Run Time'] = self.total_run_time

        return results

In [16]:
class ResultsReaderBAXFixed():
    def __init__(self, departure_airport = None, arrival_airport = None, month = None, year = None):
        self.departure_airport = departure_airport
        self.arrival_airport = arrival_airport
        self.month = month
        self.year = year
        self.aircraft_type = None
        self.aircraft_registration = None
        self.fuel_reduction_percentage = 0
        self.fuel_reduction_kg = 0
        self.uld_built_by_model = 0
        self.uld_actually_used = 0
        self.total_run_time = 0
        self.total_1D_BPP_time = 0
        self.total_3D_BPP_time = 0
        self.total_WB_time = 0
        self.MAC = 0
        self.MAC_actual = 0
        self.TOW = 0


    def read_all_results(self):
        """
        Read all the results from the results folder
        """
        results = []

        base_dir = 'Results_BAX_Fixed'

        for root, dirs, files in os.walk(base_dir):
            for dir_name in dirs:
                if dir_name.startswith('Results '):  # Check if directory name starts with 'Results'
                    full_path = os.path.join(root, dir_name)
                    pattern = os.path.join(full_path, 'Flight *')
                    flights = glob.glob(pattern)

                    for flight in flights:
                        result = self.read_individual_flight_results(flight)
                        if result:
                            results.append(result)
        
        df = pd.DataFrame(results)

        sum_data = {
            'Flight Number': 'Total',
            'Flight Date': '-',
            'Fuel Deviation Percentage': '-',
            'Fuel Deviation': df['Fuel Deviation'].sum(),
            'TOW': df['TOW'].sum(),
            'ULDs Built by Model': df['ULDs Built by Model'].sum(),
            'ULDs Actually Used': df['ULDs Actually Used'].sum(),
            '%MAC ZFW': '-',
            'Actual %MAC ZFW': '-',
            'Total Run Time': df['Total Run Time'].sum(),
            'Number of BAX ULDs': df['Number of BAX ULDs'].sum(),
            'Number of BUP ULDs': df['Number of BUP ULDs'].sum(),
            'Number of T ULDs': df['Number of T ULDs'].sum(),
            'Number of PAX A': '-',
            'Number of PAX B': '-',
            'Number of PAX C': '-',
            'Number of PAX D': '-',
            'Number of PAX E': '-',
            'Number of PAX F': '-',
            'Number of PAX G': '-',
            'Total PAX': '-',
            'Number of items': df['Number of items'].sum(),
            'Weight in Comp 1': df['Weight in Comp 1'].sum(),
            'Weight in Comp 2': df['Weight in Comp 2'].sum(),
            'Weight in Comp 3': df['Weight in Comp 3'].sum(),
            'Weight in Comp 4': df['Weight in Comp 4'].sum(),
            'Number of ULDs in Comp 1': df['Number of ULDs in Comp 1'].sum(),
            'Number of BAX ULDs in Comp 1': df['Number of BAX ULDs in Comp 1'].sum(),
            'Weight ULDs in Comp 1': df['Weight ULDs in Comp 1'].sum(),
            'Weight BAX ULDs in Comp 1': df['Weight BAX ULDs in Comp 1'].sum(),
            'Number of ULDs in Comp 2': df['Number of ULDs in Comp 2'].sum(),
            'Number of BAX ULDs in Comp 2': df['Number of BAX ULDs in Comp 2'].sum(),
            'Weight ULDs in Comp 2': df['Weight ULDs in Comp 2'].sum(),
            'Weight BAX ULDs in Comp 2': df['Weight BAX ULDs in Comp 2'].sum(),
            'Number of ULDs in Comp 3': df['Number of ULDs in Comp 3'].sum(),
            'Number of BAX ULDs in Comp 3': df['Number of BAX ULDs in Comp 3'].sum(),
            'Weight ULDs in Comp 3': df['Weight ULDs in Comp 3'].sum(),
            'Weight BAX ULDs in Comp 3': df['Weight BAX ULDs in Comp 3'].sum(),
            'Number of ULDs in Comp 4': df['Number of ULDs in Comp 4'].sum(),
            'Number of BAX ULDs in Comp 4': df['Number of BAX ULDs in Comp 4'].sum(),
            'Weight ULDs in Comp 4': df['Weight ULDs in Comp 4'].sum(),
            'Weight BAX ULDs in Comp 4': df['Weight BAX ULDs in Comp 4'].sum()
        }

        average_data = {
            'Flight Number': 'Average',
            'Flight Date': '-',
            'Fuel Deviation Percentage': df['Fuel Deviation Percentage'].mean(),
            'Fuel Deviation': df['Fuel Deviation'].mean(),
            'TOW': df['TOW'].mean(),
            'ULDs Built by Model': df['ULDs Built by Model'].mean(),
            'ULDs Actually Used': df['ULDs Actually Used'].mean(),
            '%MAC ZFW': df['%MAC ZFW'].mean(),
            'Actual %MAC ZFW': df['Actual %MAC ZFW'].mean(),
            'Total Run Time': df['Total Run Time'].mean(),
            'Number of BAX ULDs': df['Number of BAX ULDs'].mean(),
            'Number of BUP ULDs': df['Number of BUP ULDs'].mean(),
            'Number of T ULDs': df['Number of T ULDs'].mean(),
            'Number of PAX A': df['Number of PAX A'].mean(),
            'Number of PAX B': df['Number of PAX B'].mean(),
            'Number of PAX C': df['Number of PAX C'].mean(),
            'Number of PAX D': df['Number of PAX D'].mean(),
            'Number of PAX E': df['Number of PAX E'].mean(),
            'Number of PAX F': df['Number of PAX F'].mean(),
            'Number of PAX G': df['Number of PAX G'].mean(),
            'Total PAX': df['Total PAX'].mean(),
            'Number of items': df['Number of items'].mean(),
            'Weight in Comp 1': df['Weight in Comp 1'].mean(),
            'Weight in Comp 2': df['Weight in Comp 2'].mean(),
            'Weight in Comp 3': df['Weight in Comp 3'].mean(),
            'Weight in Comp 4': df['Weight in Comp 4'].mean(),
            'Number of ULDs in Comp 1': df['Number of ULDs in Comp 1'].mean(),
            'Number of BAX ULDs in Comp 1': df['Number of BAX ULDs in Comp 1'].mean(),
            'Weight ULDs in Comp 1': df['Weight ULDs in Comp 1'].mean(),
            'Weight BAX ULDs in Comp 1': df['Weight BAX ULDs in Comp 1'].mean(),
            'Number of ULDs in Comp 2': df['Number of ULDs in Comp 2'].mean(),
            'Number of BAX ULDs in Comp 2': df['Number of BAX ULDs in Comp 2'].mean(),
            'Weight ULDs in Comp 2': df['Weight ULDs in Comp 2'].mean(),
            'Weight BAX ULDs in Comp 2': df['Weight BAX ULDs in Comp 2'].mean(),
            'Number of ULDs in Comp 3': df['Number of ULDs in Comp 3'].mean(),
            'Number of BAX ULDs in Comp 3': df['Number of BAX ULDs in Comp 3'].mean(),
            'Weight ULDs in Comp 3': df['Weight ULDs in Comp 3'].mean(),
            'Weight BAX ULDs in Comp 3': df['Weight BAX ULDs in Comp 3'].mean(),
            'Number of ULDs in Comp 4': df['Number of ULDs in Comp 4'].mean(),
            'Number of BAX ULDs in Comp 4': df['Number of BAX ULDs in Comp 4'].mean(),
            'Weight ULDs in Comp 4': df['Weight ULDs in Comp 4'].mean(),
            'Weight BAX ULDs in Comp 4': df['Weight BAX ULDs in Comp 4'].mean()
        }

        sum_df = pd.DataFrame([sum_data])
        average_df = pd.DataFrame([average_data])

        df = pd.concat([df, sum_df, average_df], ignore_index=True)

        col_order = ['Flight Number', 'Flight Date', 'Aircraft Type', 'Aircraft Registration', '%MAC ZFW', 'Actual %MAC ZFW', 'Fuel Deviation Percentage',
        'Fuel Deviation', 'TOW', 'ULDs Built by Model', 'ULDs Actually Used', 'Total Run Time',
        'Number of BAX ULDs', 'Number of BUP ULDs','Number of T ULDs', 'Number of PAX A', 'Number of PAX B', 'Number of PAX C',
        'Number of PAX D', 'Number of PAX E', 'Number of PAX F', 'Number of PAX G', 'Total PAX', 'Number of items',
        'Weight in Comp 1', 'Weight in Comp 2', 'Weight in Comp 3', 'Weight in Comp 4', 'Number of ULDs in Comp 1',
        'Number of BAX ULDs in Comp 1',	'Weight ULDs in Comp 1', 'Weight BAX ULDs in Comp 1', 'Number of ULDs in Comp 2',
        'Number of BAX ULDs in Comp 2',	'Weight ULDs in Comp 2', 'Weight BAX ULDs in Comp 2', 'Number of ULDs in Comp 3',
        'Number of BAX ULDs in Comp 3', 'Weight ULDs in Comp 3', 'Weight BAX ULDs in Comp 3', 'Number of ULDs in Comp 4',	
        'Number of BAX ULDs in Comp 4', 'Weight ULDs in Comp 4', 'Weight BAX ULDs in Comp 4'
        ]

        df = df[col_order]

        return df


    def read_results(self):
        """
        Read the results from the results folder and display them in a DataFrame
        """

        base_dir = 'Results_BAX_Fixed'
        month_folder_name = f'Results {self.departure_airport}{self.arrival_airport} {self.month} {self.year}'
        month_folder = os.path.join(base_dir, month_folder_name)
        pattern = os.path.join(month_folder, 'Flight *')

        flights = glob.glob(pattern)
        results = []


        for flight in flights:
            result = self.read_individual_flight_results(flight)
            if result is not None:
                results.append(result)


        df = pd.DataFrame(results)

    
        sum_data = {
            'Flight Number': 'Total',
            'Flight Date': '-',
            'Fuel Deviation Percentage': '-',
            'Fuel Deviation': df['Fuel Deviation'].sum(),
            'TOW': df['TOW'].sum(),
            'ULDs Built by Model': df['ULDs Built by Model'].sum(),
            'ULDs Actually Used': df['ULDs Actually Used'].sum(),
            '%MAC ZFW': '-',
            'Actual %MAC ZFW': '-',
            'Total Run Time': df['Total Run Time'].sum(),
            'Number of BAX ULDs': df['Number of BAX ULDs'].sum(),
            'Number of BUP ULDs': df['Number of BUP ULDs'].sum(),
            'Number of T ULDs': df['Number of T ULDs'].sum(),
            'Number of PAX A': '-',
            'Number of PAX B': '-',
            'Number of PAX C': '-',
            'Number of PAX D': '-',
            'Number of PAX E': '-',
            'Number of PAX F': '-',
            'Number of PAX G': '-',
            'Total PAX': '-',
            'Number of items': df['Number of items'].sum(),
            'Weight in Comp 1': df['Weight in Comp 1'].sum(),
            'Weight in Comp 2': df['Weight in Comp 2'].sum(),
            'Weight in Comp 3': df['Weight in Comp 3'].sum(),
            'Weight in Comp 4': df['Weight in Comp 4'].sum(),
            'Number of ULDs in Comp 1': df['Number of ULDs in Comp 1'].sum(),
            'Number of BAX ULDs in Comp 1': df['Number of BAX ULDs in Comp 1'].sum(),
            'Weight ULDs in Comp 1': df['Weight ULDs in Comp 1'].sum(),
            'Weight BAX ULDs in Comp 1': df['Weight BAX ULDs in Comp 1'].sum(),
            'Number of ULDs in Comp 2': df['Number of ULDs in Comp 2'].sum(),
            'Number of BAX ULDs in Comp 2': df['Number of BAX ULDs in Comp 2'].sum(),
            'Weight ULDs in Comp 2': df['Weight ULDs in Comp 2'].sum(),
            'Weight BAX ULDs in Comp 2': df['Weight BAX ULDs in Comp 2'].sum(),
            'Number of ULDs in Comp 3': df['Number of ULDs in Comp 3'].sum(),
            'Number of BAX ULDs in Comp 3': df['Number of BAX ULDs in Comp 3'].sum(),
            'Weight ULDs in Comp 3': df['Weight ULDs in Comp 3'].sum(),
            'Weight BAX ULDs in Comp 3': df['Weight BAX ULDs in Comp 3'].sum(),
            'Number of ULDs in Comp 4': df['Number of ULDs in Comp 4'].sum(),
            'Number of BAX ULDs in Comp 4': df['Number of BAX ULDs in Comp 4'].sum(),
            'Weight ULDs in Comp 4': df['Weight ULDs in Comp 4'].sum(),
            'Weight BAX ULDs in Comp 4': df['Weight BAX ULDs in Comp 4'].sum()
        }

        average_data = {
            'Flight Number': 'Average',
            'Flight Date': '-',
            'Fuel Deviation Percentage': df['Fuel Deviation Percentage'].mean(),
            'Fuel Deviation': df['Fuel Deviation'].mean(),
            'TOW': df['TOW'].mean(),
            'ULDs Built by Model': df['ULDs Built by Model'].mean(),
            'ULDs Actually Used': df['ULDs Actually Used'].mean(),
            '%MAC ZFW': df['%MAC ZFW'].mean(),
            'Actual %MAC ZFW': df['Actual %MAC ZFW'].mean(),
            'Total Run Time': df['Total Run Time'].mean(),
            'Number of BAX ULDs': df['Number of BAX ULDs'].mean(),
            'Number of BUP ULDs': df['Number of BUP ULDs'].mean(),
            'Number of T ULDs': df['Number of T ULDs'].mean(),
            'Number of PAX A': df['Number of PAX A'].mean(),
            'Number of PAX B': df['Number of PAX B'].mean(),
            'Number of PAX C': df['Number of PAX C'].mean(),
            'Number of PAX D': df['Number of PAX D'].mean(),
            'Number of PAX E': df['Number of PAX E'].mean(),
            'Number of PAX F': df['Number of PAX F'].mean(),
            'Number of PAX G': df['Number of PAX G'].mean(),
            'Total PAX': df['Total PAX'].mean(),
            'Number of items': df['Number of items'].mean(),
            'Weight in Comp 1': df['Weight in Comp 1'].mean(),
            'Weight in Comp 2': df['Weight in Comp 2'].mean(),
            'Weight in Comp 3': df['Weight in Comp 3'].mean(),
            'Weight in Comp 4': df['Weight in Comp 4'].mean(),
            'Number of ULDs in Comp 1': df['Number of ULDs in Comp 1'].mean(),
            'Number of BAX ULDs in Comp 1': df['Number of BAX ULDs in Comp 1'].mean(),
            'Weight ULDs in Comp 1': df['Weight ULDs in Comp 1'].mean(),
            'Weight BAX ULDs in Comp 1': df['Weight BAX ULDs in Comp 1'].mean(),
            'Number of ULDs in Comp 2': df['Number of ULDs in Comp 2'].mean(),
            'Number of BAX ULDs in Comp 2': df['Number of BAX ULDs in Comp 2'].mean(),
            'Weight ULDs in Comp 2': df['Weight ULDs in Comp 2'].mean(),
            'Weight BAX ULDs in Comp 2': df['Weight BAX ULDs in Comp 2'].mean(),
            'Number of ULDs in Comp 3': df['Number of ULDs in Comp 3'].mean(),
            'Number of BAX ULDs in Comp 3': df['Number of BAX ULDs in Comp 3'].mean(),
            'Weight ULDs in Comp 3': df['Weight ULDs in Comp 3'].mean(),
            'Weight BAX ULDs in Comp 3': df['Weight BAX ULDs in Comp 3'].mean(),
            'Number of ULDs in Comp 4': df['Number of ULDs in Comp 4'].mean(),
            'Number of BAX ULDs in Comp 4': df['Number of BAX ULDs in Comp 4'].mean(),
            'Weight ULDs in Comp 4': df['Weight ULDs in Comp 4'].mean(),
            'Weight BAX ULDs in Comp 4': df['Weight BAX ULDs in Comp 4'].mean()
            
        }

        sum_df = pd.DataFrame([sum_data])
        average_df = pd.DataFrame([average_data])

        df = pd.concat([df, sum_df, average_df], ignore_index=True)

        col_order = ['Flight Number', 'Flight Date', 'Aircraft Type', 'Aircraft Registration', '%MAC ZFW', 'Actual %MAC ZFW', 'Fuel Deviation Percentage',
        'Fuel Deviation', 'TOW', 'ULDs Built by Model', 'ULDs Actually Used', 'Total Run Time',
        'Number of BAX ULDs', 'Number of BUP ULDs','Number of T ULDs', 'Number of PAX A', 'Number of PAX B', 'Number of PAX C',
        'Number of PAX D', 'Number of PAX E', 'Number of PAX F', 'Number of PAX G', 'Total PAX', 'Number of items',
        'Weight in Comp 1', 'Weight in Comp 2', 'Weight in Comp 3', 'Weight in Comp 4', 'Number of ULDs in Comp 1',
        'Number of BAX ULDs in Comp 1',	'Weight ULDs in Comp 1', 'Weight BAX ULDs in Comp 1', 'Number of ULDs in Comp 2',
        'Number of BAX ULDs in Comp 2',	'Weight ULDs in Comp 2', 'Weight BAX ULDs in Comp 2', 'Number of ULDs in Comp 3',
        'Number of BAX ULDs in Comp 3', 'Weight ULDs in Comp 3', 'Weight BAX ULDs in Comp 3', 'Number of ULDs in Comp 4',	
        'Number of BAX ULDs in Comp 4', 'Weight ULDs in Comp 4', 'Weight BAX ULDs in Comp 4'
        ]

        df = df[col_order]

        return df
    
    def style_baseline_results(self, df):
        """  
        Style the results DataFrame

        Args:
            df (pd.DataFrame): The DataFrame to style.
        """
        
        last_two_indices = df.tail(2).index

        df.replace('-', np.nan, inplace=True)

        styled_df = (df.style
                    .apply(lambda x: ['font-weight: bold;' if x.name in last_two_indices else '' for _ in x], axis=1)
                    .format({
                        'Fuel Deviation Percentage': '{:.3%}',
                        'Fuel Deviation': '{:,.3f} kg',
                        'TOW': '{:,.1f} kg',
                        'Total Run Time': '{:.2f} s',
                        '%MAC ZFW': '{:.4f}',
                        'Actual %MAC ZFW': '{:.4f}',
                        'ULDs Built by Model': '{:,.0f}',
                        'ULDs Actually Used': '{:,.0f}',
                        'Number of BAX ULDs': '{:,.0f}',
                        'Number of BUP ULDs': '{:,.0f}',
                        'Number of T ULDs': '{:,.0f}',
                        'Number of PAX A': '{:,.0f}',
                        'Number of PAX B': '{:,.0f}',
                        'Number of PAX C': '{:,.0f}',
                        'Number of PAX D': '{:,.0f}',
                        'Number of PAX E': '{:,.0f}',
                        'Number of PAX F': '{:,.0f}',
                        'Number of PAX G': '{:,.0f}',
                        'Total PAX': '{:,.0f}',
                        'Number of items': '{:,.0f}',
                        'Weight in Comp 1': '{:,.1f} kg',
                        'Weight in Comp 2': '{:,.1f} kg',
                        'Weight in Comp 3': '{:,.1f} kg',
                        'Weight in Comp 4': '{:,.1f} kg',
                        'Number of ULDs in Comp 1': '{:,.0f}',
                        'Number of BAX ULDs in Comp 1': '{:,.0f}',
                        'Weight ULDs in Comp 1': '{:,.1f} kg',
                        'Weight BAX ULDs in Comp 1': '{:,.1f} kg',
                        'Number of ULDs in Comp 2': '{:,.0f}',
                        'Number of BAX ULDs in Comp 2': '{:,.0f}',
                        'Weight ULDs in Comp 2': '{:,.1f} kg',
                        'Weight BAX ULDs in Comp 2': '{:,.1f} kg',
                        'Number of ULDs in Comp 3': '{:,.0f}',
                        'Number of BAX ULDs in Comp 3': '{:,.0f}',
                        'Weight ULDs in Comp 3': '{:,.1f} kg',
                        'Weight BAX ULDs in Comp 3': '{:,.1f} kg',
                        'Number of ULDs in Comp 4': '{:,.0f}',
                        'Number of BAX ULDs in Comp 4': '{:,.0f}',
                        'Weight ULDs in Comp 4': '{:,.1f} kg',
                        'Weight BAX ULDs in Comp 4': '{:,.1f} kg',
                    }, na_rep='')
                    .hide(axis='index')
                    # Apply bar only to numeric columns that do not have 'Total' or 'Average' in the index
                    .set_table_styles({
                        'A': [{'selector': 'th', 'props': [('text-align', 'left')]}],
                        'B': [{'selector': 'td', 'props': [('text-align', 'center')]}]
                    }, overwrite=False)
                    .set_properties(subset=['Flight Number'], **{'text-align': 'left'})
                    .set_table_attributes('style="font-size: 13px; border: 1px solid black;"')
                    .set_caption(f"Results for {self.departure_airport} to {self.arrival_airport} in {self.month} {self.year}"))
        
            
        return styled_df
    

    def read_individual_flight_results(self, flight_folder):
        """
        Read the results for an individual flight

        Args:
            flight_folder (str): The folder path to the flight results.
        """
        model_info_file = os.path.join(flight_folder, 'Model_Information.txt')
        if not os.path.exists(model_info_file):
            return None
        
        flight_number = os.path.basename(flight_folder).split(' ')[1]
        flight_date = os.path.basename(flight_folder).split(' ')[3:6]
        flight_date = ' '.join(flight_date)


        results = {
            'Flight Number': flight_number,
            'Flight Date': flight_date,
        }

        results_file = os.path.join(flight_folder, 'Results.txt')
        model_info_file = os.path.join(flight_folder, 'Model_Information.txt')
        general_info_file = os.path.join(flight_folder, 'General_Information.txt')
        CG_info_file = os.path.join(flight_folder, 'CG_envelope.png')

        with open(general_info_file, 'r') as file:
            content = file.read()
            aircraft_type_match = re.search(r'Aircraft Type: ([\d\w]+) ', content)
            if aircraft_type_match:
                self.aircraft_type = aircraft_type_match.group(1)

            aircraft_registration_match = re.search(r'Flight Number: ([\w\d]+) ([\w\d]+)', content)
            if aircraft_registration_match:
                self.aircraft_registration = aircraft_registration_match.group(2)

            TOW_match = re.search(r'TOW: ([\d.]+) kg', content)
            if TOW_match:
                self.TOW = float(TOW_match.group(1))
            
            results['Number of BAX ULDs'] = int(re.search(r'Number of BAX ULDs: (\d+)', content).group(1))
            results['Number of BUP ULDs'] = int(re.search(r'Number of BUP ULDs: (\d+)', content).group(1))
            results['Number of T ULDs'] = int(re.search(r'Number of T ULDs: (\d+)', content).group(1))
            
            results['Number of PAX A'] = int(re.search(r'PAX-A: (\d+)', content).group(1))
            results['Number of PAX B'] = int(re.search(r'PAX-B: (\d+)', content).group(1))
            results['Number of PAX C'] = int(re.search(r'PAX-C: (\d+)', content).group(1))
            results['Number of PAX D'] = int(re.search(r'PAX-D: (\d+)', content).group(1))
            results['Number of PAX E'] = int(re.search(r'PAX-E: (\d+)', content).group(1))
            results['Number of PAX F'] = int(re.search(r'PAX-F: (\d+)', content).group(1))
            results['Number of PAX G'] = int(re.search(r'PAX-G: (\d+)', content).group(1))
            total_pax = int(re.search(r'Total PAX: (\d+)', content).group(1))
            results['Total PAX'] = total_pax

            items_fit = int(re.search(r'(\d+) / \d+ items fit the dimensions of the ULDs', content).group(1))
            results['Number of items'] = items_fit

        # Read and process the Results.txt file
        with open(results_file, 'r') as file:
            content = file.read()

            fuel_data = re.search(r"Resulting in a fuel deviation of ([\-\d.]+)% or ([\-\d.]+) kg", content)
            if fuel_data:
                fuel_percentage = float(fuel_data.group(1)) / 100
                fuel_kg = float(fuel_data.group(2))

                self.fuel_reduction_percentage = fuel_percentage
                self.fuel_reduction_kg = fuel_kg

            uld_data = re.search(r"(\d+) ULDs are built by the model\n(\d+) ULDs were actually built", content)
            if uld_data:
                self.uld_built_by_model = int(uld_data.group(1))
                self.uld_actually_used = int(uld_data.group(2))

            mac_data = re.search(r"%MAC ZFW is ([\d.]+)", content)
            if mac_data:
                self.MAC = float(mac_data.group(1))

            mac_actual = re.search(r'The actual %MAC ZFW for this flight was ([\d.]+)', content)
            if mac_actual:
                self.MAC_actual = float(mac_actual.group(1))

            weight_uld_comp_1 = 0
            weight_uld_comp_2 = 0
            weight_uld_comp_3 = 0
            weight_uld_comp_4 = 0
            count_uld_comp_1 = 0
            count_uld_comp_2 = 0
            count_uld_comp_3 = 0
            count_uld_comp_4 = 0

            weight_bax_comp_1 = 0
            weight_bax_comp_2 = 0
            weight_bax_comp_3 = 0
            weight_bax_comp_4 = 0
            count_bax_comp_1 = 0
            count_bax_comp_2 = 0
            count_bax_comp_3 = 0
            count_bax_comp_4 = 0

            pattern = r'ULD (\w+-?\d*) with weight ([\d.]+) kg.*position (\d+)'

            for match in re.finditer(pattern, content):
                uld_type = match.group(1)
                weight = float(match.group(2))
                compartment = int(match.group(3)[0])

                if 'BAX' not in str(uld_type):
                    if compartment == 1:
                        count_uld_comp_1 += 1
                        weight_uld_comp_1 += weight
                    elif compartment == 2:
                        count_uld_comp_2 += 1
                        weight_uld_comp_2 += weight
                    elif compartment == 3:
                        count_uld_comp_3 += 1
                        weight_uld_comp_3 += weight
                    elif compartment == 4:
                        count_uld_comp_4 += 1
                        weight_uld_comp_4 += weight

                if 'BAX' in str(uld_type):
                    if compartment == 1:
                        count_bax_comp_1 += 1
                        weight_bax_comp_1 += weight
                    elif compartment == 2:
                        count_bax_comp_2 += 1
                        weight_bax_comp_2 += weight
                    elif compartment == 3:
                        count_bax_comp_3 += 1
                        weight_bax_comp_3 += weight
                    elif compartment == 4:
                        count_bax_comp_4 += 1
                        weight_bax_comp_4 += weight

            results['Aircraft Type'] = self.aircraft_type
            results['Aircraft Registration'] = self.aircraft_registration
            results['%MAC ZFW'] = self.MAC
            results['Actual %MAC ZFW'] = self.MAC_actual
            results['Fuel Deviation Percentage'] = self.fuel_reduction_percentage
            results['Fuel Deviation'] = self.fuel_reduction_kg
            results['TOW'] = self.TOW
            results['ULDs Built by Model'] = self.uld_built_by_model
            results['ULDs Actually Used'] = self.uld_actually_used
            results['Weight in Comp 1'] = float(re.search(r'Weight in Compartment 1: ([\d.]+)', content).group(1))
            results['Weight in Comp 2'] = float(re.search(r'Weight in Compartment 2: ([\d.]+)', content).group(1))
            results['Weight in Comp 3'] = float(re.search(r'Weight in Compartment 3: ([\d.]+)', content).group(1))
            results['Weight in Comp 4'] = float(re.search(r'Weight in Compartment 4: ([\d.]+)', content).group(1))
            results['Number of ULDs in Comp 1'] = count_uld_comp_1
            results['Number of BAX ULDs in Comp 1'] = count_bax_comp_1
            results['Weight ULDs in Comp 1'] = weight_uld_comp_1
            results['Weight BAX ULDs in Comp 1'] = weight_bax_comp_1
            results['Number of ULDs in Comp 2'] = count_uld_comp_2
            results['Number of BAX ULDs in Comp 2'] = count_bax_comp_2
            results['Weight ULDs in Comp 2'] = weight_uld_comp_2
            results['Weight BAX ULDs in Comp 2'] = weight_bax_comp_2
            results['Number of ULDs in Comp 3'] = count_uld_comp_3
            results['Number of BAX ULDs in Comp 3'] = count_bax_comp_3
            results['Weight ULDs in Comp 3'] = weight_uld_comp_3
            results['Weight BAX ULDs in Comp 3'] = weight_bax_comp_3
            results['Number of ULDs in Comp 4'] = count_uld_comp_4
            results['Number of BAX ULDs in Comp 4'] = count_bax_comp_4
            results['Weight ULDs in Comp 4'] = weight_uld_comp_4
            results['Weight BAX ULDs in Comp 4'] = weight_bax_comp_4

        with open(model_info_file, 'r') as file:
            content = file.read()

            total_time_data = re.search(r"Total time: ([\d.]+) seconds", content)
            if total_time_data:
                self.total_run_time = float(total_time_data.group(1))


        results['Total Run Time'] = self.total_run_time

        return results

In [17]:
class CombinedResults():
    def __init__(self):
        self.results_reader = ResultsReader()
        self.results_reader_baseline = ResultsReaderBaseline()
        self.results_reader_optimized_actual = ResultsReaderOptimizedActual()
        self.results_reader_bax_fixed = ResultsReaderBAXFixed()
        self.model_results = self.results_reader.read_all_results()
        self.baseline_results = self.results_reader_baseline.read_all_results()
        self.optimized_actual_results = self.results_reader_optimized_actual.read_all_results()
        self.bax_fixed_results = self.results_reader_bax_fixed.read_all_results()   
        self.combined_results = None
        self.combined_results_MAC = None
        self.combined_results_fuel_deviation = None

    def compare_results(self):
        """
        Compare the results of the model with the baseline results
        """
        comparison = self.model_results.merge(self.baseline_results, on=['Flight Number', 'Flight Date', 'Aircraft Type', 'Aircraft Registration', 'TOW'], suffixes=('_model', '_baseline'))
        self.optimized_actual_results.columns = [col + '_optimized_actual' if col not in ['Flight Number', 'Flight Date', 'Aircraft Type', 'Aircraft Registration', 'TOW'] else col for col in self.optimized_actual_results.columns]
        comparison = comparison.merge(self.optimized_actual_results, on=['Flight Number', 'Flight Date', 'Aircraft Type', 'Aircraft Registration', 'TOW'], suffixes=('', '_optimized_actual'))
        self.bax_fixed_results.columns = [col + '_bax_fixed' if col not in ['Flight Number', 'Flight Date', 'Aircraft Type', 'Aircraft Registration', 'TOW'] else col for col in self.bax_fixed_results.columns]
        comparison = comparison.merge(self.bax_fixed_results, on=['Flight Number', 'Flight Date', 'Aircraft Type', 'Aircraft Registration', 'TOW'], suffixes=('', '_bax_fixed'))

        data_analysis = Data_Analysis()
        df_COL_CRT = data_analysis.COL_and_CRT_analysis()

        comparison = comparison.merge(df_COL_CRT, on=['Flight Number', 'Flight Date'], suffixes=('', ''))
        comparison.dropna()
        comparison = comparison[:-2]

        columns_full = ['Flight Number', 'Flight Date', 'Aircraft Type', 'Aircraft Registration', 'TOW','IsCOL', 'IsCRT', 'IsCOLandCRT', '%MAC ZFW_model', 'Actual %MAC ZFW_model', 'Fuel Deviation Percentage_model', 'Fuel Deviation_model', 'ULDs Built by Model_model', 
        'ULDs Actually Used_model', 'Total Run Time_model', 'Number of BAX ULDs_model', 'Number of BUP ULDs_model', 'Number of T ULDs_model', 
        'Number of PAX A_model', 'Number of PAX B_model', 'Number of PAX C_model', 'Number of PAX D_model', 'Number of PAX E_model', 'Number of PAX F_model', 'Number of PAX G_model', 'Total PAX_model', 
        'Number of items_model', 'Weight in Comp 1_model', 'Weight in Comp 2_model', 'Weight in Comp 3_model', 'Weight in Comp 4_model', 'Number of ULDs in Comp 1_model', 'Number of BAX ULDs in Comp 1_model', 
        'Weight ULDs in Comp 1_model', 'Weight BAX ULDs in Comp 1_model', 'Number of ULDs in Comp 2_model', 'Number of BAX ULDs in Comp 2_model', 'Weight ULDs in Comp 2_model', 'Weight BAX ULDs in Comp 2_model', 
        'Number of ULDs in Comp 3_model', 'Number of BAX ULDs in Comp 3_model', 'Weight ULDs in Comp 3_model', 'Weight BAX ULDs in Comp 3_model', 'Number of ULDs in Comp 4_model', 'Number of BAX ULDs in Comp 4_model', 
        'Weight ULDs in Comp 4_model', 'Weight BAX ULDs in Comp 4_model', '%MAC ZFW_baseline', 'Fuel Deviation Percentage_baseline', 'Fuel Deviation_baseline', 'ULDs Built by Model_baseline', 
        'ULDs Actually Used_baseline', 'Total Run Time_baseline', 'Number of BAX ULDs_baseline', 'Number of BUP ULDs_baseline', 'Number of T ULDs_baseline', 
        'Number of PAX A_baseline', 'Number of PAX B_baseline', 'Number of PAX C_baseline', 'Number of PAX D_baseline', 'Number of PAX E_baseline', 'Number of PAX F_baseline', 'Number of PAX G_baseline', 'Total PAX_baseline', 
        'Number of items_baseline', 'Weight in Comp 1_baseline', 'Weight in Comp 2_baseline', 'Weight in Comp 3_baseline', 'Weight in Comp 4_baseline', 'Number of ULDs in Comp 1_baseline', 'Number of BAX ULDs in Comp 1_baseline', 
        'Weight ULDs in Comp 1_baseline', 'Weight BAX ULDs in Comp 1_baseline', 'Number of ULDs in Comp 2_baseline', 'Number of BAX ULDs in Comp 2_baseline', 'Weight ULDs in Comp 2_baseline', 'Weight BAX ULDs in Comp 2_baseline', 
        'Number of ULDs in Comp 3_baseline', 'Number of BAX ULDs in Comp 3_baseline', 'Weight ULDs in Comp 3_baseline', 'Weight BAX ULDs in Comp 3_baseline', 'Number of ULDs in Comp 4_baseline', 'Number of BAX ULDs in Comp 4_baseline', 
        'Weight ULDs in Comp 4_baseline', 'Weight BAX ULDs in Comp 4_baseline', '%MAC ZFW_optimized_actual', 'Fuel Deviation Percentage_optimized_actual', 'Fuel Deviation_optimized_actual', 
        'ULDs Built by Model_optimized_actual', 'ULDs Actually Used_optimized_actual', 'Total Run Time_optimized_actual', 'Number of BAX ULDs_optimized_actual', 'Number of BUP ULDs_optimized_actual', 'Number of T ULDs_optimized_actual', 
        'Number of PAX A_optimized_actual', 'Number of PAX B_optimized_actual', 'Number of PAX C_optimized_actual', 'Number of PAX D_optimized_actual', 'Number of PAX E_optimized_actual', 'Number of PAX F_optimized_actual', 
        'Number of PAX G_optimized_actual', 'Total PAX_optimized_actual', 'Number of items_optimized_actual', 'Weight in Comp 1_optimized_actual', 'Weight in Comp 2_optimized_actual', 'Weight in Comp 3_optimized_actual', 
        'Weight in Comp 4_optimized_actual', 'Number of ULDs in Comp 1_optimized_actual', 'Number of BAX ULDs in Comp 1_optimized_actual', 'Weight ULDs in Comp 1_optimized_actual', 'Weight BAX ULDs in Comp 1_optimized_actual', 
        'Number of ULDs in Comp 2_optimized_actual', 'Number of BAX ULDs in Comp 2_optimized_actual', 'Weight ULDs in Comp 2_optimized_actual', 'Weight BAX ULDs in Comp 2_optimized_actual', 'Number of ULDs in Comp 3_optimized_actual', 
        'Number of BAX ULDs in Comp 3_optimized_actual', 'Weight ULDs in Comp 3_optimized_actual', 'Weight BAX ULDs in Comp 3_optimized_actual', 'Number of ULDs in Comp 4_optimized_actual', 'Number of BAX ULDs in Comp 4_optimized_actual', 
        'Weight ULDs in Comp 4_optimized_actual', 'Weight BAX ULDs in Comp 4_optimized_actual', '%MAC ZFW_bax_fixed', 'Actual %MAC ZFW_bax_fixed', 'Fuel Deviation Percentage_bax_fixed', 'Fuel Deviation_bax_fixed', 'ULDs Built by Model_bax_fixed', 
        'ULDs Actually Used_bax_fixed', 'Total Run Time_bax_fixed', 'Number of BAX ULDs_bax_fixed', 'Number of BUP ULDs_bax_fixed', 'Number of T ULDs_bax_fixed', 
        'Number of PAX A_bax_fixed', 'Number of PAX B_bax_fixed', 'Number of PAX C_bax_fixed', 'Number of PAX D_bax_fixed', 'Number of PAX E_bax_fixed', 'Number of PAX F_bax_fixed', 'Number of PAX G_bax_fixed', 'Total PAX_bax_fixed', 
        'Number of items_bax_fixed', 'Weight in Comp 1_bax_fixed', 'Weight in Comp 2_bax_fixed', 'Weight in Comp 3_bax_fixed', 'Weight in Comp 4_bax_fixed', 'Number of ULDs in Comp 1_bax_fixed', 'Number of BAX ULDs in Comp 1_bax_fixed', 
        'Weight ULDs in Comp 1_bax_fixed', 'Weight BAX ULDs in Comp 1_bax_fixed', 'Number of ULDs in Comp 2_bax_fixed', 'Number of BAX ULDs in Comp 2_bax_fixed', 'Weight ULDs in Comp 2_bax_fixed', 'Weight BAX ULDs in Comp 2_bax_fixed', 
        'Number of ULDs in Comp 3_bax_fixed', 'Number of BAX ULDs in Comp 3_bax_fixed', 'Weight ULDs in Comp 3_bax_fixed', 'Weight BAX ULDs in Comp 3_bax_fixed', 'Number of ULDs in Comp 4_bax_fixed', 'Number of BAX ULDs in Comp 4_bax_fixed', 
        'Weight ULDs in Comp 4_bax_fixed', 'Weight BAX ULDs in Comp 4_bax_fixed']

        comparison_full = comparison[columns_full].copy()
        comparison_full.rename(columns = {'Actual %MAC ZFW_model': 'Actual %MAC ZFW'}, inplace=True)

        columns_general_MAC = ['Flight Number', 'Flight Date', 'Aircraft Type', '%MAC ZFW_model', '%MAC ZFW_baseline', '%MAC ZFW_optimized_actual', '%MAC ZFW_bax_fixed', 'Actual %MAC ZFW_model']
        columns_fuel_deviation = ['Flight Number', 'Flight Date', 'Aircraft Type', 'Fuel Deviation Percentage_model', 'Fuel Deviation Percentage_baseline', 'Fuel Deviation Percentage_optimized_actual', 'Fuel Deviation Percentage_bax_fixed',
                                  'Fuel Deviation_model', 'Fuel Deviation_baseline', 'Fuel Deviation_optimized_actual', 'Fuel Deviation_bax_fixed']

        comparison_MAC = comparison[columns_general_MAC].copy()
        comparison_MAC.rename(columns = {'Actual %MAC ZFW_model': 'Actual %MAC ZFW'}, inplace=True)
        cols_to_convert_MAC = ['%MAC ZFW_model', '%MAC ZFW_baseline', '%MAC ZFW_optimized_actual', '%MAC ZFW_bax_fixed', 'Actual %MAC ZFW']
        comparison_MAC[cols_to_convert_MAC] = comparison_MAC[cols_to_convert_MAC].apply(pd.to_numeric, errors='coerce')

        comparison_fuel_deviation = comparison[columns_fuel_deviation].copy()
        cols_to_convert_fuel_deviation = ['Fuel Deviation Percentage_model', 'Fuel Deviation Percentage_baseline', 'Fuel Deviation Percentage_optimized_actual', 'Fuel Deviation Percentage_bax_fixed',
                                           'Fuel Deviation_model', 'Fuel Deviation_baseline', 'Fuel Deviation_optimized_actual', 'Fuel Deviation_bax_fixed']



        average_data_general_MAC = {
            'Flight Number': 'Average',
            'Flight Date': '-',
            'Aircraft Type': '-',
            'Aircraft Registration': '-',
            '%MAC ZFW_model': comparison_MAC['%MAC ZFW_model'].mean(),
            '%MAC ZFW_baseline': comparison_MAC['%MAC ZFW_baseline'].mean(),
            '%MAC ZFW_optimized_actual': comparison_MAC['%MAC ZFW_optimized_actual'].mean(),
            '%MAC ZFW_bax_fixed': comparison_MAC['%MAC ZFW_bax_fixed'].mean(),
            'Actual %MAC ZFW': comparison_MAC['Actual %MAC ZFW'].mean()
        }

        average_data_fuel_deviation = {
            'Flight Number': 'Average',
            'Flight Date': '-',
            'Aircraft Type': '-',
            'Aircraft Registration': '-',
            'Fuel Deviation Percentage_model': comparison_fuel_deviation['Fuel Deviation Percentage_model'].mean(),
            'Fuel Deviation Percentage_baseline': comparison_fuel_deviation['Fuel Deviation Percentage_baseline'].mean(),
            'Fuel Deviation Percentage_optimized_actual': comparison_fuel_deviation['Fuel Deviation Percentage_optimized_actual'].mean(),
            'Fuel Deviation Percentage_bax_fixed': comparison_fuel_deviation['Fuel Deviation Percentage_bax_fixed'].mean(),
            'Fuel Deviation_model': comparison_fuel_deviation['Fuel Deviation_model'].mean(),
            'Fuel Deviation_baseline': comparison_fuel_deviation['Fuel Deviation_baseline'].mean(),
            'Fuel Deviation_optimized_actual': comparison_fuel_deviation['Fuel Deviation_optimized_actual'].mean(),
            'Fuel Deviation_bax_fixed': comparison_fuel_deviation['Fuel Deviation_bax_fixed'].mean()
        }

        sum_data_fuel_deviation = {
            'Flight Number': 'Total',
            'Flight Date': '-',
            'Aircraft Type': '-',
            'Aircraft Registration': '-',
            'Fuel Deviation Percentage_model': '-',
            'Fuel Deviation Percentage_baseline': '-',
            'Fuel Deviation Percentage_optimized_actual': '-',
            'Fuel Deviation Percentage_bax_fixed': '-',
            'Fuel Deviation_model': comparison_fuel_deviation['Fuel Deviation_model'].sum(),
            'Fuel Deviation_baseline': comparison_fuel_deviation['Fuel Deviation_baseline'].sum(),
            'Fuel Deviation_optimized_actual': comparison_fuel_deviation['Fuel Deviation_optimized_actual'].sum(),
            'Fuel Deviation_bax_fixed': comparison_fuel_deviation['Fuel Deviation_bax_fixed'].sum()
        }

        sum_data_full = {
            'Flight Number': 'Total',
            'Flight Date': '-',
            'TOW': comparison_full['TOW'].sum(),
            'Fuel Deviation Percentage_model': '-',
            'Fuel Deviation_model': comparison_full['Fuel Deviation_model'].sum(),
            'ULDs Built by Model_model': comparison_full['ULDs Built by Model_model'].sum(),
            'ULDs Actually Used_model': comparison_full['ULDs Actually Used_model'].sum(),
            '%MAC ZFW_model': '-',
            'Actual %MAC ZFW': '-',
            'Total Run Time_model': comparison_full['Total Run Time_model'].sum(),
            'Number of BAX ULDs_model': comparison_full['Number of BAX ULDs_model'].sum(),
            'Number of BUP ULDs_model': comparison_full['Number of BUP ULDs_model'].sum(),
            'Number of T ULDs_model': comparison_full['Number of T ULDs_model'].sum(),
            'Number of PAX A_model': '-',
            'Number of PAX B_model': '-',
            'Number of PAX C_model': '-',
            'Number of PAX D_model': '-',
            'Number of PAX E_model': '-',
            'Number of PAX F_model': '-',
            'Number of PAX G_model': '-',
            'Total PAX_model': '-',
            'Number of items_model': comparison_full['Number of items_model'].sum(),
            'Weight in Comp 1_model': comparison_full['Weight in Comp 1_model'].sum(),
            'Weight in Comp 2_model': comparison_full['Weight in Comp 2_model'].sum(),
            'Weight in Comp 3_model': comparison_full['Weight in Comp 3_model'].sum(),
            'Weight in Comp 4_model': comparison_full['Weight in Comp 4_model'].sum(),
            'Number of ULDs in Comp 1_model': comparison_full['Number of ULDs in Comp 1_model'].sum(),
            'Number of BAX ULDs in Comp 1_model': comparison_full['Number of BAX ULDs in Comp 1_model'].sum(),
            'Weight ULDs in Comp 1_model': comparison_full['Weight ULDs in Comp 1_model'].sum(),
            'Weight BAX ULDs in Comp 1_model': comparison_full['Weight BAX ULDs in Comp 1_model'].sum(),
            'Number of ULDs in Comp 2_model': comparison_full['Number of ULDs in Comp 2_model'].sum(),
            'Number of BAX ULDs in Comp 2_model': comparison_full['Number of BAX ULDs in Comp 2_model'].sum(),
            'Weight ULDs in Comp 2_model': comparison_full['Weight ULDs in Comp 2_model'].sum(),
            'Weight BAX ULDs in Comp 2_model': comparison_full['Weight BAX ULDs in Comp 2_model'].sum(),
            'Number of ULDs in Comp 3_model': comparison_full['Number of ULDs in Comp 3_model'].sum(),
            'Number of BAX ULDs in Comp 3_model': comparison_full['Number of BAX ULDs in Comp 3_model'].sum(),
            'Weight ULDs in Comp 3_model': comparison_full['Weight ULDs in Comp 3_model'].sum(),
            'Weight BAX ULDs in Comp 3_model': comparison_full['Weight BAX ULDs in Comp 3_model'].sum(),
            'Number of ULDs in Comp 4_model': comparison_full['Number of ULDs in Comp 4_model'].sum(),
            'Number of BAX ULDs in Comp 4_model': comparison_full['Number of BAX ULDs in Comp 4_model'].sum(),
            'Weight ULDs in Comp 4_model': comparison_full['Weight ULDs in Comp 4_model'].sum(),
            'Weight BAX ULDs in Comp 4_model': comparison_full['Weight BAX ULDs in Comp 4_model'].sum(),
            'Fuel Deviation Percentage_baseline': '-',
            'Fuel Deviation_baseline': comparison_full['Fuel Deviation_baseline'].sum(),
            'ULDs Built by Model_baseline': comparison_full['ULDs Built by Model_baseline'].sum(),
            'ULDs Actually Used_baseline': comparison_full['ULDs Actually Used_baseline'].sum(),
            '%MAC ZFW_baseline': '-',
            'Total Run Time_baseline': comparison_full['Total Run Time_baseline'].sum(),
            'Number of BAX ULDs_baseline': comparison_full['Number of BAX ULDs_baseline'].sum(),
            'Number of BUP ULDs_baseline': comparison_full['Number of BUP ULDs_baseline'].sum(),
            'Number of T ULDs_baseline': comparison_full['Number of T ULDs_baseline'].sum(),
            'Number of PAX A_baseline': '-',
            'Number of PAX B_baseline': '-',
            'Number of PAX C_baseline': '-',
            'Number of PAX D_baseline': '-',
            'Number of PAX E_baseline': '-',
            'Number of PAX F_baseline': '-',
            'Number of PAX G_baseline': '-',
            'Total PAX_baseline': '-',
            'Number of items_baseline': comparison_full['Number of items_baseline'].sum(),
            'Weight in Comp 1_baseline': comparison_full['Weight in Comp 1_baseline'].sum(),
            'Weight in Comp 2_baseline': comparison_full['Weight in Comp 2_baseline'].sum(),
            'Weight in Comp 3_baseline': comparison_full['Weight in Comp 3_baseline'].sum(),
            'Weight in Comp 4_baseline': comparison_full['Weight in Comp 4_baseline'].sum(),
            'Number of ULDs in Comp 1_baseline': comparison_full['Number of ULDs in Comp 1_baseline'].sum(),
            'Number of BAX ULDs in Comp 1_baseline': comparison_full['Number of BAX ULDs in Comp 1_baseline'].sum(),
            'Weight ULDs in Comp 1_baseline': comparison_full['Weight ULDs in Comp 1_baseline'].sum(),
            'Weight BAX ULDs in Comp 1_baseline': comparison_full['Weight BAX ULDs in Comp 1_baseline'].sum(),
            'Number of ULDs in Comp 2_baseline': comparison_full['Number of ULDs in Comp 2_baseline'].sum(),
            'Number of BAX ULDs in Comp 2_baseline': comparison_full['Number of BAX ULDs in Comp 2_baseline'].sum(),
            'Weight ULDs in Comp 2_baseline': comparison_full['Weight ULDs in Comp 2_baseline'].sum(),
            'Weight BAX ULDs in Comp 2_baseline': comparison_full['Weight BAX ULDs in Comp 2_baseline'].sum(),
            'Number of ULDs in Comp 3_baseline': comparison_full['Number of ULDs in Comp 3_baseline'].sum(),
            'Number of BAX ULDs in Comp 3_baseline': comparison_full['Number of BAX ULDs in Comp 3_baseline'].sum(),
            'Weight ULDs in Comp 3_baseline': comparison_full['Weight ULDs in Comp 3_baseline'].sum(),
            'Weight BAX ULDs in Comp 3_baseline': comparison_full['Weight BAX ULDs in Comp 3_baseline'].sum(),
            'Number of ULDs in Comp 4_baseline': comparison_full['Number of ULDs in Comp 4_baseline'].sum(),
            'Number of BAX ULDs in Comp 4_baseline': comparison_full['Number of BAX ULDs in Comp 4_baseline'].sum(),
            'Weight ULDs in Comp 4_baseline': comparison_full['Weight ULDs in Comp 4_baseline'].sum(),
            'Weight BAX ULDs in Comp 4_baseline': comparison_full['Weight BAX ULDs in Comp 4_baseline'].sum(),
            'Fuel Deviation Percentage_optimized_actual': '-',
            'Fuel Deviation_optimized_actual': comparison_full['Fuel Deviation_optimized_actual'].sum(),
            'ULDs Built by Model_optimized_actual': comparison_full['ULDs Built by Model_optimized_actual'].sum(),
            'ULDs Actually Used_optimized_actual': comparison_full['ULDs Actually Used_optimized_actual'].sum(),
            '%MAC ZFW_optimized_actual': '-',
            'Total Run Time_optimized_actual': comparison_full['Total Run Time_optimized_actual'].sum(),
            'Number of BAX ULDs_optimized_actual': comparison_full['Number of BAX ULDs_optimized_actual'].sum(),
            'Number of BUP ULDs_optimized_actual': comparison_full['Number of BUP ULDs_optimized_actual'].sum(),
            'Number of T ULDs_optimized_actual': comparison_full['Number of T ULDs_optimized_actual'].sum(),
            'Number of PAX A_optimized_actual': '-',
            'Number of PAX B_optimized_actual': '-',
            'Number of PAX C_optimized_actual': '-',
            'Number of PAX D_optimized_actual': '-',
            'Number of PAX E_optimized_actual': '-',
            'Number of PAX F_optimized_actual': '-',
            'Number of PAX G_optimized_actual': '-',
            'Total PAX_optimized_actual': '-',
            'Number of items_optimized_actual': comparison_full['Number of items_optimized_actual'].sum(),
            'Weight in Comp 1_optimized_actual': comparison_full['Weight in Comp 1_optimized_actual'].sum(),
            'Weight in Comp 2_optimized_actual': comparison_full['Weight in Comp 2_optimized_actual'].sum(),
            'Weight in Comp 3_optimized_actual': comparison_full['Weight in Comp 3_optimized_actual'].sum(),
            'Weight in Comp 4_optimized_actual': comparison_full['Weight in Comp 4_optimized_actual'].sum(),
            'Number of ULDs in Comp 1_optimized_actual': comparison_full['Number of ULDs in Comp 1_optimized_actual'].sum(),
            'Number of BAX ULDs in Comp 1_optimized_actual': comparison_full['Number of BAX ULDs in Comp 1_optimized_actual'].sum(),
            'Weight ULDs in Comp 1_optimized_actual': comparison_full['Weight ULDs in Comp 1_optimized_actual'].sum(),
            'Weight BAX ULDs in Comp 1_optimized_actual': comparison_full['Weight BAX ULDs in Comp 1_optimized_actual'].sum(),
            'Number of ULDs in Comp 2_optimized_actual': comparison_full['Number of ULDs in Comp 2_optimized_actual'].sum(),
            'Number of BAX ULDs in Comp 2_optimized_actual': comparison_full['Number of BAX ULDs in Comp 2_optimized_actual'].sum(),
            'Weight ULDs in Comp 2_optimized_actual': comparison_full['Weight ULDs in Comp 2_optimized_actual'].sum(),
            'Weight BAX ULDs in Comp 2_optimized_actual': comparison_full['Weight BAX ULDs in Comp 2_optimized_actual'].sum(),
            'Number of ULDs in Comp 3_optimized_actual': comparison_full['Number of ULDs in Comp 3_optimized_actual'].sum(),
            'Number of BAX ULDs in Comp 3_optimized_actual': comparison_full['Number of BAX ULDs in Comp 3_optimized_actual'].sum(),
            'Weight ULDs in Comp 3_optimized_actual': comparison_full['Weight ULDs in Comp 3_optimized_actual'].sum(),
            'Weight BAX ULDs in Comp 3_optimized_actual': comparison_full['Weight BAX ULDs in Comp 3_optimized_actual'].sum(),
            'Number of ULDs in Comp 4_optimized_actual': comparison_full['Number of ULDs in Comp 4_optimized_actual'].sum(),
            'Number of BAX ULDs in Comp 4_optimized_actual': comparison_full['Number of BAX ULDs in Comp 4_optimized_actual'].sum(),
            'Weight ULDs in Comp 4_optimized_actual': comparison_full['Weight ULDs in Comp 4_optimized_actual'].sum(),
            'Weight BAX ULDs in Comp 4_optimized_actual': comparison_full['Weight BAX ULDs in Comp 4_optimized_actual'].sum(),
            'Fuel Deviation_bax_fixed': comparison_full['Fuel Deviation_bax_fixed'].sum(),
            'ULDs Built by Model_bax_fixed': comparison_full['ULDs Built by Model_bax_fixed'].sum(),
            'ULDs Actually Used_bax_fixed': comparison_full['ULDs Actually Used_bax_fixed'].sum(),
            '%MAC ZFW_bax_fixed': '-',
            'Actual %MAC ZFW': '-',
            'Total Run Time_bax_fixed': comparison_full['Total Run Time_bax_fixed'].sum(),
            'Number of BAX ULDs_bax_fixed': comparison_full['Number of BAX ULDs_bax_fixed'].sum(),
            'Number of BUP ULDs_bax_fixed': comparison_full['Number of BUP ULDs_bax_fixed'].sum(),
            'Number of T ULDs_bax_fixed': comparison_full['Number of T ULDs_bax_fixed'].sum(),
            'Number of PAX A_bax_fixed': '-',
            'Number of PAX B_bax_fixed': '-',
            'Number of PAX C_bax_fixed': '-',
            'Number of PAX D_bax_fixed': '-',
            'Number of PAX E_bax_fixed': '-',
            'Number of PAX F_bax_fixed': '-',
            'Number of PAX G_bax_fixed': '-',
            'Total PAX_bax_fixed': '-',
            'Number of items_bax_fixed': comparison_full['Number of items_bax_fixed'].sum(),
            'Weight in Comp 1_bax_fixed': comparison_full['Weight in Comp 1_bax_fixed'].sum(),
            'Weight in Comp 2_bax_fixed': comparison_full['Weight in Comp 2_bax_fixed'].sum(),
            'Weight in Comp 3_bax_fixed': comparison_full['Weight in Comp 3_bax_fixed'].sum(),
            'Weight in Comp 4_bax_fixed': comparison_full['Weight in Comp 4_bax_fixed'].sum(),
            'Number of ULDs in Comp 1_bax_fixed': comparison_full['Number of ULDs in Comp 1_bax_fixed'].sum(),
            'Number of BAX ULDs in Comp 1_bax_fixed': comparison_full['Number of BAX ULDs in Comp 1_bax_fixed'].sum(),
            'Weight ULDs in Comp 1_bax_fixed': comparison_full['Weight ULDs in Comp 1_bax_fixed'].sum(),
            'Weight BAX ULDs in Comp 1_bax_fixed': comparison_full['Weight BAX ULDs in Comp 1_bax_fixed'].sum(),
            'Number of ULDs in Comp 2_bax_fixed': comparison_full['Number of ULDs in Comp 2_bax_fixed'].sum(),
            'Number of BAX ULDs in Comp 2_bax_fixed': comparison_full['Number of BAX ULDs in Comp 2_bax_fixed'].sum(),
            'Weight ULDs in Comp 2_bax_fixed': comparison_full['Weight ULDs in Comp 2_bax_fixed'].sum(),
            'Weight BAX ULDs in Comp 2_bax_fixed': comparison_full['Weight BAX ULDs in Comp 2_bax_fixed'].sum(),
            'Number of ULDs in Comp 3_bax_fixed': comparison_full['Number of ULDs in Comp 3_bax_fixed'].sum(),
            'Number of BAX ULDs in Comp 3_bax_fixed': comparison_full['Number of BAX ULDs in Comp 3_bax_fixed'].sum(),
            'Weight ULDs in Comp 3_bax_fixed': comparison_full['Weight ULDs in Comp 3_bax_fixed'].sum(),
            'Weight BAX ULDs in Comp 3_bax_fixed': comparison_full['Weight BAX ULDs in Comp 3_bax_fixed'].sum(),
            'Number of ULDs in Comp 4_bax_fixed': comparison_full['Number of ULDs in Comp 4_bax_fixed'].sum(),
            'Number of BAX ULDs in Comp 4_bax_fixed': comparison_full['Number of BAX ULDs in Comp 4_bax_fixed'].sum(),
            'Weight ULDs in Comp 4_bax_fixed': comparison_full['Weight ULDs in Comp 4_bax_fixed'].sum(),
            'Weight BAX ULDs in Comp 4_bax_fixed': comparison_full['Weight BAX ULDs in Comp 4_bax_fixed'].sum()
        }

        average_data_full = {
            'Flight Number': 'Average',
            'Flight Date': '-',
            'TOW': comparison_full['TOW'].mean(),
            'Fuel Deviation Percentage_model': comparison_full['Fuel Deviation Percentage_model'].mean(),
            'Fuel Deviation_model': comparison_full['Fuel Deviation_model'].mean(),
            'ULDs Built by Model_model': comparison_full['ULDs Built by Model_model'].mean(),
            'ULDs Actually Used_model': comparison_full['ULDs Actually Used_model'].mean(),
            '%MAC ZFW_model': comparison_full['%MAC ZFW_model'].mean(),
            'Total Run Time_model': comparison_full['Total Run Time_model'].mean(),
            'Number of BAX ULDs_model': comparison_full['Number of BAX ULDs_model'].mean(),
            'Number of BUP ULDs_model': comparison_full['Number of BUP ULDs_model'].mean(),
            'Number of T ULDs_model': comparison_full['Number of T ULDs_model'].mean(),
            'Number of PAX A_model': comparison_full['Number of PAX A_model'].mean(),
            'Number of PAX B_model': comparison_full['Number of PAX B_model'].mean(),
            'Number of PAX C_model': comparison_full['Number of PAX C_model'].mean(),
            'Number of PAX D_model': comparison_full['Number of PAX D_model'].mean(),
            'Number of PAX E_model': comparison_full['Number of PAX E_model'].mean(),
            'Number of PAX F_model': comparison_full['Number of PAX F_model'].mean(),
            'Number of PAX G_model': comparison_full['Number of PAX G_model'].mean(),
            'Total PAX_model': comparison_full['Total PAX_model'].mean(),
            'Number of items_model': comparison_full['Number of items_model'].mean(),
            'Weight in Comp 1_model': comparison_full['Weight in Comp 1_model'].mean(),
            'Weight in Comp 2_model': comparison_full['Weight in Comp 2_model'].mean(),
            'Weight in Comp 3_model': comparison_full['Weight in Comp 3_model'].mean(),
            'Weight in Comp 4_model': comparison_full['Weight in Comp 4_model'].mean(),
            'Number of ULDs in Comp 1_model': comparison_full['Number of ULDs in Comp 1_model'].mean(),
            'Number of BAX ULDs in Comp 1_model': comparison_full['Number of BAX ULDs in Comp 1_model'].mean(),
            'Weight ULDs in Comp 1_model': comparison_full['Weight ULDs in Comp 1_model'].mean(),
            'Weight BAX ULDs in Comp 1_model': comparison_full['Weight BAX ULDs in Comp 1_model'].mean(),
            'Number of ULDs in Comp 2_model': comparison_full['Number of ULDs in Comp 2_model'].mean(),
            'Number of BAX ULDs in Comp 2_model': comparison_full['Number of BAX ULDs in Comp 2_model'].mean(),
            'Weight ULDs in Comp 2_model': comparison_full['Weight ULDs in Comp 2_model'].mean(),
            'Weight BAX ULDs in Comp 2_model': comparison_full['Weight BAX ULDs in Comp 2_model'].mean(),
            'Number of ULDs in Comp 3_model': comparison_full['Number of ULDs in Comp 3_model'].mean(),
            'Number of BAX ULDs in Comp 3_model': comparison_full['Number of BAX ULDs in Comp 3_model'].mean(),
            'Weight ULDs in Comp 3_model': comparison_full['Weight ULDs in Comp 3_model'].mean(),
            'Weight BAX ULDs in Comp 3_model': comparison_full['Weight BAX ULDs in Comp 3_model'].mean(),
            'Number of ULDs in Comp 4_model': comparison_full['Number of ULDs in Comp 4_model'].mean(),
            'Number of BAX ULDs in Comp 4_model': comparison_full['Number of BAX ULDs in Comp 4_model'].mean(),
            'Weight ULDs in Comp 4_model': comparison_full['Weight ULDs in Comp 4_model'].mean(),
            'Weight BAX ULDs in Comp 4_model': comparison_full['Weight BAX ULDs in Comp 4_model'].mean(),
            'Fuel Deviation Percentage_baseline': comparison_full['Fuel Deviation Percentage_baseline'].mean(),
            'Fuel Deviation_baseline': comparison_full['Fuel Deviation_baseline'].mean(),
            'ULDs Built by Model_baseline': comparison_full['ULDs Built by Model_baseline'].mean(),
            'ULDs Actually Used_baseline': comparison_full['ULDs Actually Used_baseline'].mean(),
            '%MAC ZFW_baseline': comparison_full['%MAC ZFW_baseline'].mean(),
            'Total Run Time_baseline': comparison_full['Total Run Time_baseline'].mean(),
            'Number of BAX ULDs_baseline': comparison_full['Number of BAX ULDs_baseline'].mean(),
            'Number of BUP ULDs_baseline': comparison_full['Number of BUP ULDs_baseline'].mean(),
            'Number of T ULDs_baseline': comparison_full['Number of T ULDs_baseline'].mean(),
            'Number of PAX A_baseline': comparison_full['Number of PAX A_baseline'].mean(),
            'Number of PAX B_baseline': comparison_full['Number of PAX B_baseline'].mean(),
            'Number of PAX C_baseline': comparison_full['Number of PAX C_baseline'].mean(),
            'Number of PAX D_baseline': comparison_full['Number of PAX D_baseline'].mean(),
            'Number of PAX E_baseline': comparison_full['Number of PAX E_baseline'].mean(),
            'Number of PAX F_baseline': comparison_full['Number of PAX F_baseline'].mean(),
            'Number of PAX G_baseline': comparison_full['Number of PAX G_baseline'].mean(),
            'Total PAX_baseline': comparison_full['Total PAX_baseline'].mean(),
            'Number of items_baseline': comparison_full['Number of items_baseline'].mean(),
            'Weight in Comp 1_baseline': comparison_full['Weight in Comp 1_baseline'].mean(),
            'Weight in Comp 2_baseline': comparison_full['Weight in Comp 2_baseline'].mean(),
            'Weight in Comp 3_baseline': comparison_full['Weight in Comp 3_baseline'].mean(),
            'Weight in Comp 4_baseline': comparison_full['Weight in Comp 4_baseline'].mean(),
            'Number of ULDs in Comp 1_baseline': comparison_full['Number of ULDs in Comp 1_baseline'].mean(),
            'Number of BAX ULDs in Comp 1_baseline': comparison_full['Number of BAX ULDs in Comp 1_baseline'].mean(),
            'Weight ULDs in Comp 1_baseline': comparison_full['Weight ULDs in Comp 1_baseline'].mean(),
            'Weight BAX ULDs in Comp 1_baseline': comparison_full['Weight BAX ULDs in Comp 1_baseline'].mean(),
            'Number of ULDs in Comp 2_baseline': comparison_full['Number of ULDs in Comp 2_baseline'].mean(),
            'Number of BAX ULDs in Comp 2_baseline': comparison_full['Number of BAX ULDs in Comp 2_baseline'].mean(),
            'Weight ULDs in Comp 2_baseline': comparison_full['Weight ULDs in Comp 2_baseline'].mean(),
            'Weight BAX ULDs in Comp 2_baseline': comparison_full['Weight BAX ULDs in Comp 2_baseline'].mean(),
            'Number of ULDs in Comp 3_baseline': comparison_full['Number of ULDs in Comp 3_baseline'].mean(),
            'Number of BAX ULDs in Comp 3_baseline': comparison_full['Number of BAX ULDs in Comp 3_baseline'].mean(),
            'Weight ULDs in Comp 3_baseline': comparison_full['Weight ULDs in Comp 3_baseline'].mean(),
            'Weight BAX ULDs in Comp 3_baseline': comparison_full['Weight BAX ULDs in Comp 3_baseline'].mean(),
            'Number of ULDs in Comp 4_baseline': comparison_full['Number of ULDs in Comp 4_baseline'].mean(),
            'Number of BAX ULDs in Comp 4_baseline': comparison_full['Number of BAX ULDs in Comp 4_baseline'].mean(),
            'Weight ULDs in Comp 4_baseline': comparison_full['Weight ULDs in Comp 4_baseline'].mean(),
            'Weight BAX ULDs in Comp 4_baseline': comparison_full['Weight BAX ULDs in Comp 4_baseline'].mean(),
            'Fuel Deviation Percentage_optimized_actual': comparison_full['Fuel Deviation Percentage_optimized_actual'].mean(),
            'Fuel Deviation_optimized_actual': comparison_full['Fuel Deviation_optimized_actual'].mean(),
            'ULDs Built by Model_optimized_actual': comparison_full['ULDs Built by Model_optimized_actual'].mean(),
            'ULDs Actually Used_optimized_actual': comparison_full['ULDs Actually Used_optimized_actual'].mean(),
            '%MAC ZFW_optimized_actual': comparison_full['%MAC ZFW_optimized_actual'].mean(),
            'Total Run Time_optimized_actual': comparison_full['Total Run Time_optimized_actual'].mean(),
            'Number of BAX ULDs_optimized_actual': comparison_full['Number of BAX ULDs_optimized_actual'].mean(),
            'Number of BUP ULDs_optimized_actual': comparison_full['Number of BUP ULDs_optimized_actual'].mean(),
            'Number of T ULDs_optimized_actual': comparison_full['Number of T ULDs_optimized_actual'].mean(),
            'Number of PAX A_optimized_actual': comparison_full['Number of PAX A_optimized_actual'].mean(),
            'Number of PAX B_optimized_actual': comparison_full['Number of PAX B_optimized_actual'].mean(),
            'Number of PAX C_optimized_actual': comparison_full['Number of PAX C_optimized_actual'].mean(),
            'Number of PAX D_optimized_actual': comparison_full['Number of PAX D_optimized_actual'].mean(),
            'Number of PAX E_optimized_actual': comparison_full['Number of PAX E_optimized_actual'].mean(),
            'Number of PAX F_optimized_actual': comparison_full['Number of PAX F_optimized_actual'].mean(),
            'Number of PAX G_optimized_actual': comparison_full['Number of PAX G_optimized_actual'].mean(),
            'Total PAX_optimized_actual': comparison_full['Total PAX_optimized_actual'].mean(),
            'Number of items_optimized_actual': comparison_full['Number of items_optimized_actual'].mean(),
            'Weight in Comp 1_optimized_actual': comparison_full['Weight in Comp 1_optimized_actual'].mean(),
            'Weight in Comp 2_optimized_actual': comparison_full['Weight in Comp 2_optimized_actual'].mean(),
            'Weight in Comp 3_optimized_actual': comparison_full['Weight in Comp 3_optimized_actual'].mean(),
            'Weight in Comp 4_optimized_actual': comparison_full['Weight in Comp 4_optimized_actual'].mean(),
            'Number of ULDs in Comp 1_optimized_actual': comparison_full['Number of ULDs in Comp 1_optimized_actual'].mean(),
            'Number of BAX ULDs in Comp 1_optimized_actual': comparison_full['Number of BAX ULDs in Comp 1_optimized_actual'].mean(),
            'Weight ULDs in Comp 1_optimized_actual': comparison_full['Weight ULDs in Comp 1_optimized_actual'].mean(),
            'Weight BAX ULDs in Comp 1_optimized_actual': comparison_full['Weight BAX ULDs in Comp 1_optimized_actual'].mean(),
            'Number of ULDs in Comp 2_optimized_actual': comparison_full['Number of ULDs in Comp 2_optimized_actual'].mean(),
            'Number of BAX ULDs in Comp 2_optimized_actual': comparison_full['Number of BAX ULDs in Comp 2_optimized_actual'].mean(),
            'Weight ULDs in Comp 2_optimized_actual': comparison_full['Weight ULDs in Comp 2_optimized_actual'].mean(),
            'Weight BAX ULDs in Comp 2_optimized_actual': comparison_full['Weight BAX ULDs in Comp 2_optimized_actual'].mean(),
            'Number of ULDs in Comp 3_optimized_actual': comparison_full['Number of ULDs in Comp 3_optimized_actual'].mean(),
            'Number of BAX ULDs in Comp 3_optimized_actual': comparison_full['Number of BAX ULDs in Comp 3_optimized_actual'].mean(),
            'Weight ULDs in Comp 3_optimized_actual': comparison_full['Weight ULDs in Comp 3_optimized_actual'].mean(),
            'Weight BAX ULDs in Comp 3_optimized_actual': comparison_full['Weight BAX ULDs in Comp 3_optimized_actual'].mean(),
            'Number of ULDs in Comp 4_optimized_actual': comparison_full['Number of ULDs in Comp 4_optimized_actual'].mean(),
            'Number of BAX ULDs in Comp 4_optimized_actual': comparison_full['Number of BAX ULDs in Comp 4_optimized_actual'].mean(),
            'Weight ULDs in Comp 4_optimized_actual': comparison_full['Weight ULDs in Comp 4_optimized_actual'].mean(),
            'Weight BAX ULDs in Comp 4_optimized_actual': comparison_full['Weight BAX ULDs in Comp 4_optimized_actual'].mean(),
            'Fuel Deviation Percentage_bax_fixed': comparison_full['Fuel Deviation Percentage_bax_fixed'].mean(),
            'Fuel Deviation_bax_fixed': comparison_full['Fuel Deviation_bax_fixed'].mean(),
            'ULDs Built by Model_bax_fixed': comparison_full['ULDs Built by Model_bax_fixed'].mean(),
            'ULDs Actually Used_bax_fixed': comparison_full['ULDs Actually Used_bax_fixed'].mean(),
            '%MAC ZFW_bax_fixed': comparison_full['%MAC ZFW_bax_fixed'].mean(),
            'Total Run Time_bax_fixed': comparison_full['Total Run Time_bax_fixed'].mean(),
            'Number of BAX ULDs_bax_fixed': comparison_full['Number of BAX ULDs_bax_fixed'].mean(),
            'Number of BUP ULDs_bax_fixed': comparison_full['Number of BUP ULDs_bax_fixed'].mean(),
            'Number of T ULDs_bax_fixed': comparison_full['Number of T ULDs_bax_fixed'].mean(),
            'Number of PAX A_bax_fixed': comparison_full['Number of PAX A_bax_fixed'].mean(),
            'Number of PAX B_bax_fixed': comparison_full['Number of PAX B_bax_fixed'].mean(),
            'Number of PAX C_bax_fixed': comparison_full['Number of PAX C_bax_fixed'].mean(),
            'Number of PAX D_bax_fixed': comparison_full['Number of PAX D_bax_fixed'].mean(),
            'Number of PAX E_bax_fixed': comparison_full['Number of PAX E_bax_fixed'].mean(),
            'Number of PAX F_bax_fixed': comparison_full['Number of PAX F_bax_fixed'].mean(),
            'Number of PAX G_bax_fixed': comparison_full['Number of PAX G_bax_fixed'].mean(),
            'Total PAX_bax_fixed': comparison_full['Total PAX_bax_fixed'].mean(),
            'Number of items_bax_fixed': comparison_full['Number of items_bax_fixed'].mean(),
            'Weight in Comp 1_bax_fixed': comparison_full['Weight in Comp 1_bax_fixed'].mean(),
            'Weight in Comp 2_bax_fixed': comparison_full['Weight in Comp 2_bax_fixed'].mean(),
            'Weight in Comp 3_bax_fixed': comparison_full['Weight in Comp 3_bax_fixed'].mean(),
            'Weight in Comp 4_bax_fixed': comparison_full['Weight in Comp 4_bax_fixed'].mean(),
            'Number of ULDs in Comp 1_bax_fixed': comparison_full['Number of ULDs in Comp 1_bax_fixed'].mean(),
            'Number of BAX ULDs in Comp 1_bax_fixed': comparison_full['Number of BAX ULDs in Comp 1_bax_fixed'].mean(),
            'Weight ULDs in Comp 1_bax_fixed': comparison_full['Weight ULDs in Comp 1_bax_fixed'].mean(),
            'Weight BAX ULDs in Comp 1_bax_fixed': comparison_full['Weight BAX ULDs in Comp 1_bax_fixed'].mean(),
            'Number of ULDs in Comp 2_bax_fixed': comparison_full['Number of ULDs in Comp 2_bax_fixed'].mean(),
            'Number of BAX ULDs in Comp 2_bax_fixed': comparison_full['Number of BAX ULDs in Comp 2_bax_fixed'].mean(),
            'Weight ULDs in Comp 2_bax_fixed': comparison_full['Weight ULDs in Comp 2_bax_fixed'].mean(),
            'Weight BAX ULDs in Comp 2_bax_fixed': comparison_full['Weight BAX ULDs in Comp 2_bax_fixed'].mean(),
            'Number of ULDs in Comp 3_bax_fixed': comparison_full['Number of ULDs in Comp 3_bax_fixed'].mean(),
            'Number of BAX ULDs in Comp 3_bax_fixed': comparison_full['Number of BAX ULDs in Comp 3_bax_fixed'].mean(),
            'Weight ULDs in Comp 3_bax_fixed': comparison_full['Weight ULDs in Comp 3_bax_fixed'].mean(),
            'Weight BAX ULDs in Comp 3_bax_fixed': comparison_full['Weight BAX ULDs in Comp 3_bax_fixed'].mean(),
            'Number of ULDs in Comp 4_bax_fixed': comparison_full['Number of ULDs in Comp 4_bax_fixed'].mean(),
            'Number of BAX ULDs in Comp 4_bax_fixed': comparison_full['Number of BAX ULDs in Comp 4_bax_fixed'].mean(),
            'Weight ULDs in Comp 4_bax_fixed': comparison_full['Weight ULDs in Comp 4_bax_fixed'].mean(),
            'Weight BAX ULDs in Comp 4_bax_fixed': comparison_full['Weight BAX ULDs in Comp 4_bax_fixed'].mean()
        }


        average_data_general_MAC = pd.DataFrame([average_data_general_MAC])
        average_data_fuel_deviation = pd.DataFrame([average_data_fuel_deviation])
        sum_data_fuel_deviation = pd.DataFrame([sum_data_fuel_deviation])
        sum_data_full = pd.DataFrame([sum_data_full])
        average_data_full = pd.DataFrame([average_data_full])

        self.combined_results = pd.concat([comparison_full, sum_data_full, average_data_full], ignore_index = True)
        self.combined_results_MAC = pd.concat([comparison_MAC, average_data_general_MAC], ignore_index=True)
        self.combined_results_fuel_deviation = pd.concat([comparison_fuel_deviation, average_data_fuel_deviation, sum_data_fuel_deviation], ignore_index=True)

        return self.combined_results_MAC, self.combined_results_fuel_deviation, self.combined_results        
    

In [18]:
class Data_Analysis():
    def __init__(self):
        self.piece_information_csv = 'Inputfiles/PieceInformationSpotfire.csv'  # Changed from Windows path
        self.buildup_information_csv = 'Inputfiles/BuildUpInformationSpotfire.csv'  # Changed from Windows path
        self.loadlocation_information_csv = 'Inputfiles/LoadLocationsSpotfire.csv'  # Changed from Windows path
       
    def custom_date_parser(self, date_string):
        """
        Parses date strings that are in either 'd/m/YYYY' or 'd-m-YYYY' format.

        Args:
            date_string (str): The date string to parse.
        """
        for fmt in ('%d/%m/%Y', '%d-%m-%Y', '%Y-%m-%d'):
            try:
                return datetime.strptime(date_string, fmt)
            except ValueError:
                continue
        return
        
    def format_date(self, date):
        """
        Formats a datetime object to 'dd MMM YYYY' in uppercase.

        Args:
            date (datetime): The date to format.
        """
        return date.strftime('%d %b %y').upper()

    def dimensions_proportions_per_commodity(self, destination, commodity_code, proportion_type):
        """
        Fetch the average dimension proportion (height, width, or length) for a given commodity code.
        
        Args:
            commodity_code (str): The commodity code to filter by.
            proportion_type (str): The type of proportion to return ('HeightProportion', 'WidthProportion', 'LengthProportion').
        """
        piece_information_data = pd.read_csv(self.piece_information_csv)
        filtered_df = piece_information_data[(piece_information_data['BookingLinePieceIsInformational'] == True) & (piece_information_data['BookingDestinationStationCode'] == destination)].copy()
        filtered_df['BookingLinePieceHeight'] = filtered_df['BookingLinePieceHeight'].str.replace(',', '.').astype(float)
        filtered_df['BookingLinePieceWidth'] = filtered_df['BookingLinePieceWidth'].str.replace(',', '.').astype(float)
        filtered_df['BookingLinePieceLength'] = filtered_df['BookingLinePieceLength'].str.replace(',', '.').astype(float)

        # Calculate total dimensions to normalize each dimension as a proportion
        filtered_df['TotalDimensions'] = filtered_df['BookingLinePieceHeight'] + filtered_df['BookingLinePieceWidth'] + filtered_df['BookingLinePieceLength']
        filtered_df['HeightProportion'] = filtered_df['BookingLinePieceHeight'] / filtered_df['TotalDimensions']
        filtered_df['WidthProportion'] = filtered_df['BookingLinePieceWidth'] / filtered_df['TotalDimensions']
        filtered_df['LengthProportion'] = filtered_df['BookingLinePieceLength'] / filtered_df['TotalDimensions']

        # Group by BookingCommodityCode and calculate the average proportion for each dimension
        grouped_data = filtered_df.groupby('BookingCommodityCode').agg({
            'HeightProportion': 'mean',
            'WidthProportion': 'mean',
            'LengthProportion': 'mean'
        }).reset_index()

        proportion = grouped_data.set_index('BookingCommodityCode').loc[commodity_code, proportion_type]

        return proportion
    
    def threshold_volume_in_AKE(self):
        """  
        Find the volume threshold for items placed in AKE ULDs
        """
        buildup_information_data = pd.read_csv(self.buildup_information_csv)
        filtered_buildup_df = buildup_information_data[buildup_information_data['ULD'].astype(str).str[0:3] == 'AKE'].copy()
        serialnumbers_in_AKE = filtered_buildup_df['AirWaybillNumber'].unique()
        
        piece_information_data = pd.read_csv(self.piece_information_csv)
        volume_per_piece_in_AKE = piece_information_data[piece_information_data['BookingAirWaybillNumber'].isin(serialnumbers_in_AKE)].copy()
        volume_per_piece_in_AKE = volume_per_piece_in_AKE['BookingLinePieceVolume'].str.replace(',', '.').astype(float)
        
        Q1 = volume_per_piece_in_AKE.quantile(0.25)
        Q3 = volume_per_piece_in_AKE.quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        normal_volume_data = volume_per_piece_in_AKE[(volume_per_piece_in_AKE >= lower_bound) &
                                                        (volume_per_piece_in_AKE <= upper_bound)]
        
        volume_threshold = normal_volume_data.quantile(0.75)

        return volume_threshold
    
    def weight_distribution_actual(self):
        """  
        Extract the weight distribution of BAX and ULDs in each compartment
        """
        loadlocations_data = pd.read_csv(self.loadlocation_information_csv)

        data = loadlocations_data[(loadlocations_data['DeadloadType'] == 'B') | (loadlocations_data['DeadloadType'] == 'C')][[
            'FlightLegDepartureKey', 'LoadLocation', 'UldGrossWeight', 'DeadloadType', 'SerialNumber']]
        
        data = data.drop_duplicates(subset=['FlightLegDepartureKey', 'LoadLocation', 'DeadloadType', 'SerialNumber'])
        data = data.drop('SerialNumber', axis=1)
        
        data['Flight Number'] = data['FlightLegDepartureKey'].apply(lambda x: 'KL' + str(int(x.split('|')[2])))
        data['Flight Date'] = pd.to_datetime(data['FlightLegDepartureKey'].apply(lambda x: x.split('|')[0])).dt.strftime('%d %b %y').str.upper()
        data = data.drop('FlightLegDepartureKey', axis=1)
        data['Compartment'] = data['LoadLocation'].str.extract('(\d)').astype(int)  # Extract the first digit and convert to integer

        # Grouping and summing weights by category
        grouped_data = data.groupby(['Flight Number', 'Flight Date', 'DeadloadType', 'Compartment']).agg({'UldGrossWeight': 'sum'}).reset_index()

        # Creating pivot tables for BAX and ULD, filling missing values with 0 for non-existent compartments
        pivot_bax = grouped_data[grouped_data['DeadloadType'] == 'B'].pivot_table(index=['Flight Number', 'Flight Date'],
                         columns='Compartment', values='UldGrossWeight', aggfunc='sum', fill_value=0)
        pivot_bax.columns = [f'Weight BAX ULDs in Comp {int(col)}_actual' for col in pivot_bax.columns]

        pivot_uld = grouped_data[grouped_data['DeadloadType'] == 'C'].pivot_table(index=['Flight Number', 'Flight Date'],
                         columns='Compartment', values='UldGrossWeight', aggfunc='sum', fill_value=0)
        pivot_uld.columns = [f'Weight ULDs in Comp {int(col)}_actual' for col in pivot_uld.columns]

        # Joining the pivoted tables back into a single DataFrame, ensuring all columns are included
        result_data = pd.merge(pivot_bax.reset_index(), pivot_uld.reset_index(), on=['Flight Number', 'Flight Date'], how='outer').fillna(0)
        result_data = result_data.drop(['Weight ULDs in Comp 0_actual', 'Weight ULDs in Comp 5_actual', 'Weight BAX ULDs in Comp 5_actual'], axis = 1)

        result_data['Weight in Comp 1_actual'] = result_data['Weight BAX ULDs in Comp 1_actual'] + result_data['Weight ULDs in Comp 1_actual']
        result_data['Weight in Comp 2_actual'] = result_data['Weight BAX ULDs in Comp 2_actual'] + result_data['Weight ULDs in Comp 2_actual']
        result_data['Weight in Comp 3_actual'] = result_data['Weight BAX ULDs in Comp 3_actual'] + result_data['Weight ULDs in Comp 3_actual']
        result_data['Weight in Comp 4_actual'] = result_data['Weight BAX ULDs in Comp 4_actual'] + result_data['Weight ULDs in Comp 4_actual']

        return result_data
    
    def COL_and_CRT_analysis(self):
        """    
        Find if COL and CRT are present in the flight
        """
        piece_information_data = pd.read_csv(self.piece_information_csv)
        loadlocations_data = pd.read_csv(self.loadlocation_information_csv)

        data_pieces = piece_information_data.copy()
        data_pieces_grouped = data_pieces.groupby(['BookingDestinationStationCode', 'BookingSegmentFlightDateLT'])[['IsCOL', 'IsCRT']].sum().reset_index()
        list_of_destinations = ['LAX', 'SIN', 'ICN', 'IAH']
        data_pieces_grouped = data_pieces_grouped[data_pieces_grouped['BookingDestinationStationCode'].isin(list_of_destinations)]

        data_pieces_grouped['IsCOL'] = data_pieces_grouped['IsCOL'] >= 1
        data_pieces_grouped['IsCRT'] = data_pieces_grouped['IsCRT'] >= 1
        data_pieces_grouped['IsCOLandCRT'] = (data_pieces_grouped['IsCOL']) & (data_pieces_grouped['IsCRT'])

        data_pieces_grouped['BookingSegmentFlightDateLT'] = data_pieces_grouped['BookingSegmentFlightDateLT'].apply(self.custom_date_parser)
        data_pieces_grouped['BookingSegmentFlightDateLT'] = data_pieces_grouped['BookingSegmentFlightDateLT'].apply(self.format_date)
        replacements = {'LAX': 'KL601', 'SIN': 'KL835', 'ICN': 'KL855', 'IAH': 'KL661'}
        data_pieces_grouped['BookingDestinationStationCode'] = data_pieces_grouped['BookingDestinationStationCode'].replace(replacements)

        data_pieces_grouped = data_pieces_grouped.rename(columns={'BookingDestinationStationCode': 'Flight Number', 'BookingSegmentFlightDateLT': 'Flight Date'})

        data_loadlocations = loadlocations_data.copy()
        data_loadlocations['Flight Date'] = data_loadlocations['FlightLegDepartureKey'].apply(lambda x: x.split('|')[0])
        data_loadlocations['Flight Number'] = data_loadlocations['FlightLegDepartureKey'].apply(lambda x: 'KL' + x.split('|')[2][1:4])
        destinations = ['KL601', 'KL835', 'KL855', 'KL661']
        data_loadlocations = data_loadlocations[data_loadlocations['Flight Number'].isin(destinations)]
        data_loadlocations_grouped = data_loadlocations.groupby(['Flight Number', 'Flight Date'])['SpecialHandlingCode'].agg(list).reset_index()
        
        data_loadlocations_grouped['Flight Date'] = data_loadlocations_grouped['Flight Date'].apply(self.custom_date_parser)
        data_loadlocations_grouped['Flight Date'] = data_loadlocations_grouped['Flight Date'].apply(self.format_date)
        data_loadlocations_grouped['IsCOL'] = data_loadlocations_grouped['SpecialHandlingCode'].apply(lambda x: 'COL' in x)
        data_loadlocations_grouped['IsCRT'] = data_loadlocations_grouped['SpecialHandlingCode'].apply(lambda x: 'CRT' in x)
        data_loadlocations_grouped['IsCOLandCRT'] = data_loadlocations_grouped['IsCOL'] & data_loadlocations_grouped['IsCRT']
        data_loadlocations_grouped = data_loadlocations_grouped.drop(['SpecialHandlingCode'], axis=1)

        df_COL_CRT = pd.merge(data_pieces_grouped, data_loadlocations_grouped, on=['Flight Number', 'Flight Date'], how='outer')
        df_COL_CRT['IsCOL'] = df_COL_CRT['IsCOL_x'].fillna(False) | df_COL_CRT['IsCOL_y'].fillna(False)
        df_COL_CRT['IsCRT'] = df_COL_CRT['IsCRT_x'].fillna(False) | df_COL_CRT['IsCRT_y'].fillna(False)
        df_COL_CRT['IsCOLandCRT'] = df_COL_CRT['IsCOLandCRT_x'].fillna(False) | df_COL_CRT['IsCOLandCRT_y'].fillna(False)
        df_COL_CRT = df_COL_CRT.drop(columns=['IsCOL_x', 'IsCOL_y', 'IsCRT_x', 'IsCRT_y', 'IsCOLandCRT_x', 'IsCOLandCRT_y'])

        return df_COL_CRT